In [8]:
# ============================================================================
# SECTION 0: REQUIRED PACKAGES
# ============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import scipy
import copy
from copy import deepcopy
from dsgepy import DSGE
from fredapi import Fred
from sympy import symbols, Matrix
import pickle
from scipy import stats, signal, linalg
from scipy.stats import chi2, invgamma, norm, beta
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from statsmodels.tsa.statespace.mlemodel import MLEModel
from statsmodels.tsa.seasonal import STL
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import scipy.linalg as slinalg
from scipy.linalg import solve_discrete_lyapunov, kron
from scipy.signal import savgol_filter
from darts import TimeSeries
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import TimeSeriesSplit, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from scipy.optimize import minimize, minimize_scalar, differential_evolution
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

In [9]:
# ============================================================================
# STREAMLINED DSGE MODEL - ACCEPTANCE RATE AND PRIORS OUTPUT ONLY
# ============================================================================

def hp_filter(x, lambda_hp=1600):
    """HP filter implementation for proper output gap extraction."""
    T = len(x)
    if T < 4:
        return x, np.zeros(T)
        
    D2 = np.zeros((T-2, T))
    for i in range(T-2):
        D2[i, i] = 1
        D2[i, i+1] = -2
        D2[i, i+2] = 1
    
    I = np.eye(T)
    A = I + lambda_hp * D2.T @ D2
    try:
        trend = np.linalg.solve(A, x)
        cycle = x - trend
        return trend, cycle
    except np.linalg.LinAlgError:
        t = np.arange(T)
        coeffs = np.polyfit(t, x, 1)
        trend = np.polyval(coeffs, t)
        cycle = x - trend
        return trend, cycle

# Monkey-patch for HDF compatibility
class DummyStore:
    def __init__(self, path):
        self.path = path.replace('.h5', '.pkl')
        self.data = {}
    def __setitem__(self, key, value):
        self.data[key] = value
    def __getitem__(self, key):
        return self.data[key]
    def close(self):
        with open(self.path, 'wb') as f:
            pickle.dump(self.data, f)

pd.HDFStore = DummyStore
pd.read_hdf = lambda path, key=None, **kwargs: pickle.load(open(path.replace('.h5','.pkl'), 'rb'))[key]
pd.DataFrame.to_hdf = lambda self, path, key=None, **kwargs: pickle.dump({key: self}, open(path.replace('.h5','.pkl'),'wb'))

# ============================================================================
# DATA PREPARATION
# ============================================================================

# FRED API setup
afkey = 'ada1a8ba0569ed784226f2c26db44ab1'
fred = Fred(api_key=afkey)

start = '1990-01-01'
end = pd.Timestamp.today().strftime('%Y-%m-%d')

print("🔄 Fetching data with realistic shock proxies...")

# ENHANCED series - adding shock proxies but keeping core structure
series = {
    # Original core series
    'gdp': fred.get_series('GDPC1', start, end),
    'cpi': fred.get_series('CPILFESL', start, end), # Using Core CPI
    'fed': fred.get_series('FEDFUNDS', start, end),
    'tfp': fred.get_series('A191RL1Q225SBEA', start, end),
    
    # NEW: Realistic shock proxies
    'sentiment': fred.get_series('UMCSENT', start, end),     # Demand shock proxy
    'oil_price': fred.get_series('DCOILWTICO', start, end),  # Supply shock proxy
}

# Combine and then drop NaNs. This ensures we get the most recent date
# where ALL series have a value.
raw_data = pd.DataFrame(series).resample('Q-DEC').last().dropna()  # Calendar quarters

# Get the actual end date from the cleaned raw_data
end = raw_data.index[-1].strftime('%Y-%m-%d')
print(f"Using sample period: {start} to {end}")
print(f"Raw data shape: {raw_data.shape}")

# ===== TRANSFORMATIONS WITH SHOCK PROXIES =====

# 1. PROPER OUTPUT GAP using HP filter (UNCHANGED)
print("\n🔧 Creating proper output gap using HP filter...")
log_gdp = np.log(raw_data['gdp'])
gdp_trend, output_gap = hp_filter(log_gdp.values, lambda_hp=1600)
output_gap = pd.Series(output_gap, index=raw_data.index)

# 2. INFLATION using Core CPI (UNCHANGED)
print("\n🔧 Using Core CPI for inflation...")
inflation_quarterly = np.log(raw_data['cpi']).diff()

# 3. INTEREST RATE TRANSFORMATION (UNCHANGED)
fed_funds = raw_data['fed']
fed_trend = fed_funds.rolling(window=20, min_periods=10).mean()
fed_trend = fed_trend.bfill().ffill()
interest_gap = fed_funds - fed_trend

# 4. TFP (FIXED SCALING)
print("\n🔧 FIXED: TFP transformation with proper scaling...")
tfp_demeaned = raw_data['tfp'] - raw_data['tfp'].mean()
tfp_scaled = tfp_demeaned / raw_data['tfp'].std()  # Standardize to unit variance

# 5. NEW: Transform shock proxies for later analysis
print("\n🔧 NEW: Processing shock proxies...")
# Demand shock proxy from consumer sentiment
sentiment_detrended = (raw_data['sentiment'] - raw_data['sentiment'].rolling(20).mean()) / raw_data['sentiment'].std()

# Supply shock proxy from oil prices
oil_changes = np.log(raw_data['oil_price']).diff()
oil_shock_proxy = (oil_changes - oil_changes.mean()) / oil_changes.std()

# Combine into dataset (SAME STRUCTURE AS ORIGINAL)
data = pd.DataFrame({
    'y':  output_gap,
    'pi': inflation_quarterly,
    'i':  interest_gap,
    'a':  tfp_demeaned,
    # NEW: Add shock proxies for later analysis
    'demand_proxy': sentiment_detrended,
    'supply_proxy': oil_shock_proxy
})

# Drop missing values *after* all transformations
data = data.dropna()

# Create the df_obs variable used by the forecasting functions (SAME AS ORIGINAL)
df_obs = data[['y', 'pi', 'i']].copy()

print("\n✅ Data ingestion with shock proxies complete.")

# ============================================================================
# DSGE MODEL SPECIFICATION
# ============================================================================

print("\n🏗️  Enhanced DSGE model - adding realistic shocks to working structure...")

# --- Define symbols (ENHANCED but stable) ---
# endogenous variables at t - ADDING demand and supply shocks
y, pi, i, a, d, s, v, exp_y, exp_pi = symbols('y, pi, i, a, d, s, v, exp_y, exp_pi')
endog = Matrix([y, pi, i, a, d, s, v, exp_y, exp_pi])

# endogenous variables at t-1
yl, pil, il, al, dl, sl, vl, exp_yl, exp_pil = symbols('yl, pil, il, al, dl, sl, vl, exp_yl, exp_pil')
endogl = Matrix([yl, pil, il, al, dl, sl, vl, exp_yl, exp_pil])

# exogenous shocks - ENHANCED
eps_a, eps_d, eps_s, eps_v = symbols('eps_a, eps_d, eps_s, eps_v')
exog = Matrix([eps_a, eps_d, eps_s, eps_v])

# expectational shocks (UNCHANGED)
eta_y, eta_pi = symbols('eta_y, eta_pi')
expec = Matrix([eta_y, eta_pi])

# parameters - ENHANCED
sigma, varphi, alpha, beta, theta, phi_pi, phi_y, rho_a, sigma_a, rho_d, sigma_d, rho_s, sigma_s, rho_v, sigma_v = \
    symbols('sigma varphi alpha beta theta phi_pi phi_y rho_a sigma_a rho_d sigma_d rho_s sigma_s rho_v sigma_v')

# summary parameters (UNCHANGED)
psi_nya = (1 + varphi) / (sigma * (1 - alpha) + varphi + alpha)
kappa = (1 - theta) * (1 - theta * beta) * (sigma * (1 - alpha) + varphi + alpha) / theta

# ENHANCED STATE EQUATIONS - using sigma_pi for supply shock to match your original
print("📝 Enhanced state equations with realistic shocks...")

# 1. IS Curve WITH demand shock
eq1 = y - exp_y + (1/sigma)*(i - exp_pi) - psi_nya*(rho_a - 1)*a - d

# 2. Phillips Curve WITH supply shock (using same structure as your original sigma_pi*eps_pi)
eq2 = pi - beta*exp_pi - kappa*y - s

# 3. Taylor Rule WITH monetary policy shock (UNCHANGED)
eq3 = i - phi_pi*pi - phi_y*y - v

# 4. Technology shock (UNCHANGED)
eq4 = a - rho_a*al - sigma_a*eps_a

# 5. NEW: Demand shock process
eq5 = d - rho_d*dl - sigma_d*eps_d

# 6. NEW: Supply shock process
eq6 = s - rho_s*sl - sigma_s*eps_s

# 7. Monetary policy shock (UNCHANGED)
eq7 = v - rho_v*vl - sigma_v*eps_v

# 8-9. Expectational equations (UNCHANGED)
eq8 = y - exp_yl - eta_y
eq9 = pi - exp_pil - eta_pi

equations = Matrix([eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9])

# observation equations (UNCHANGED - CRITICAL FOR STABILITY)
obs01 = y
obs02 = pi
obs03 = i
obs_equations = Matrix([obs01, obs02, obs03])
obs_names = ['Output Gap', 'Inflation', 'Interest Rate']

# ============================================================================
# CONSERVATIVE CALIBRATION
# ============================================================================

def solve_calvo_parameter(target_kappa, sigma_val, varphi_val, alpha_val, beta_val):
    slope_factor = (sigma_val * (1 - alpha_val) + varphi_val + alpha_val)
    
    A = beta_val * slope_factor
    B = -(1 + beta_val) * slope_factor - target_kappa
    C = slope_factor
    
    discriminant = B**2 - 4*A*C
    if discriminant >= 0:
        theta1 = (-B - np.sqrt(discriminant)) / (2*A)
        theta2 = (-B + np.sqrt(discriminant)) / (2*A)
        
        valid_solutions = [theta for theta in [theta1, theta2] if 0 < theta < 1]
        if valid_solutions:
            return valid_solutions[0]
    
    return min(0.99, max(0.01, 1 - target_kappa/slope_factor))

# CALIBRATION - keeping your working values, just adding new shock parameters
print("\n🎛️  Enhanced calibration based on working original:")
print("-" * 50)

target_kappa = 0.10  # Same as your working version
theta_target = solve_calvo_parameter(target_kappa, 1.0, 2.0, 0.33, 0.99)

calib = {
    # ORIGINAL WORKING PARAMETERS (UNCHANGED)
    sigma:   1.0,      
    varphi:  2.0,      
    alpha:   0.33,     
    beta:    0.99,     
    theta:   theta_target,  
    phi_pi:  2.0,      
    phi_y:   0.5,      
    rho_a:   0.9,      
    sigma_a: 0.8,      
    rho_v:   0.3,      
    sigma_v: 0.25,     
    
    # FIXED: Much smaller shock volatilities for numerical stability
    rho_d:   0.5,      # Demand shock persistence
    sigma_d: 0.02,     # Demand shock volatility (VERY SMALL)
    rho_s:   0.4,      # Supply shock persistence  
    sigma_s: 0.01      # Supply shock volatility (TINY)
}

# Instantiate DSGE model for solution check
model = DSGE(
    endog=endog,
    endogl=endogl,
    exog=exog,
    expec=expec,
    state_equations=equations,
    obs_equations=obs_equations,
    obs_names=obs_names,
    calib_dict=calib
)

# Check existence and uniqueness
print(f'\n🔍 Model Solution Check:')
print(f'   Existence & Uniqueness: {model.eu}')

if model.eu != [1, 1]:
    print('   ⚠️  Model solution issues - using more conservative parameters...')
    # More conservative fallback parameters
    calib[sigma_d] = 0.001
    calib[sigma_s] = 0.001
    calib[rho_d] = 0.1
    calib[rho_s] = 0.1
    
    model = DSGE(
        endog=endog, endogl=endogl, exog=exog, expec=expec,
        state_equations=equations, obs_equations=obs_equations,
        obs_names=obs_names, calib_dict=calib
    )
    print(f'   Conservative fallback solution check: {model.eu}')

# ============================================================================
# ENHANCED DSGE MODEL ESTIMATION
# ============================================================================

# Use actual observed data (not simulated) - SAME AS ORIGINAL
df_obs = data[['y', 'pi', 'i']].copy()

# ENHANCED PARAMETER SETUP - keeping your working structure
calib_param = {
    varphi: 2.0,       # SAME AS ORIGINAL
    alpha: 0.33,       # SAME AS ORIGINAL  
    beta: 0.99         # SAME AS ORIGINAL
}

# ENHANCED estimation parameters (adding new shock parameters)
estimate_param = Matrix([
    sigma,     # Risk aversion [ORIGINAL]
    theta,     # Price stickiness [ORIGINAL]
    phi_pi,    # Taylor rule: inflation [ORIGINAL]
    phi_y,     # Taylor rule: output [ORIGINAL]
    rho_a,     # TFP shock persistence [ORIGINAL]
    sigma_a,   # TFP shock volatility [ORIGINAL]
    rho_v,     # Monetary shock persistence [ORIGINAL]
    sigma_v,   # Monetary shock volatility [ORIGINAL]
    rho_d,     # NEW: Demand shock persistence
    sigma_d,   # NEW: Demand shock volatility
    rho_s,     # NEW: Supply shock persistence
    sigma_s    # NEW: Supply shock volatility
])

# FIXED Priors for Bayesian estimation - the key fix!
prior_dict = {
    # ORIGINAL priors (keep working ones)
    sigma:    {'dist': 'normal',   'mean':  1.30, 'std': 0.20, 'label': '$\\sigma$'},
    theta:    {'dist': 'beta',     'mean':  theta_target, 'std': 0.10, 'label': '$\\theta$'},
    phi_pi:   {'dist': 'normal',   'mean':  1.50, 'std': 0.35, 'label': '$\\phi_{\\pi}$'},
    phi_y:    {'dist': 'gamma',    'mean':  0.25, 'std': 0.10, 'label': '$\\phi_{y}$'},
    rho_a:    {'dist': 'beta',     'mean':  0.80, 'std': 0.15, 'label': '$\\rho_a$'},
    sigma_a:  {'dist': 'invgamma', 'mean':  0.50, 'std': 0.25, 'label': '$\\sigma_a$'},
    rho_v:    {'dist': 'beta',     'mean':  0.50, 'std': 0.20, 'label': '$\\rho_v$'},
    sigma_v:  {'dist': 'invgamma', 'mean':  0.25, 'std': 0.15, 'label': '$\\sigma_v$'},
    
    # FIXED: Much more reasonable priors for new shock parameters
    rho_d:    {'dist': 'beta',     'mean':  0.60, 'std': 0.25, 'label': '$\\rho_d$'},
    sigma_d:  {'dist': 'invgamma', 'mean':  0.40, 'std': 0.25, 'label': '$\\sigma_d$'},
    rho_s:    {'dist': 'beta',     'mean':  0.50, 'std': 0.25, 'label': '$\\rho_s$'}, 
    sigma_s:  {'dist': 'invgamma', 'mean':  0.35, 'std': 0.20, 'label': '$\\sigma_s$'}
}

# Instantiate the DSGE model (SAME API as original)
dsge = DSGE(
    endog=endog,
    endogl=endogl,
    exog=exog,
    expec=expec,
    state_equations=equations,
    estimate_params=estimate_param,
    calib_dict=calib_param,
    obs_equations=obs_equations,
    prior_dict=prior_dict,
    obs_data=df_obs,
    obs_names=['y', 'pi', 'i'],
    verbose=True
)

print("\n🎯 Running DSGE estimation for acceptance rate and priors...")

# Run estimation (SAME as original)
dsge.estimate(file_path='enhanced_example_snkm.pkl', nsim=3000, ck=0.3)

print("✅ Estimation completed successfully!")

# Extract results
dsge.eval_chains(burnin=0.1, show_charts=False)

print("\n" + "="*60)
print("DSGE ESTIMATION RESULTS")
print("="*60)

print(f"\n📊 ACCEPTANCE RATE: Available in chain diagnostics")

print(f"\n📋 POSTERIOR ESTIMATES (DSGE-ESTIMATED COEFFICIENT PRIORS):")
print("-" * 60)
print(dsge.posterior_table)

print("\n" + "="*60)
print("ESTIMATION COMPLETE")
print("="*60)

🔄 Fetching data with realistic shock proxies...
Using sample period: 1990-01-01 to 2025-06-30
Raw data shape: (142, 6)

🔧 Creating proper output gap using HP filter...

🔧 Using Core CPI for inflation...

🔧 FIXED: TFP transformation with proper scaling...

🔧 NEW: Processing shock proxies...

✅ Data ingestion with shock proxies complete.

🏗️  Enhanced DSGE model - adding realistic shocks to working structure...
📝 Enhanced state equations with realistic shocks...

🎛️  Enhanced calibration based on working original:
--------------------------------------------------

🔍 Model Solution Check:
   Existence & Uniqueness: [1, 1]

🎯 Running DSGE estimation for acceptance rate and priors...


Metropolis-Hastings: 100%|██████████| 2999/2999 [06:45<00:00,  7.39it/s]


Acceptance rate: 15.5 percent
✅ Estimation completed successfully!

DSGE ESTIMATION RESULTS

📊 ACCEPTANCE RATE: Available in chain diagnostics

📋 POSTERIOR ESTIMATES (DSGE-ESTIMATED COEFFICIENT PRIORS):
------------------------------------------------------------
        prior dist prior mean prior std  posterior mode  posterior mean  \
sigma       normal        1.3       0.2        1.339170        1.355637   
theta         beta   0.837091       0.1        0.154051        0.208095   
phi_pi      normal        1.5      0.35        2.812820        2.772075   
phi_y        gamma       0.25       0.1        0.210660        0.320874   
rho_a         beta        0.8      0.15        0.219071        0.201246   
sigma_a   invgamma        0.5      0.25        0.255921        0.600514   
rho_v         beta        0.5       0.2        0.454664        0.553591   
sigma_v   invgamma       0.25      0.15        0.183908        0.176113   
rho_d         beta        0.6      0.25        0.896433      

In [10]:
# Add to END of Document 1 (after dsge.eval_chains())
def extract_posteriors_for_dfm(dsge_model):
    """Extract DSGE posterior estimates for DFM"""
    posterior_table = dsge_model.posterior_table
    dsge_posteriors = {}
    
    for param_name in posterior_table.index:
        row = posterior_table.loc[param_name]
        posterior_mean = row['posterior mean']
        ci_5 = row['posterior 5.0%'] 
        ci_95 = row['posterior 95.0%']
        posterior_std = (ci_95 - ci_5) / 3.29  # Approx std from 90% CI
        
        dsge_posteriors[param_name] = {
            'mean': posterior_mean,
            'std': posterior_std
        }
    
    return dsge_posteriors

dsge_posteriors = extract_posteriors_for_dfm(dsge)

dsge_posteriors

{'sigma': {'mean': 1.3556370879752022, 'std': 0.20626582964649523},
 'theta': {'mean': 0.20809459869685257, 'std': 0.04422556482937241},
 'phi_pi': {'mean': 2.772075466811264, 'std': 0.07889314492838308},
 'phi_y': {'mean': 0.32087362248079543, 'std': 0.09649278056218742},
 'rho_a': {'mean': 0.2012461230275492, 'std': 0.17840187388143014},
 'sigma_a': {'mean': 0.600514079811355, 'std': 0.32354307673570587},
 'rho_v': {'mean': 0.5535913127783123, 'std': 0.10012655055969089},
 'sigma_v': {'mean': 0.17611315612483597, 'std': 0.01332117734018096},
 'rho_d': {'mean': 0.7458799317478219, 'std': 0.15906394417821443},
 'sigma_d': {'mean': 0.19131571253191917, 'std': 0.087735946611838},
 'rho_s': {'mean': 0.510334723621336, 'std': 0.0858674143111458},
 'sigma_s': {'mean': 0.10276576757863215, 'std': 0.06950924911500318}}

In [11]:
# Core Dynamic Factor Model with DSGE-Informed VAR Priors
# Minimal modification to incorporate DSGE-estimated coefficient priors
import warnings
warnings.filterwarnings('ignore')

# FRED API Setup
afkey = 'ada1a8ba0569ed784226f2c26db44ab1'  # Replace with your FRED API key
fred = Fred(api_key=afkey)

# Economically structured blocks with consistent shock treatment
economic_blocks = {
    'Demand_Block': {
        'trend_cyclical': ['GDPC1', 'PAYEMS', 'INDPRO'],  # Growth rates
        'shock': ['ICSA']  # Level (inverse shock indicator)
    },

    'Supply_Block': {
        'trend_cyclical': ['CPIAUCSL', 'PCEPI', 'AHETPI'],  # Growth rates
        'shock': ['DCOILWTICO']  # Level (supply shock)
    },

    'Monetary_Block': {
        'trend_cyclical': ['FEDFUNDS', 'DGS10', 'M2SL'],  # Mix of levels and growth
        'shock': ['AAA']  # Level (financial stress indicator)
    }
}

def fetch_series_robust(series_id, start_date, fred_api):
    """Robust series fetching with comprehensive error handling"""
    try:
        series = fred_api.get_series(series_id, start_date)
        if series.empty or len(series) < 20:
            print(f"Warning: {series_id} insufficient data")
            return None

        series = series.dropna()
        series = series.to_frame(name=series_id)
        series.index = pd.to_datetime(series.index)
        return series
    except Exception as e:
        print(f"Failed to fetch {series_id}: {e}")
        return None

def prepare_econometric_data(start_date="2000-01-01"):
    """Prepare data with econometrically consistent transformations"""
    print("Fetching economic data from FRED...")

    all_series = {}

    # Fetch all series
    for block_name, block_data in economic_blocks.items():
        print(f"\nFetching {block_name}:")

        for var_type in ['trend_cyclical', 'shock']:
            for series_id in block_data[var_type]:
                data = fetch_series_robust(series_id, start_date, fred)
                if data is not None:
                    all_series[series_id] = data
                    print(f"  {series_id} ✓ ({len(data)} obs)")
                else:
                    print(f"  {series_id} ✗")

    if len(all_series) < 8:  # Need most variables for robust estimation
        raise ValueError(f"Only {len(all_series)} series fetched. Need at least 8.")

    print(f"\nSuccessfully fetched {len(all_series)} series")

    # Find common time range
    all_start_dates = [s.index.min() for s in all_series.values()]
    all_end_dates = [s.index.max() for s in all_series.values()]

    common_start = max(all_start_dates)
    common_end = min(all_end_dates)

    print(f"Common date range: {common_start.strftime('%Y-%m')} to {common_end.strftime('%Y-%m')}")

    # Create quarterly index
    quarterly_index = pd.date_range(
        start=common_start.replace(day=1),
        end=common_end,
        freq='Q'
    )

    print(f"Quarterly observations: {len(quarterly_index)}")

    # Process series with econometrically consistent transformations
    quarterly_data = pd.DataFrame(index=quarterly_index)

    for series_id, series_data in all_series.items():
        series_trimmed = series_data[common_start:common_end].copy()

        # Determine transformation based on economic theory
        if series_id in ['GDPC1', 'PAYEMS', 'INDPRO']:
            # Real activity: QoQ annualized growth rates
            quarterly_last = series_trimmed.resample('Q').last()
            quarterly_series = (quarterly_last.pct_change(1) * 400)  # Annualized QoQ

        elif series_id in ['CPIAUCSL', 'PCEPI', 'AHETPI']:
            # Prices/wages: YoY growth rates (standard for inflation)
            quarterly_last = series_trimmed.resample('Q').last()
            quarterly_series = quarterly_last.pct_change(4) * 100  # YoY %

        elif series_id == 'M2SL':
            # Money supply: YoY growth rate
            quarterly_last = series_trimmed.resample('Q').last()
            quarterly_series = quarterly_last.pct_change(4) * 100  # YoY %

        elif series_id in ['FEDFUNDS', 'DGS10']:
            # Interest rates: levels (already in percentage terms)
            quarterly_series = series_trimmed.resample('Q').last()

        elif series_id == 'ICSA':
            # Initial claims: level (weekly average, inverse demand shock)
            quarterly_series = series_trimmed.resample('Q').mean()
            # Transform to make it a positive demand indicator
            quarterly_series = -np.log(quarterly_series)  # Negative log for inverse relationship

        elif series_id == 'DCOILWTICO':
            # Oil prices: log level (shock variable, variance stabilized)
            quarterly_last = series_trimmed.resample('Q').last()
            quarterly_series = np.log(quarterly_last.replace(0, np.nan))  # Log level

        elif series_id == 'AAA':
            # Credit spread: level (financial stress indicator)
            quarterly_series = series_trimmed.resample('Q').last()

        # Align to quarterly index
        quarterly_data[series_id] = quarterly_series.reindex(quarterly_index)

    # Remove rows with too many missing values
    quarterly_data = quarterly_data.dropna(thresh=len(quarterly_data.columns) * 0.7)

    # Forward fill remaining missing values
    quarterly_data = quarterly_data.fillna(method='ffill').fillna(method='bfill')

    print(f"Final dataset shape: {quarterly_data.shape}")
    print(f"Date range: {quarterly_data.index[0].strftime('%Y-Q%q')} to {quarterly_data.index[-1].strftime('%Y-Q%q')}")

    # Organize into blocks based on what we actually got
    organized_blocks = {}
    for block_name, block_data in economic_blocks.items():
        all_block_vars = block_data['trend_cyclical'] + block_data['shock']
        available_vars = [v for v in all_block_vars if v in quarterly_data.columns]

        if len(available_vars) > 0:
            organized_blocks[block_name] = quarterly_data[available_vars]
            print(f"{block_name}: {available_vars}")
    return organized_blocks, quarterly_data


class EconometricDFM:
    """
    DSGE-Informed Dynamic Factor Model
    
    Key addition: Uses DSGE posterior estimates as Bayesian priors for VAR coefficients
    """

    def __init__(self, organized_blocks, combined_data, dsge_priors=None):
        self.organized_blocks = organized_blocks
        self.combined_data = combined_data.copy()
        
        # NEW: Store DSGE-estimated priors
        self.dsge_priors = dsge_priors

        # Store block structure for identification
        self.block_structure = {}
        var_counter = 0

        for block_name, block_data in organized_blocks.items():
            n_vars = len(block_data.columns)
            self.block_structure[block_name] = {
                'variables': list(block_data.columns),
                'start_idx': var_counter,
                'end_idx': var_counter + n_vars,
                'size': n_vars
            }
            var_counter += n_vars

        self.N_vars = len(self.combined_data.columns)
        self.T_obs = len(self.combined_data)
        self.n_factors = min(3, len(self.block_structure))  # One factor per block

        print(f"\nDSGE-Informed DFM Setup:")
        print(f"  Variables: {self.N_vars}")
        print(f"  Observations: {self.T_obs}")
        print(f"  Factors: {self.n_factors}")
        if dsge_priors is not None:
            print(f"  DSGE priors: {len(dsge_priors)} parameters")

        # Standardize data properly
        self.scaler = StandardScaler()
        self.data_standardized = pd.DataFrame(
            self.scaler.fit_transform(self.combined_data),
            index=self.combined_data.index,
            columns=self.combined_data.columns
        )

        # Define variable scales and types for cross-validation output
        self.variable_info = {
            'GDPC1': {'name': 'GDP (Billions $)', 'typical_scale': 20000, 'type': 'level'},
            'PAYEMS': {'name': 'EMPLOYMENT (Thousands)', 'typical_scale': 150000, 'type': 'level'},
            'INDPRO': {'name': 'INDUSTRIAL PRODUCTION (Index)', 'typical_scale': 100, 'type': 'index'},
            'ICSA': {'name': 'INITIAL CLAIMS (Thousands)', 'typical_scale': 300, 'type': 'level'},
            'CPIAUCSL': {'name': 'INFLATION (Index)', 'typical_scale': 250, 'type': 'index'},
            'PCEPI': {'name': 'PCE INFLATION (Index)', 'typical_scale': 110, 'type': 'index'},
            'AHETPI': {'name': 'WAGES (Dollars/Hour)', 'typical_scale': 25, 'type': 'level'},
            'DCOILWTICO': {'name': 'OIL PRICES ($/Barrel)', 'typical_scale': 60, 'type': 'level'},
            'FEDFUNDS': {'name': 'INTEREST RATE (Percent)', 'typical_scale': 5, 'type': 'rate'},
            'DGS10': {'name': '10Y TREASURY (Percent)', 'typical_scale': 5, 'type': 'rate'},
            'M2SL': {'name': 'MONEY SUPPLY (Billions $)', 'typical_scale': 15000, 'type': 'level'},
            'AAA': {'name': 'CREDIT SPREAD (Percent)', 'typical_scale': 4, 'type': 'rate'}
        }

        # Results storage
        self.factors = None
        self.loadings = None
        self.factor_var = None
        self.idiosyncratic_var = None

    def extract_factors_pca(self):
        """Extract factors using Principal Component Analysis"""
        print("\n" + "="*60)
        print("STEP 1: FACTOR EXTRACTION VIA PCA")
        print("="*60)

        # Apply PCA to standardized data
        pca = PCA(n_components=self.n_factors)
        factors_pca = pca.fit_transform(self.data_standardized)

        # Store factors with economic names
        factor_names = ['Demand_Factor', 'Supply_Factor', 'Monetary_Factor'][:self.n_factors]

        self.factors = pd.DataFrame(
            factors_pca,
            index=self.data_standardized.index,
            columns=factor_names
        )

        # Store loadings (PCA components)
        self.pca_loadings = pd.DataFrame(
            pca.components_.T,
            index=self.combined_data.columns,
            columns=factor_names
        )

        print(f"PCA explained variance: {pca.explained_variance_ratio_}")
        print(f"Cumulative explained variance: {np.cumsum(pca.explained_variance_ratio_)}")

        return self.factors

    def estimate_factor_loadings(self):
        """Estimate factor loadings via regression"""
        print("\n" + "="*60)
        print("STEP 2: FACTOR LOADING ESTIMATION")
        print("="*60)

        if self.factors is None:
            raise ValueError("Must extract factors first")

        # Estimate loadings: X_t = Λ F_t + ε_t
        X = self.data_standardized.values  # T x N
        F = self.factors.values           # T x r

        # OLS: Λ = (F'F)^(-1) F'X'
        FtF_inv = np.linalg.inv(F.T @ F)
        Lambda = (FtF_inv @ F.T @ X).T  # N x r

        # Calculate residuals and fit statistics
        X_fitted = F @ Lambda.T
        residuals = X - X_fitted

        # R-squared for each variable
        r_squared = np.zeros(self.N_vars)
        idiosyncratic_var = np.zeros(self.N_vars)

        for i in range(self.N_vars):
            ss_res = np.sum(residuals[:, i]**2)
            ss_tot = np.sum((X[:, i] - np.mean(X[:, i]))**2)
            r_squared[i] = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
            idiosyncratic_var[i] = np.var(residuals[:, i])

        self.loadings = pd.DataFrame(
            Lambda,
            index=self.combined_data.columns,
            columns=self.factors.columns
        )

        self.idiosyncratic_var = pd.Series(
            idiosyncratic_var,
            index=self.combined_data.columns
        )

        # Print basic results
        print("\nFactor Loading Results:")
        print("=" * 70)
        print(f"{'Variable':<12} {'R²':<6} {'Demand':<10} {'Supply':<10} {'Monetary':<10}")
        print("-" * 70)

        for i, var in enumerate(self.combined_data.columns):
            loadings_row = [Lambda[i, j] if j < Lambda.shape[1] else 0 for j in range(3)]
            print(f"{var:<12} {r_squared[i]:<6.3f} "
                  f"{loadings_row[0]:<10.3f} {loadings_row[1]:<10.3f} {loadings_row[2]:<10.3f}")

        print(f"\nAverage R²: {np.mean(r_squared):.3f}")
        return self.loadings

    def _construct_dsge_priors(self, optimal_lag):
        """
        NEW METHOD: Construct Bayesian VAR priors from DSGE estimates
        
        Maps DSGE structural parameters to reduced-form VAR coefficient priors
        """
        if self.dsge_priors is None:
            return None
            
        print(f"\n📋 Constructing DSGE-informed priors for VAR({optimal_lag})...")
        
        K = self.n_factors
        n_coeffs = K * K * optimal_lag + K  # VAR coeffs + constants
        
        # Extract key DSGE parameters (using median/mode from posterior)
        dsge_params = {}
        for param_name in ['sigma', 'theta', 'phi_pi', 'phi_y', 'rho_a', 'rho_v']:
            if param_name in self.dsge_priors:
                # Get posterior mean as point estimate
                dsge_params[param_name] = self.dsge_priors[param_name].get('mean', 1.0)
        
        # Construct theory-consistent priors
        prior_mean = np.zeros(n_coeffs)
        prior_var = np.ones(n_coeffs)
        
        # Constants (intercepts) - weakly informative
        prior_var[:K] = 1.0
        
        # VAR coefficient priors based on DSGE structure
        coeff_start = K
        
        for lag in range(1, optimal_lag + 1):
            for i in range(K):  # Each factor equation
                for j in range(K):  # Each factor lag
                    coeff_idx = coeff_start + (lag-1)*K*K + i*K + j
                    
                    if i == j:  # Own lags
                        if lag == 1:
                            # First own lag - use DSGE persistence parameters
                            if i == 0:  # Demand factor - related to output persistence
                                prior_mean[coeff_idx] = dsge_params.get('rho_a', 0.5) * 0.7
                                prior_var[coeff_idx] = 0.1
                            elif i == 1:  # Supply factor - related to price stickiness
                                stickiness = dsge_params.get('theta', 0.7)
                                prior_mean[coeff_idx] = min(0.9, stickiness * 1.2)
                                prior_var[coeff_idx] = 0.1
                            else:  # Monetary factor - use monetary persistence
                                prior_mean[coeff_idx] = dsge_params.get('rho_v', 0.3)
                                prior_var[coeff_idx] = 0.15
                        else:
                            # Higher order own lags - decay
                            prior_mean[coeff_idx] = prior_mean[coeff_start + (i*K + j)] * 0.5
                            prior_var[coeff_idx] = 0.2
                    else:  # Cross-lags
                        if lag == 1:
                            # Theory-based cross-effects
                            if (i == 0 and j == 2):  # Monetary -> Demand (IS curve)
                                prior_mean[coeff_idx] = -1.0 / dsge_params.get('sigma', 1.0) * 0.1
                                prior_var[coeff_idx] = 0.05
                            elif (i == 1 and j == 0):  # Demand -> Supply (Phillips curve)  
                                prior_mean[coeff_idx] = dsge_params.get('phi_y', 0.5) * 0.2
                                prior_var[coeff_idx] = 0.05
                            elif (i == 2 and j == 1):  # Supply -> Monetary (Taylor rule)
                                prior_mean[coeff_idx] = dsge_params.get('phi_pi', 1.5) * 0.1
                                prior_var[coeff_idx] = 0.05
                            else:
                                prior_mean[coeff_idx] = 0.0
                                prior_var[coeff_idx] = 0.1
                        else:
                            # Higher order cross-lags - shrink to zero
                            prior_mean[coeff_idx] = 0.0
                            prior_var[coeff_idx] = 0.1 / lag
        
        print(f"   Prior mean range: [{np.min(prior_mean):.3f}, {np.max(prior_mean):.3f}]")
        print(f"   Prior variance range: [{np.min(prior_var):.3f}, {np.max(prior_var):.3f}]")
        
        return {
            'mean': prior_mean,
            'variance': prior_var,
            'dsge_params': dsge_params
        }

    def estimate_factor_var(self, max_lags=6, ic='bic'):
        """
        MODIFIED: Estimate VAR model with optional DSGE-informed priors
        """
        print("\n" + "="*60)
        print("STEP 3: DSGE-INFORMED FACTOR VAR ESTIMATION")
        print("="*60)

        if self.factors is None:
            raise ValueError("Must extract factors first")

        F = self.factors.values
        T, K = F.shape

        # Lag selection using information criteria
        optimal_lag = self._select_optimal_lag(F, max_lags, ic)
        print(f"Optimal lag selected: {optimal_lag} (using {ic.upper()})")

        # Construct DSGE priors if available
        dsge_prior_info = self._construct_dsge_priors(optimal_lag)
        
        # Estimate VAR model (Bayesian if priors available, OLS otherwise)
        if dsge_prior_info is not None:
            print("🎯 Using DSGE-informed Bayesian estimation")
            self.factor_var = self._estimate_bayesian_var(F, optimal_lag, dsge_prior_info)
        else:
            print("📊 Using standard OLS estimation")
            self.factor_var = self._estimate_var_model(F, optimal_lag)

        # Check model stability
        self._check_var_stability()

        return self.factor_var

    def _estimate_bayesian_var(self, data, lags, prior_info, verbose=True):
        """
        NEW METHOD: Bayesian VAR estimation with DSGE-informed priors
        """
        T, K = data.shape
        
        # Construct regression matrices
        Y = data[lags:, :]  # Dependent variables
        T_eff = Y.shape[0]
        
        # Design matrix
        X = np.ones((T_eff, 1))  # Constant
        for lag in range(1, lags + 1):
            X = np.hstack([X, data[lags-lag:-lag, :]])
        
        n_regressors = X.shape[1]
        
        # Prior parameters
        beta_prior = prior_info['mean'].reshape(-1, 1)
        V_prior = np.diag(prior_info['variance'])
        
        # Bayesian updating
        V_prior_inv = np.linalg.inv(V_prior)
        XtX = X.T @ X
        
        # Posterior covariance for coefficients
        V_posterior_inv = V_prior_inv + np.kron(np.eye(K), XtX)
        V_posterior = np.linalg.inv(V_posterior_inv)
        
        # Posterior mean for coefficients  
        XtY = X.T @ Y
        beta_posterior_vec = V_posterior @ (V_prior_inv @ beta_prior + XtY.flatten('F').reshape(-1, 1))
        beta_posterior = beta_posterior_vec.reshape((n_regressors, K), order='F')
        
        # Residual sum of squares (for residual covariance)
        Y_fitted = X @ beta_posterior
        residuals = Y - Y_fitted
        
        # Residual covariance matrix (ML estimate)
        sigma_u = (residuals.T @ residuals) / T_eff
        
        # Coefficient matrices
        const = beta_posterior[0, :]
        var_coeffs = beta_posterior[1:, :].T
        
        # Model statistics
        log_likelihood = self._compute_log_likelihood(residuals, sigma_u)
        
        if verbose:
            print(f"Bayesian VAR({lags}) estimation completed:")
            print(f"  Effective sample: {T_eff}")
            print(f"  Log-likelihood: {log_likelihood:.2f}")
            print(f"  DSGE prior influence: {'High' if np.max(prior_info['variance']) < 0.5 else 'Moderate'}")
        
        return {
            'coefficients': var_coeffs,
            'constant': const, 
            'sigma_u': sigma_u,
            'residuals': residuals,
            'log_likelihood': log_likelihood,
            'lags': lags,
            'T_eff': T_eff,
            'X_design': X,
            'Y_data': Y,
            'bayesian': True,
            'prior_info': prior_info,
            'posterior_beta': beta_posterior,
            'posterior_cov': V_posterior
        }

    def _select_optimal_lag(self, data, max_lags, criterion='bic'):
        """Select optimal VAR lag using information criteria"""
        T, K = data.shape
        max_feasible_lags = min(max_lags, T // (4 * K))  # Ensure sufficient degrees of freedom

        ic_values = []

        for p in range(1, max_feasible_lags + 1):
            try:
                var_result = self._estimate_var_model(data, p, verbose=False)
                T_eff = T - p

                # Calculate information criteria
                log_det_sigma = np.log(np.linalg.det(var_result['sigma_u']))
                n_params = K + K**2 * p  # Constants + VAR coefficients

                if criterion.lower() == 'aic':
                    ic = log_det_sigma + (2 * n_params) / T_eff
                elif criterion.lower() == 'bic':
                    ic = log_det_sigma + (n_params * np.log(T_eff)) / T_eff
                elif criterion.lower() == 'hqic':
                    ic = log_det_sigma + (2 * n_params * np.log(np.log(T_eff))) / T_eff
                else:
                    raise ValueError("Criterion must be 'aic', 'bic', or 'hqic'")

                ic_values.append((p, ic))

            except np.linalg.LinAlgError:
                continue

        if not ic_values:
            print(f"Warning: Could not estimate VAR models. Using lag=1")
            return 1

        # Select lag with minimum IC
        optimal_lag = min(ic_values, key=lambda x: x[1])[0]
        return optimal_lag

    def _estimate_var_model(self, data, lags, verbose=True):
        """Estimate VAR(p) model using OLS"""
        T, K = data.shape

        if T <= lags * K + K:
            raise ValueError(f"Insufficient observations for VAR({lags})")

        # Construct regression matrices
        Y = data[lags:, :]  # Dependent variables (T-p) x K
        T_eff = Y.shape[0]

        # Construct regressor matrix X
        X = np.ones((T_eff, 1))  # Constant term

        for lag in range(1, lags + 1):
            X = np.hstack([X, data[lags-lag:-lag, :]])  # Add lagged factors

        # OLS estimation: Y = X β + u
        XtX_inv = np.linalg.inv(X.T @ X)
        beta = XtX_inv @ X.T @ Y  # (1 + K*p) x K

        # Residuals and covariance matrix
        u = Y - X @ beta  # Residuals
        sigma_u = (u.T @ u) / T_eff  # Residual covariance matrix

        # Coefficient matrices
        const = beta[0, :]  # Constant terms
        var_coeffs = beta[1:, :].T  # K x (K*p) VAR coefficient matrix

        # Standard errors (for diagnostics)
        var_beta = np.kron(sigma_u, XtX_inv)
        se_beta = np.sqrt(np.diag(var_beta)).reshape(beta.shape)

        # Model statistics
        log_likelihood = self._compute_log_likelihood(u, sigma_u)

        if verbose:
            print(f"VAR({lags}) estimation completed:")
            print(f"  Effective sample: {T_eff}")
            print(f"  Log-likelihood: {log_likelihood:.2f}")

        return {
            'coefficients': var_coeffs,      # K x (K*p)
            'constant': const,               # K x 1
            'sigma_u': sigma_u,             # K x K residual covariance
            'residuals': u,                 # (T-p) x K
            'standard_errors': se_beta,     # Full parameter standard errors
            'log_likelihood': log_likelihood,
            'lags': lags,
            'T_eff': T_eff,
            'X_design': X,                  # Design matrix
            'Y_data': Y,                    # Dependent variable matrix
            'bayesian': False
        }

    def _compute_log_likelihood(self, residuals, sigma_u):
        """Compute log-likelihood for VAR model"""
        T, K = residuals.shape
        log_likelihood = -0.5 * T * K * np.log(2 * np.pi)
        log_likelihood -= 0.5 * T * np.log(np.linalg.det(sigma_u))
        log_likelihood -= 0.5 * np.trace(residuals.T @ residuals @ np.linalg.inv(sigma_u))
        return log_likelihood

    def _check_var_stability(self):
        """Check VAR model stability via companion matrix eigenvalues"""
        if self.factor_var is None:
            return False

        Phi = self.factor_var['coefficients']
        p = self.factor_var['lags']
        K = Phi.shape[0]

        # Construct companion matrix
        if p == 1:
            companion = Phi
        else:
            # For VAR(p): K*p x K*p companion matrix
            companion = np.zeros((K * p, K * p))
            companion[:K, :] = Phi  # Top block: VAR coefficients

            # Identity blocks for lagged terms
            for i in range(1, p):
                start_row, end_row = i * K, (i + 1) * K
                start_col, end_col = (i - 1) * K, i * K
                companion[start_row:end_row, start_col:end_col] = np.eye(K)

        # Check stability condition
        eigenvals = np.linalg.eigvals(companion)
        max_modulus = np.max(np.abs(eigenvals))
        is_stable = max_modulus < 1.0

        estimation_type = "Bayesian" if self.factor_var.get('bayesian', False) else "OLS"
        print(f"\n{estimation_type} VAR Stability Check:")
        print(f"  Maximum eigenvalue modulus: {max_modulus:.4f}")
        print(f"  Model is {'stable' if is_stable else 'unstable'} {'✓' if is_stable else '✗'}")

        self.companion_matrix = companion
        self.eigenvalues = eigenvals
        return is_stable

    def forecast_var(self, horizon=8, confidence_level=0.95):
        """Generate VAR forecasts using companion form"""
        print("\n" + "="*60)
        print("VAR FORECASTING WITH COMPANION FORM")
        print("="*60)

        if self.factor_var is None:
            raise ValueError("Must estimate VAR model first")

        # Extract VAR parameters
        Phi = self.factor_var['coefficients']    # K x (K*p)
        c = self.factor_var['constant']          # K x 1
        Sigma = self.factor_var['sigma_u']       # K x K
        p = self.factor_var['lags']
        K = Phi.shape[0]

        estimation_type = "Bayesian" if self.factor_var.get('bayesian', False) else "OLS"
        print(f"Forecasting {K} factors for {horizon} periods using {estimation_type} VAR({p})")

        # Analytical forecasting using companion form
        forecasts_analytical = self._forecast_companion_form(horizon)

        # Bootstrap forecasting for confidence intervals
        forecasts_bootstrap = self._forecast_bootstrap(horizon, n_bootstrap=500, confidence_level=confidence_level)

        # Combine results
        forecast_dates = pd.date_range(
            start=self.factors.index[-1] + pd.DateOffset(months=3),
            periods=horizon,
            freq='Q'
        )

        self.factor_forecasts = {
            'mean': pd.DataFrame(
                forecasts_analytical['mean'],
                index=forecast_dates,
                columns=self.factors.columns
            ),
            'analytical_std': pd.DataFrame(
                forecasts_analytical['std'],
                index=forecast_dates,
                columns=self.factors.columns
            ),
            'bootstrap_lower': pd.DataFrame(
                forecasts_bootstrap['lower'],
                index=forecast_dates,
                columns=self.factors.columns
            ),
            'bootstrap_upper': pd.DataFrame(
                forecasts_bootstrap['upper'],
                index=forecast_dates,
                columns=self.factors.columns
            )
        }

        print(f"\nForecast Summary (1-quarter ahead):")
        print("-" * 50)
        for i, factor in enumerate(self.factors.columns):
            mean_fcst = forecasts_analytical['mean'][0, i]
            std_fcst = forecasts_analytical['std'][0, i]
            print(f"{factor}: {mean_fcst:7.3f} ± {std_fcst:.3f}")

        return self.factor_forecasts

    def _forecast_companion_form(self, horizon):
        """Analytical VAR forecasting using companion matrix"""
        Phi = self.factor_var['coefficients']
        c = self.factor_var['constant']
        Sigma = self.factor_var['sigma_u']
        p = self.factor_var['lags']
        K = Phi.shape[0]

        # Initialize forecast arrays
        forecasts = np.zeros((horizon, K))
        forecast_variance = np.zeros((horizon, K))

        if p == 1:
            # VAR(1) case
            for h in range(horizon):
                if h == 0:
                    forecasts[h, :] = c + Phi @ self.factors.values[-1, :]
                    forecast_variance[h, :] = np.diag(Sigma)
                else:
                    forecasts[h, :] = c + Phi @ forecasts[h-1, :]
                    Phi_h = np.linalg.matrix_power(Phi, h)
                    forecast_variance[h, :] = np.diag(Phi_h @ Sigma @ Phi_h.T)
        else:
            # VAR(p) case - use companion form
            companion = self.companion_matrix
            K_comp = companion.shape[0]

            # Get initial conditions
            F_init = self.factors.values[-p:, :].flatten('F')
            z_init = np.zeros(K_comp)
            z_init[:K*p] = F_init

            c_comp = np.zeros(K_comp)
            c_comp[:K] = c

            Sigma_comp = np.zeros((K_comp, K_comp))
            Sigma_comp[:K, :K] = Sigma

            z_forecast = z_init.copy()

            for h in range(horizon):
                z_forecast = c_comp + companion @ z_forecast
                forecasts[h, :] = z_forecast[:K]

                if h == 0:
                    forecast_variance[h, :] = np.diag(Sigma)
                else:
                    cum_var = np.zeros((K, K))
                    for j in range(h + 1):
                        A_j = np.linalg.matrix_power(companion, j)
                        cum_var += A_j[:K, :K] @ Sigma @ A_j[:K, :K].T
                    forecast_variance[h, :] = np.diag(cum_var)

        return {
            'mean': forecasts,
            'std': np.sqrt(forecast_variance)
        }

    def _forecast_bootstrap(self, horizon, n_bootstrap=500, confidence_level=0.95):
        """Bootstrap VAR forecasting for confidence intervals"""
        Phi = self.factor_var['coefficients']
        c = self.factor_var['constant']
        residuals = self.factor_var['residuals']
        p = self.factor_var['lags']
        K = Phi.shape[0]

        bootstrap_paths = np.zeros((n_bootstrap, horizon, K))

        for b in range(n_bootstrap):
            boot_indices = np.random.choice(len(residuals), size=horizon, replace=True)
            boot_residuals = residuals[boot_indices, :]

            if p == 1:
                F_last = self.factors.values[-1, :].copy()
                for h in range(horizon):
                    F_forecast = c + Phi @ F_last + boot_residuals[h, :]
                    bootstrap_paths[b, h, :] = F_forecast
                    F_last = F_forecast
            else:
                F_history = self.factors.values[-p:, :].copy()
                for h in range(horizon):
                    F_lagged = F_history.flatten('F')
                    F_forecast = c + Phi @ F_lagged + boot_residuals[h, :]
                    bootstrap_paths[b, h, :] = F_forecast
                    F_history = np.vstack([F_history[1:, :], F_forecast.reshape(1, -1)])

        # Calculate statistics
        alpha = 1 - confidence_level
        lower_pct = 100 * (alpha / 2)
        upper_pct = 100 * (1 - alpha / 2)

        return {
            'lower': np.percentile(bootstrap_paths, lower_pct, axis=0),
            'upper': np.percentile(bootstrap_paths, upper_pct, axis=0)
        }

    def generate_variable_forecasts(self, horizon=8):
        """Generate forecasts for all observable variables using factor forecasts"""
        print("\n" + "="*60)
        print("VARIABLE FORECASTING VIA FACTOR MODEL")
        print("="*60)

        if not hasattr(self, 'factor_forecasts'):
            self.forecast_var(horizon)

        if self.loadings is None:
            raise ValueError("Must estimate factor loadings first")

        Lambda = self.loadings.values  # N x K
        F_forecasts = self.factor_forecasts['mean'].values  # horizon x K
        X_forecasts_std = F_forecasts @ Lambda.T  # horizon x N

        # Transform back to original scale
        X_forecasts = self.scaler.inverse_transform(X_forecasts_std)

        # Create forecast DataFrames
        forecast_dates = self.factor_forecasts['mean'].index

        variable_forecasts = pd.DataFrame(
            X_forecasts,
            index=forecast_dates,
            columns=self.combined_data.columns
        )

        self.variable_forecasts = {
            'mean': variable_forecasts
        }

        print(f"Generated forecasts for {len(self.combined_data.columns)} variables")
        return self.variable_forecasts

    def perform_cross_validation(self, k_folds=5, forecast_horizon=4):
        """Perform k-fold cross-validation and collect comprehensive fold-level metrics"""
        print("\n" + "="*100)
        print("5-FOLD CROSS-VALIDATION WITH COMPREHENSIVE METRICS")
        print("="*100)
        
        T = len(self.combined_data)
        fold_size = T // k_folds
        min_train_size = max(20, self.n_factors * 4)  # Ensure sufficient training data
        
        # Store metrics for each fold and each variable
        fold_metrics = {var: [] for var in self.combined_data.columns}
        
        print(f"Total observations: {T}")
        print(f"Fold size: {fold_size}")
        print(f"Forecast horizon: {forecast_horizon}")
        
        for fold in range(k_folds):
            print(f"\nProcessing Fold {fold + 1}/{k_folds}...")
            
            # Define train/test split for time series (expanding window)
            test_start = min_train_size + fold * fold_size
            test_end = min(test_start + forecast_horizon, T)
            train_end = test_start
            
            if test_end > T or train_end < min_train_size:
                print(f"  Skipping fold {fold + 1} - insufficient data")
                continue
                
            print(f"  Train: {0} to {train_end-1} ({train_end} obs)")
            print(f"  Test: {test_start} to {test_end-1} ({test_end - test_start} obs)")
            
            try:
                # Create training data
                train_data = self.combined_data.iloc[:train_end].copy()
                test_data = self.combined_data.iloc[test_start:test_end].copy()
                
                # Create temporary model for this fold
                temp_organized_blocks = {}
                for block_name, block_data in self.organized_blocks.items():
                    temp_organized_blocks[block_name] = block_data.iloc[:train_end]
                
                # Create and train fold model - PASS DSGE PRIORS TO EACH FOLD
                fold_model = EconometricDFM(temp_organized_blocks, train_data, dsge_priors=self.dsge_priors)
                fold_model.extract_factors_pca()
                fold_model.estimate_factor_loadings()
                fold_model.estimate_factor_var()
                
                # Generate forecasts
                fold_model.forecast_var(horizon=len(test_data))
                fold_model.generate_variable_forecasts(horizon=len(test_data))
                
                # Store results for each variable
                forecasts = fold_model.variable_forecasts['mean']
                
                for var in self.combined_data.columns:
                    if var in forecasts.columns and var in test_data.columns:
                        var_forecasts = forecasts[var].values[:len(test_data)]
                        var_actuals = test_data[var].values
                        
                        # For Theil's U2, we need naive no-change forecast
                        # Use the last training observation as naive forecast
                        var_naive = np.full_like(var_actuals, train_data[var].iloc[-1])
                        
                        # Remove any NaN values
                        valid_idx = ~(np.isnan(var_forecasts) | np.isnan(var_actuals))
                        if valid_idx.sum() > 0:
                            # Calculate metrics for this fold
                            fold_metric = self._calculate_comprehensive_forecast_metrics(
                                var_forecasts[valid_idx], 
                                var_actuals[valid_idx], 
                                var_naive[valid_idx],
                                var
                            )
                            fold_metrics[var].append(fold_metric)
                
                print(f"  Fold {fold + 1} completed successfully")
                
            except Exception as e:
                print(f"  Fold {fold + 1} failed: {e}")
                continue
        
        # Calculate statistics across folds for each variable
        self.cv_fold_statistics = {}
        for var in self.combined_data.columns:
            if len(fold_metrics[var]) > 0:
                self.cv_fold_statistics[var] = self._calculate_fold_statistics(fold_metrics[var])
        
        return self.cv_fold_statistics

    def _calculate_comprehensive_forecast_metrics(self, forecasts, actuals, naive_forecasts, var_name):
        """Calculate all comprehensive forecast accuracy metrics for a single fold"""
        # Basic error metrics
        errors = forecasts - actuals
        abs_errors = np.abs(errors)
        
        # Naive forecast errors for Theil's U2
        naive_errors = naive_forecasts - actuals
        
        # Core metrics
        rmse = np.sqrt(np.mean(errors**2))
        mae = np.mean(abs_errors)
        
        # Percentage errors (handle division by zero)
        with np.errstate(divide='ignore', invalid='ignore'):
            # MAPE - Mean Absolute Percentage Error
            epsilon = 1e-8
            mape = np.mean(np.abs(errors / np.maximum(np.abs(actuals), epsilon))) * 100
            
            # SMAPE - Symmetric Mean Absolute Percentage Error  
            denominator = (np.abs(forecasts) + np.abs(actuals)) / 2
            smape = np.mean(np.abs(errors) / np.maximum(denominator, epsilon)) * 100
            
            # MdAPE - Median Absolute Percentage Error
            ape = np.abs(errors / np.maximum(np.abs(actuals), epsilon)) * 100
            ape_finite = ape[np.isfinite(ape)]
            mdape = np.median(ape_finite) if len(ape_finite) > 0 else np.inf
        
        # Theil's U1 statistic (Inequality Coefficient)
        if len(actuals) > 0:
            mean_actual_sq = np.mean(actuals**2)
            mean_forecast_sq = np.mean(forecasts**2)
            
            if mean_actual_sq > 0 and mean_forecast_sq > 0:
                theil_u1 = rmse / np.sqrt(mean_actual_sq + mean_forecast_sq)
            else:
                theil_u1 = np.nan
        else:
            theil_u1 = np.nan
        
        # Theil's U2 statistic (Relative RMSE vs naive no-change forecast)
        naive_rmse = np.sqrt(np.mean(naive_errors**2))
        if naive_rmse > 0:
            theil_u2 = rmse / naive_rmse
        else:
            theil_u2 = np.nan
        
        # MASE - Mean Absolute Scaled Error
        if len(actuals) > 1:
            # Scale by naive seasonal forecast (here just first differences)
            naive_errors_seas = np.abs(np.diff(actuals))
            if len(naive_errors_seas) > 0 and np.mean(naive_errors_seas) > 0:
                mase = mae / np.mean(naive_errors_seas)
            else:
                mase = np.inf
        else:
            mase = np.inf
        
        # R-squared
        ss_res = np.sum(errors**2)
        ss_tot = np.sum((actuals - np.mean(actuals))**2)
        r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
        
        return {
            'rmse': rmse,
            'mae': mae,
            'mape': mape,
            'smape': smape,
            'theil_u1': theil_u1,
            'theil_u2': theil_u2,
            'mdape': mdape,
            'mase': mase,
            'r2': r2
        }

    def _calculate_fold_statistics(self, fold_metrics_list):
        """Calculate mean across folds for each metric"""
        metrics_dict = {}
        
        # Get all metric names
        metric_names = fold_metrics_list[0].keys()
        
        for metric_name in metric_names:
            values = [fold_metrics[metric_name] for fold_metrics in fold_metrics_list]
            values = np.array(values)
            
            # Handle infinite values
            finite_values = values[np.isfinite(values)]
            
            if len(finite_values) > 0:
                metrics_dict[metric_name] = np.mean(finite_values)
            else:
                metrics_dict[metric_name] = np.nan
        
        return metrics_dict

    def print_cv_summary(self):
        """Print cross-validation summary with comprehensive metrics"""
        if not hasattr(self, 'cv_fold_statistics'):
            print("No cross-validation results available. Run perform_cross_validation() first.")
            return
        
        print("\n" + "="*140)
        print("DSGE-INFORMED DFM CROSS-VALIDATION SUMMARY")
        print("="*140)
        
        # Print header
        print(f"{'Variable':<15} {'CV RMSE':<10} {'CV MAE':<10} {'CV MAPE':<10} {'CV SMAPE':<10} {'CV Theil_U1':<12} {'CV Theil_U2':<12} {'CV MdAPE':<10} {'CV MASE':<10} {'CV R²':<8}")
        print("-" * 140)
        
        # Print results for each variable
        for var in self.combined_data.columns:
            if var not in self.cv_fold_statistics:
                continue
                
            var_stats = self.cv_fold_statistics[var]
            
            # Format values, handling NaN and inf
            rmse_str = f"{var_stats['rmse']:.4f}" if np.isfinite(var_stats['rmse']) else "NaN"
            mae_str = f"{var_stats['mae']:.4f}" if np.isfinite(var_stats['mae']) else "NaN"
            mape_str = f"{var_stats['mape']:.2f}" if np.isfinite(var_stats['mape']) else "NaN"
            smape_str = f"{var_stats['smape']:.2f}" if np.isfinite(var_stats['smape']) else "NaN"
            theil_u1_str = f"{var_stats['theil_u1']:.4f}" if np.isfinite(var_stats['theil_u1']) else "NaN"
            theil_u2_str = f"{var_stats['theil_u2']:.4f}" if np.isfinite(var_stats['theil_u2']) else "NaN"
            mdape_str = f"{var_stats['mdape']:.2f}" if np.isfinite(var_stats['mdape']) else "NaN"
            mase_str = f"{var_stats['mase']:.4f}" if np.isfinite(var_stats['mase']) else "NaN"
            r2_str = f"{var_stats['r2']:.3f}" if np.isfinite(var_stats['r2']) else "NaN"
            
            print(f"{var:<15} {rmse_str:<10} {mae_str:<10} {mape_str:<10} {smape_str:<10} "
                  f"{theil_u1_str:<12} {theil_u2_str:<12} {mdape_str:<10} {mase_str:<10} {r2_str:<8}")
        
        # Summary statistics
        print("\n" + "="*70)
        print("DSGE-INFORMED MODEL SUMMARY STATISTICS")
        print("="*70)
        
        # Calculate averages across variables
        valid_rmse = [stats['rmse'] for stats in self.cv_fold_statistics.values() 
                     if np.isfinite(stats['rmse'])]
        valid_mae = [stats['mae'] for stats in self.cv_fold_statistics.values() 
                    if np.isfinite(stats['mae'])]
        valid_r2 = [stats['r2'] for stats in self.cv_fold_statistics.values() 
                   if np.isfinite(stats['r2'])]
        valid_theil_u1 = [stats['theil_u1'] for stats in self.cv_fold_statistics.values() 
                          if np.isfinite(stats['theil_u1'])]
        valid_theil_u2 = [stats['theil_u2'] for stats in self.cv_fold_statistics.values() 
                          if np.isfinite(stats['theil_u2'])]
        valid_mase = [stats['mase'] for stats in self.cv_fold_statistics.values() 
                     if np.isfinite(stats['mase'])]
        
        if valid_rmse:
            print(f"Average CV RMSE: {np.mean(valid_rmse):.4f}")
            print(f"Median CV RMSE: {np.median(valid_rmse):.4f}")
        if valid_mae:
            print(f"Average CV MAE: {np.mean(valid_mae):.4f}")
        if valid_r2:
            print(f"Average CV R²: {np.mean(valid_r2):.3f}")
        if valid_theil_u1:
            print(f"Average CV Theil's U1: {np.mean(valid_theil_u1):.4f}")
        if valid_theil_u2:
            print(f"Average CV Theil's U2: {np.mean(valid_theil_u2):.4f}")
        if valid_mase:
            print(f"Average CV MASE: {np.mean(valid_mase):.4f}")
            print(f"Median CV MASE: {np.median(valid_mase):.4f}")
            
        print(f"Variables with valid metrics: {len(self.cv_fold_statistics)}/{len(self.combined_data.columns)}")
        
        # Show DSGE influence
        if self.dsge_priors is not None:
            print(f"DSGE priors used: {len(self.dsge_priors)} structural parameters")
            print("Model uses Bayesian VAR with DSGE-informed coefficient restrictions")

    def run_complete_estimation(self, forecast_horizon=8, run_cv=True):
        """Run complete DSGE-informed DFM estimation pipeline"""
        print("DSGE-INFORMED DYNAMIC FACTOR MODEL - COMPLETE ESTIMATION")
        print("=" * 80)

        try:
            # Core estimation steps
            self.extract_factors_pca()
            self.estimate_factor_loadings()
            self.estimate_factor_var()

            # Forecasting
            factor_forecasts = self.forecast_var(forecast_horizon)
            variable_forecasts = self.generate_variable_forecasts(forecast_horizon)

            # Cross-validation
            if run_cv:
                cv_results = self.perform_cross_validation(k_folds=5, forecast_horizon=4)
                self.print_cv_summary()

            print("\n" + "="*80)
            print("DSGE-INFORMED DFM ESTIMATION COMPLETED SUCCESSFULLY")
            print("="*80)

            results = {
                'factors': self.factors,
                'loadings': self.loadings,
                'factor_var': self.factor_var,
                'factor_forecasts': factor_forecasts,
                'variable_forecasts': variable_forecasts
            }
            
            if run_cv:
                results['cv_results'] = cv_results

            return results

        except Exception as e:
            print(f"Error in DSGE-informed DFM estimation: {e}")
            import traceback
            traceback.print_exc()
            return None


def run_econometric_dfm(dsge_posterior_estimates, start_date="2000-01-01", forecast_horizon=8):
    """
    NEW FUNCTION: Run DSGE-informed DFM with structural priors
    
    Parameters:
    -----------
    dsge_posterior_estimates : dict
        Dictionary of DSGE posterior parameter estimates from Document 1
        Should contain keys like 'sigma', 'theta', 'phi_pi', 'phi_y', etc.
    """

    try:
        # Prepare data
        print("PREPARING ECONOMETRIC DATA FOR DSGE-INFORMED DFM")
        print("=" * 60)
        organized_blocks, combined_data = prepare_econometric_data(start_date)

        # Check data quality
        if len(combined_data.columns) < 6:
            print(f"Warning: Only {len(combined_data.columns)} variables available")

        # Initialize DSGE-informed DFM
        print(f"\nINITIALIZING DSGE-INFORMED DFM")
        print("=" * 60)
        print(f"DSGE priors available: {list(dsge_posterior_estimates.keys()) if dsge_posterior_estimates else 'None'}")
        
        model = EconometricDFM(organized_blocks, combined_data, dsge_priors=dsge_posterior_estimates)

        # Run complete estimation
        results = model.run_complete_estimation(forecast_horizon)
        return model, results

    except Exception as e:
        print(f"Error in DSGE-informed DFM: {e}")
        import traceback
        traceback.print_exc()
        return None, None


def run_core_economic_analysis(dsge_posterior_estimates=None, start_date="2000-01-01", forecast_horizon=8):
    """
    NEW MAIN FUNCTION: Run DSGE-informed econometric analysis
    
    Parameters:
    -----------
    dsge_posterior_estimates : dict, optional
        DSGE posterior parameter estimates. If None, runs standard DFM.
    """

    print("DSGE-INFORMED VAR-DFM ANALYSIS")
    print("=" * 80)

    if dsge_posterior_estimates is not None:
        print("Using DSGE-estimated priors for VAR coefficient restrictions")
        model, results = run_econometric_dfm(dsge_posterior_estimates, start_date, forecast_horizon)
    else:
        print("No DSGE priors provided - running standard DFM")
        # Fallback to standard model
        organized_blocks, combined_data = prepare_econometric_data(start_date)
        model = EconometricDFM(organized_blocks, combined_data, dsge_priors=None)
        results = model.run_complete_estimation(forecast_horizon)

    if model is not None:
        print("\n" + "="*80)
        print("DSGE-INFORMED ANALYSIS COMPLETED SUCCESSFULLY")
        print("="*80)

        estimation_type = "DSGE-Informed Bayesian" if model.dsge_priors else "Standard OLS"
        print(f"Model Summary:")
        print(f"  • {model.N_vars} macroeconomic variables")
        print(f"  • {model.T_obs} quarterly observations")
        print(f"  • {model.n_factors} economic factors extracted")
        print(f"  • VAR({model.factor_var['lags']}) dynamics estimated ({estimation_type})")
        print(f"  • {forecast_horizon} quarter forecasts generated")

        if model.dsge_priors:
            print(f"  • DSGE structural priors: {len(model.dsge_priors)} parameters")

        return {
            'model': model,
            'results': results
        }
    else:
        print("Model estimation failed")
        return None


# ============================================================================
# EXECUTION WITH DSGE PRIORS
# ============================================================================

if __name__ == "__main__":
    # Run DSGE-informed analysis
    # Replace with:
    analysis_results = run_core_economic_analysis(
    dsge_posterior_estimates=dsge_posteriors,  # Use actual estimates from Document 1
    start_date="2000-01-01",
    forecast_horizon=8
)

    if analysis_results:
        model = analysis_results['model']
        results = analysis_results['results']
        dsge_posterior_estimates=dsge_posteriors
        
        print(f"\n🎯 INTEGRATION SUMMARY:")
        print(f"   DSGE parameters used: {len(dsge_posteriors)}")
        print(f"   VAR estimation method: {'Bayesian (DSGE-informed)' if model.dsge_priors else 'OLS'}")
        print(f"   Model stability: {hasattr(model, 'eigenvalues')}")
        
        print("\n" + "="*80)
        print("DSGE-DFM INTEGRATION COMPLETED SUCCESSFULLY")
        print("="*80)

DSGE-INFORMED VAR-DFM ANALYSIS
Using DSGE-estimated priors for VAR coefficient restrictions
PREPARING ECONOMETRIC DATA FOR DSGE-INFORMED DFM
Fetching economic data from FRED...

Fetching Demand_Block:
  GDPC1 ✓ (102 obs)
  PAYEMS ✓ (308 obs)
  INDPRO ✓ (308 obs)
  ICSA ✓ (1341 obs)

Fetching Supply_Block:
  CPIAUCSL ✓ (308 obs)
  PCEPI ✓ (307 obs)
  AHETPI ✓ (308 obs)
  DCOILWTICO ✓ (6441 obs)

Fetching Monetary_Block:
  FEDFUNDS ✓ (308 obs)
  DGS10 ✓ (6429 obs)
  M2SL ✓ (307 obs)
  AAA ✓ (308 obs)

Successfully fetched 12 series
Common date range: 2000-01 to 2025-04
Quarterly observations: 101
Final dataset shape: (97, 12)
Date range: 2001-Qq to 2025-Qq
Demand_Block: ['GDPC1', 'PAYEMS', 'INDPRO', 'ICSA']
Supply_Block: ['CPIAUCSL', 'PCEPI', 'AHETPI', 'DCOILWTICO']
Monetary_Block: ['FEDFUNDS', 'DGS10', 'M2SL', 'AAA']

INITIALIZING DSGE-INFORMED DFM
DSGE priors available: ['sigma', 'theta', 'phi_pi', 'phi_y', 'rho_a', 'sigma_a', 'rho_v', 'sigma_v', 'rho_d', 'sigma_d', 'rho_s', 'sigma_s']

In [5]:
# Advanced Analysis and Visualization Components for Econometric DFM
# This extends the core model estimation with comprehensive diagnostics and analysis

class AdvancedDFMAnalyzer:
    """
    Advanced analysis components for the Econometric DFM

    This class extends the core EconometricDFM with:
    - Comprehensive VAR diagnostics
    - Impulse response functions
    - Historical decomposition
    - Forecast quality assessment
    - Advanced visualization
    """

    def __init__(self, dfm_model):
        """Initialize with a fitted EconometricDFM model"""
        self.model = dfm_model
        self.impulse_responses = None
        self.historical_decomp = None

    def enhanced_var_diagnostics(self):
        """Run comprehensive VAR model diagnostics"""
        print(f"\nVAR Model Diagnostics (Enhanced):")
        print("-" * 40)

        var_result = self.model.factor_var
        u = var_result['residuals']
        T_eff, K = u.shape

        # Portmanteau test for serial correlation
        max_lag_test = min(8, T_eff // 4)
        lm_stat = self._portmanteau_test(u, max_lag_test)
        lm_pval = 1 - chi2.cdf(lm_stat, df=K**2 * max_lag_test)

        print(f"  Portmanteau test (lag {max_lag_test}): {lm_stat:.2f} (p-val: {lm_pval:.3f})")
        print(f"  Serial correlation: {'Not detected' if lm_pval > 0.05 else 'Detected'} "
              f"{'✓' if lm_pval > 0.05 else '✗'}")

        # Normality test (Jarque-Bera for each equation)
        jb_stats = []
        for k in range(K):
            resid_k = u[:, k]
            # Simplified JB test
            skew = np.mean(((resid_k - np.mean(resid_k)) / np.std(resid_k))**3)
            kurt = np.mean(((resid_k - np.mean(resid_k)) / np.std(resid_k))**4) - 3
            jb_k = T_eff / 6 * (skew**2 + kurt**2 / 4)
            jb_stats.append(jb_k)

        avg_jb = np.mean(jb_stats)
        print(f"  Average Jarque-Bera: {avg_jb:.2f}")

        # Factor persistence (AR(1) diagonal elements)
        if var_result['lags'] >= 1:
            AR1_diag = np.diag(var_result['coefficients'][:, :K])
            print(f"  Factor persistence:")
            for i, factor_name in enumerate(self.model.factors.columns):
                pers = AR1_diag[i] if i < len(AR1_diag) else 0
                print(f"    {factor_name}: {pers:.3f}")

        # Residual correlations
        residual_corr = np.corrcoef(u.T)
        off_diag_corr = np.abs(residual_corr[np.triu_indices_from(residual_corr, k=1)])
        avg_cross_corr = np.mean(off_diag_corr)
        print(f"  Average cross-correlation of residuals: {avg_cross_corr:.3f}")

    def _portmanteau_test(self, residuals, max_lag):
        """Portmanteau test for serial correlation (Ljung-Box type)"""
        T, K = residuals.shape

        # Calculate sample autocorrelations
        autocorrs = []
        for lag in range(1, max_lag + 1):
            if T > lag:
                resid_lagged = residuals[:-lag, :]
                resid_current = residuals[lag:, :]

                # Cross-correlation matrix at lag h
                C_h = np.cov(resid_current.T, resid_lagged.T)[:K, K:]
                C_0 = np.cov(residuals.T)

                # Normalized autocorrelation
                rho_h = np.linalg.solve(C_0, C_h) @ np.linalg.solve(C_0, C_h.T)
                autocorrs.append(np.trace(rho_h))

        # Ljung-Box statistic (simplified)
        lm_stat = T * (T + 2) * sum(autocorr / (T - lag - 1)
                                   for lag, autocorr in enumerate(autocorrs, 1))

        return lm_stat

    def enhanced_forecast_quality(self):
        """Enhanced forecast quality and continuity assessment"""
        print("\nEnhanced Forecast Quality Assessment:")
        print("-" * 50)

        # Check forecast continuity (no large jumps)
        continuity_issues = 0

        for var in self.model.combined_data.columns:
            last_actual = self.model.combined_data[var].iloc[-1]
            first_forecast = self.model.variable_forecasts['mean'][var].iloc[0]

            # Historical volatility for jump assessment
            var_changes = self.model.combined_data[var].diff().dropna()
            typical_change = var_changes.std()

            jump_size = abs(first_forecast - last_actual)
            jump_ratio = jump_size / typical_change if typical_change > 0 else 0

            is_continuous = jump_ratio < 3.0  # Allow up to 3 standard deviations

            if not is_continuous:
                continuity_issues += 1

            print(f"{var:<12}: Jump={jump_size:7.2f} ({jump_ratio:.1f}σ) "
                  f"{'✓' if is_continuous else '✗'}")

        print(f"\nContinuity assessment: {self.model.N_vars - continuity_issues}/{self.model.N_vars} variables pass")

        # Factor forecast consistency check
        print(f"\nFactor Forecast Consistency:")
        print("-" * 50)

        for factor in self.model.factors.columns:
            factor_std = self.model.factors[factor].std()
            forecast_range = (self.model.factor_forecasts['mean'][factor].max() -
                            self.model.factor_forecasts['mean'][factor].min())
            consistency_ratio = forecast_range / (4 * factor_std)  # 4 std as reasonable range

            print(f"{factor}: Range ratio = {consistency_ratio:.2f} "
                  f"{'✓' if consistency_ratio < 1.5 else '⚠' if consistency_ratio < 2.5 else '✗'}")

    def compute_impulse_responses(self, horizon=20, orthogonalize=True):
        """
        Compute impulse response functions with proper identification
        """
        print("\n" + "="*60)
        print("IMPULSE RESPONSE FUNCTION ANALYSIS")
        print("="*60)

        if self.model.factor_var is None:
            raise ValueError("Must estimate VAR model first")

        Phi = self.model.factor_var['coefficients']
        Sigma = self.model.factor_var['sigma_u']
        p = self.model.factor_var['lags']
        K = Phi.shape[0]

        # Shock identification
        if orthogonalize:
            try:
                # Cholesky decomposition for recursive identification
                P = np.linalg.cholesky(Sigma)  # Lower triangular
                shock_names = [f'{name} (Orthogonal)' for name in self.model.factors.columns]
                print("Using Cholesky decomposition for orthogonal shocks")
                print("Ordering: Demand → Supply → Monetary")
            except np.linalg.LinAlgError:
                print("Warning: Cholesky decomposition failed, using identity matrix")
                P = np.eye(K)
                shock_names = [f'{name} (Raw)' for name in self.model.factors.columns]
        else:
            P = np.eye(K)
            shock_names = [f'{name} (Raw)' for name in self.model.factors.columns]

        # Compute IRFs
        irfs = np.zeros((horizon, K, K))  # horizon x responses x shocks

        for shock_idx in range(K):
            # Unit shock vector
            shock_vector = P[:, shock_idx]

            # Compute response over horizon
            for h in range(horizon):
                if h == 0:
                    irfs[h, :, shock_idx] = shock_vector
                else:
                    if p == 1:
                        irfs[h, :, shock_idx] = Phi @ irfs[h-1, :, shock_idx]
                    else:
                        # Use companion matrix for VAR(p)
                        if h == 1:
                            # First period response
                            extended_shock = np.zeros(self.model.companion_matrix.shape[0])
                            extended_shock[:K] = shock_vector
                            response = self.model.companion_matrix @ extended_shock
                            irfs[h, :, shock_idx] = response[:K]
                        else:
                            # Subsequent periods
                            prev_extended = np.zeros(self.model.companion_matrix.shape[0])
                            prev_extended[:K] = irfs[h-1, :, shock_idx]
                            response = self.model.companion_matrix @ prev_extended
                            irfs[h, :, shock_idx] = response[:K]

        self.impulse_responses = {
            'irfs': irfs,
            'shock_names': shock_names,
            'response_names': list(self.model.factors.columns),
            'horizon': horizon,
            'identification_matrix': P
        }

        # Print summary statistics
        print(f"Computed {K}x{K} impulse responses over {horizon} periods")

        # Peak responses
        print(f"\nPeak Response Analysis:")
        for i, response_name in enumerate(self.model.factors.columns):
            for j, shock_name in enumerate(shock_names):
                irf_series = irfs[:, i, j]
                peak_period = np.argmax(np.abs(irf_series))
                peak_value = irf_series[peak_period]
                print(f"  {response_name} → {shock_name}: Peak = {peak_value:.3f} at period {peak_period}")

        return self.impulse_responses

    def plot_impulse_responses(self):
        """Plot impulse response functions"""
        if self.impulse_responses is None:
            self.compute_impulse_responses()

        irfs = self.impulse_responses['irfs']
        shock_names = self.impulse_responses['shock_names']
        response_names = self.impulse_responses['response_names']
        horizon = self.impulse_responses['horizon']

        fig, axes = plt.subplots(len(response_names), len(shock_names),
                                figsize=(5*len(shock_names), 4*len(response_names)))

        if len(response_names) == 1:
            axes = axes.reshape(1, -1)
        if len(shock_names) == 1:
            axes = axes.reshape(-1, 1)

        for i, response_name in enumerate(response_names):
            for j, shock_name in enumerate(shock_names):
                ax = axes[i, j] if len(response_names) > 1 else axes[j]

                irf_series = irfs[:, i, j]
                periods = range(horizon)

                ax.plot(periods, irf_series, linewidth=2, color=f'C{j}')
                ax.axhline(y=0, color='black', linestyle='-', alpha=0.3)
                ax.set_title(f'{response_name} → {shock_name}', fontsize=10)
                ax.grid(True, alpha=0.3)
                ax.set_xlabel('Quarters')

        plt.suptitle('Impulse Response Functions', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()

    def historical_decomposition(self, periods_back=40):
        """Compute historical decomposition of factors"""
        print("\n" + "="*60)
        print("HISTORICAL DECOMPOSITION")
        print("="*60)

        if self.model.factor_var is None:
            raise ValueError("Must estimate VAR model first")

        residuals = self.model.factor_var['residuals']
        Phi = self.model.factor_var['coefficients']
        c = self.model.factor_var['constant']
        p = self.model.factor_var['lags']
        K = Phi.shape[0]

        T_residuals = len(residuals)
        periods = min(periods_back, T_residuals)

        # Use Cholesky identification for decomposition
        try:
            P = np.linalg.cholesky(self.model.factor_var['sigma_u'])
            orthogonal_residuals = np.linalg.solve(P, residuals.T).T
        except:
            P = np.eye(K)
            orthogonal_residuals = residuals

        # Initialize decomposition arrays
        historical_decomp = np.zeros((periods, K, K))  # periods x factors x shocks

        # Compute decomposition recursively
        for t in range(periods):
            time_idx = T_residuals - periods + t

            for shock_idx in range(K):
                if t == 0:
                    # Initial impact
                    historical_decomp[t, :, shock_idx] = orthogonal_residuals[time_idx, shock_idx] * P[:, shock_idx]
                else:
                    # Propagated impact from previous period
                    if p == 1:
                        historical_decomp[t, :, shock_idx] = (
                            Phi @ historical_decomp[t-1, :, shock_idx] +
                            orthogonal_residuals[time_idx, shock_idx] * P[:, shock_idx]
                        )
                    else:
                        # For VAR(p), use companion form (simplified)
                        historical_decomp[t, :, shock_idx] = (
                            Phi[:, :K] @ historical_decomp[t-1, :, shock_idx] +
                            orthogonal_residuals[time_idx, shock_idx] * P[:, shock_idx]
                        )

        # Store results
        decomp_dates = self.model.factors.index[-periods:]

        self.historical_decomp = {
            'contributions': historical_decomp,
            'dates': decomp_dates,
            'shock_names': ['Demand', 'Supply', 'Monetary'][:K],
            'factor_names': list(self.model.factors.columns)
        }

        print(f"Computed historical decomposition for {periods} periods")
        return self.historical_decomp

    def plot_historical_decomposition(self):
        """Plot historical decomposition"""
        if self.historical_decomp is None:
            self.historical_decomposition()

        decomp = self.historical_decomp['contributions']
        dates = self.historical_decomp['dates']
        shock_names = self.historical_decomp['shock_names']
        factor_names = self.historical_decomp['factor_names']

        fig, axes = plt.subplots(len(factor_names), 1, figsize=(15, 4*len(factor_names)))

        if len(factor_names) == 1:
            axes = [axes]

        colors = ['blue', 'red', 'green']

        for factor_idx, factor_name in enumerate(factor_names):
            ax = axes[factor_idx]

            # Plot contributions from each shock
            bottom_pos = np.zeros(len(dates))
            bottom_neg = np.zeros(len(dates))

            for shock_idx, shock_name in enumerate(shock_names):
                contributions = decomp[:, factor_idx, shock_idx]

                # Separate positive and negative contributions
                pos_contrib = np.maximum(contributions, 0)
                neg_contrib = np.minimum(contributions, 0)

                # Stack positive contributions upward
                ax.fill_between(dates, bottom_pos, bottom_pos + pos_contrib,
                               label=f'{shock_name} (+)', alpha=0.7, color=colors[shock_idx % len(colors)])
                bottom_pos += pos_contrib

                # Stack negative contributions downward
                ax.fill_between(dates, bottom_neg, bottom_neg + neg_contrib,
                               alpha=0.7, color=colors[shock_idx % len(colors)])
                bottom_neg += neg_contrib

            # Add actual factor values for comparison
            factor_actual = self.model.factors[factor_name].iloc[-len(dates):].values
            ax.plot(dates, factor_actual, color='black', linewidth=2, label='Actual')

            ax.axhline(y=0, color='black', linestyle='-', alpha=0.5)
            ax.set_title(f'{factor_name} - Historical Decomposition')
            ax.legend(loc='upper left')
            ax.grid(True, alpha=0.3)
            ax.tick_params(axis='x', rotation=45)

        plt.suptitle('Historical Shock Decomposition', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()

    def plot_comprehensive_results(self):
        """Comprehensive visualization of all DFM results"""
        fig = plt.figure(figsize=(20, 16))
        gs = fig.add_gridspec(4, 3, hspace=0.4, wspace=0.3)

        # Row 1: Factor time series with forecasts
        for i, factor in enumerate(self.model.factors.columns):
            ax = fig.add_subplot(gs[0, i])

            # Historical factors
            ax.plot(self.model.factors.index, self.model.factors[factor],
                   linewidth=2, color=f'C{i}', label='Historical')

            # Add forecasts if available
            if hasattr(self.model, 'factor_forecasts'):
                forecast_mean = self.model.factor_forecasts['mean'][factor]

                ax.plot(forecast_mean.index, forecast_mean,
                       color=f'C{i}', linewidth=2, linestyle='--', label='Forecast')

                # Bootstrap confidence intervals
                if 'bootstrap_lower' in self.model.factor_forecasts:
                    lower = self.model.factor_forecasts['bootstrap_lower'][factor]
                    upper = self.model.factor_forecasts['bootstrap_upper'][factor]
                    ax.fill_between(forecast_mean.index, lower, upper,
                                   color=f'C{i}', alpha=0.2, label='95% CI')

                # Mark forecast start
                ax.axvline(x=self.model.factors.index[-1], color='red', linestyle=':', alpha=0.7)

            ax.set_title(f'{factor}', fontsize=12, fontweight='bold')
            ax.legend(fontsize=8)
            ax.grid(True, alpha=0.3)
            ax.tick_params(axis='x', rotation=45)

        # Row 2: Factor loadings heatmap
        ax_loadings = fig.add_subplot(gs[1, :])

        # Create enhanced heatmap
        loadings_matrix = self.model.loadings.values.T
        im = ax_loadings.imshow(loadings_matrix, aspect='auto', cmap='RdBu_r',
                               vmin=-np.max(np.abs(loadings_matrix)),
                               vmax=np.max(np.abs(loadings_matrix)))

        ax_loadings.set_title('Factor Loading Matrix', fontsize=14, fontweight='bold')
        ax_loadings.set_xticks(range(len(self.model.loadings.index)))
        ax_loadings.set_xticklabels(self.model.loadings.index, rotation=45, ha='right')
        ax_loadings.set_yticks(range(len(self.model.loadings.columns)))
        ax_loadings.set_yticklabels(self.model.loadings.columns)

        # Add value annotations
        for i in range(loadings_matrix.shape[0]):
            for j in range(loadings_matrix.shape[1]):
                text_color = 'white' if abs(loadings_matrix[i, j]) > 0.5 else 'black'
                ax_loadings.text(j, i, f'{loadings_matrix[i, j]:.2f}',
                               ha="center", va="center", color=text_color, fontsize=8)

        plt.colorbar(im, ax=ax_loadings, orientation='horizontal', pad=0.1, label='Loading Magnitude')

        # Row 3: VAR diagnostics
        ax_stability = fig.add_subplot(gs[2, 0])
        if hasattr(self.model, 'eigenvalues'):
            # Plot eigenvalues in complex plane
            real_parts = np.real(self.model.eigenvalues)
            imag_parts = np.imag(self.model.eigenvalues)

            ax_stability.scatter(real_parts, imag_parts, s=50, alpha=0.7)

            # Unit circle
            theta = np.linspace(0, 2*np.pi, 100)
            ax_stability.plot(np.cos(theta), np.sin(theta), 'r--', alpha=0.5, label='Unit Circle')

            ax_stability.set_xlabel('Real Part')
            ax_stability.set_ylabel('Imaginary Part')
            ax_stability.set_title('VAR Eigenvalues')
            ax_stability.legend()
            ax_stability.grid(True, alpha=0.3)
            ax_stability.axis('equal')

        # Row 3: Model fit (R-squared)
        ax_fit = fig.add_subplot(gs[2, 1])
        if hasattr(self.model, 'variable_forecasts'):
            # Calculate R-squared from factor model
            r_squared = []
            for var in self.model.combined_data.columns:
                X_actual = self.model.data_standardized[var].values
                F_values = self.model.factors.values
                loadings_var = self.model.loadings.loc[var].values

                X_fitted = F_values @ loadings_var
                ss_res = np.sum((X_actual - X_fitted)**2)
                ss_tot = np.sum((X_actual - np.mean(X_actual))**2)
                r_sq = 1 - ss_res / ss_tot if ss_tot > 0 else 0
                r_squared.append(r_sq)

            bars = ax_fit.bar(range(len(r_squared)), r_squared, color='steelblue', alpha=0.7)
            ax_fit.set_title('Model Fit (R²)')
            ax_fit.set_xticks(range(len(r_squared)))
            ax_fit.set_xticklabels(self.model.combined_data.columns, rotation=45, ha='right')
            ax_fit.set_ylabel('R²')
            ax_fit.set_ylim(0, 1)
            ax_fit.grid(True, alpha=0.3)

            # Add value labels on bars
            for bar, r2 in zip(bars, r_squared):
                ax_fit.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                           f'{r2:.2f}', ha='center', va='bottom', fontsize=8)

        # Row 3: Forecast continuity assessment
        ax_continuity = fig.add_subplot(gs[2, 2])
        if hasattr(self.model, 'variable_forecasts'):
            # Check forecast jumps
            jump_ratios = []
            for var in self.model.combined_data.columns:
                last_actual = self.model.combined_data[var].iloc[-1]
                first_forecast = self.model.variable_forecasts['mean'][var].iloc[0]
                var_changes = self.model.combined_data[var].diff().dropna()
                typical_change = var_changes.std()
                jump_size = abs(first_forecast - last_actual)
                jump_ratio = jump_size / typical_change if typical_change > 0 else 0
                jump_ratios.append(jump_ratio)

            bars = ax_continuity.bar(range(len(jump_ratios)), jump_ratios,
                                   color='green' if all(jr < 3 for jr in jump_ratios) else 'orange', alpha=0.7)
            ax_continuity.set_title('Forecast Continuity\n(Jump size / Historical σ)')
            ax_continuity.set_xticks(range(len(jump_ratios)))
            ax_continuity.set_xticklabels(self.model.combined_data.columns, rotation=45, ha='right')
            ax_continuity.set_ylabel('Jump Ratio')
            ax_continuity.axhline(y=3, color='red', linestyle='--', alpha=0.7, label='Warning threshold')
            ax_continuity.legend()
            ax_continuity.grid(True, alpha=0.3)

        # Row 4: Variable forecasts (selected key variables)
        key_variables = ['GDPC1', 'CPIAUCSL', 'FEDFUNDS'] if all(v in self.model.combined_data.columns for v in ['GDPC1', 'CPIAUCSL', 'FEDFUNDS']) else self.model.combined_data.columns[:3]

        for i, var in enumerate(key_variables):
            if i >= 3:
                break

            ax = fig.add_subplot(gs[3, i])

            # Historical data
            hist_data = self.model.combined_data[var]
            ax.plot(hist_data.index, hist_data, color='black', linewidth=1.5, label='Historical')

            # Forecasts with uncertainty
            if hasattr(self.model, 'variable_forecasts'):
                forecast_mean = self.model.variable_forecasts['mean'][var]

                ax.plot(forecast_mean.index, forecast_mean,
                       color='blue', linewidth=2, linestyle='--', label='Forecast')

                # Mark forecast start
                ax.axvline(x=hist_data.index[-1], color='red', linestyle=':', alpha=0.7)

            ax.set_title(f'{var} - Forecast', fontweight='bold')
            ax.legend(fontsize=8)
            ax.grid(True, alpha=0.3)
            ax.tick_params(axis='x', rotation=45)

        plt.suptitle('Econometric Dynamic Factor Model - Complete Analysis',
                     fontsize=16, fontweight='bold', y=0.98)
        plt.show()


def run_complete_analysis_with_diagnostics(core_model):
    """
    Run complete advanced analysis on a fitted core model

    Parameters:
    - core_model: Fitted EconometricDFM instance from core estimation

    Returns:
    - Advanced analyzer with all diagnostics computed
    """

    print("\n" + "="*80)
    print("RUNNING ADVANCED ANALYSIS AND DIAGNOSTICS")
    print("="*80)

    # Initialize advanced analyzer
    analyzer = AdvancedDFMAnalyzer(core_model)

    # Run comprehensive diagnostics
    print("\n1. Enhanced VAR Diagnostics")
    analyzer.enhanced_var_diagnostics()

    print("\n2. Enhanced Forecast Quality Assessment")
    analyzer.enhanced_forecast_quality()

    # Advanced structural analysis
    print("\n3. Impulse Response Functions")
    analyzer.compute_impulse_responses(horizon=20, orthogonalize=True)

    print("\n4. Historical Decomposition")
    analyzer.historical_decomposition(periods_back=min(40, core_model.T_obs//2))

    # Comprehensive visualization
    print("\n5. Generating Comprehensive Plots")
    analyzer.plot_impulse_responses()
    analyzer.plot_historical_decomposition()
    analyzer.plot_comprehensive_results()

    print("\n" + "="*80)
    print("ADVANCED ANALYSIS COMPLETED")
    print("="*80)

    return analyzer


# Example usage function
def complete_dfm_pipeline_with_advanced_analysis(start_date="2000-01-01", forecast_horizon=8):
    """
    Complete DFM pipeline combining core estimation with advanced analysis

    This function:
    1. Runs the core model estimation
    2. Applies all advanced diagnostics and analysis
    3. Generates comprehensive visualizations

    Returns both the core model and advanced analyzer
    """

    print("COMPLETE ECONOMETRIC DFM PIPELINE")
    print("="*80)

    # Step 1: Run core estimation (assumes you have the core functions available)
    # This would use your core estimation code
    print("Step 1: Running Core Model Estimation...")

    # Step 2: Advanced analysis
    print("Step 2: Running Advanced Analysis...")
    # analyzer = run_complete_analysis_with_diagnostics(core_model)

    print("Complete pipeline finished!")

    return None  # Would return (core_model, analyzer)


# Additional utility functions for enhanced analysis

def compare_forecast_scenarios(model, scenarios={'baseline': {}, 'stressed': {'shock_scale': 2.0}}):
    """Compare forecasts under different scenarios"""
    print("\nForecast Scenario Comparison")
    print("="*40)

    scenario_results = {}

    for scenario_name, scenario_params in scenarios.items():
        print(f"\nRunning {scenario_name} scenario...")

        # This would modify the model parameters according to scenario
        # and re-run forecasts
        scenario_results[scenario_name] = {
            'forecasts': model.factor_forecasts['mean'].copy()
        }

    return scenario_results


def generate_forecast_report(model, analyzer):
    """Generate a comprehensive forecast report"""

    report = {
        'model_summary': {
            'n_variables': model.N_vars,
            'n_observations': model.T_obs,
            'n_factors': model.n_factors,
            'var_lags': model.factor_var['lags'],
            'stability': hasattr(model, 'eigenvalues') and np.max(np.abs(model.eigenvalues)) < 1.0
        },
        'factor_persistence': {},
        'forecast_summary': {},
        'diagnostics': {}
    }

    # Add factor persistence
    if model.factor_var['lags'] >= 1:
        AR1_diag = np.diag(model.factor_var['coefficients'][:, :model.n_factors])
        for i, factor_name in enumerate(model.factors.columns):
            if i < len(AR1_diag):
                report['factor_persistence'][factor_name] = AR1_diag[i]

    # Forecast summary statistics
    for factor in model.factors.columns:
        if hasattr(model, 'factor_forecasts'):
            factor_forecast = model.factor_forecasts['mean'][factor]
            report['forecast_summary'][factor] = {
                'mean_forecast': factor_forecast.mean(),
                'forecast_volatility': factor_forecast.std(),
                'trend': 'increasing' if factor_forecast.iloc[-1] > factor_forecast.iloc[0] else 'decreasing'
            }

    return report


def enhanced_bootstrap_forecasts(model, horizon=8, n_bootstrap=2000, confidence_levels=[0.68, 0.90, 0.95]):
    """
    Enhanced bootstrap forecasting with multiple confidence levels and scenario analysis
    """
    print(f"\nEnhanced Bootstrap Forecasting")
    print("="*50)

    Phi = model.factor_var['coefficients']
    c = model.factor_var['constant']
    residuals = model.factor_var['residuals']
    p = model.factor_var['lags']
    K = Phi.shape[0]

    # Multiple bootstrap samples for robust inference
    bootstrap_paths = np.zeros((n_bootstrap, horizon, K))

    print(f"Running {n_bootstrap} bootstrap replications...")

    for b in range(n_bootstrap):
        if b % 500 == 0:
            print(f"  Bootstrap replication {b}/{n_bootstrap}")

        # Resample residuals with replacement
        boot_indices = np.random.choice(len(residuals), size=horizon, replace=True)
        boot_residuals = residuals[boot_indices, :]

        if p == 1:
            F_last = model.factors.values[-1, :].copy()
            for h in range(horizon):
                F_forecast = c + Phi @ F_last + boot_residuals[h, :]
                bootstrap_paths[b, h, :] = F_forecast
                F_last = F_forecast
        else:
            F_history = model.factors.values[-p:, :].copy()
            for h in range(horizon):
                F_lagged = F_history.flatten('F')
                F_forecast = c + Phi @ F_lagged + boot_residuals[h, :]
                bootstrap_paths[b, h, :] = F_forecast
                F_history = np.vstack([F_history[1:, :], F_forecast.reshape(1, -1)])

    # Calculate multiple confidence levels
    bootstrap_results = {}

    for conf_level in confidence_levels:
        alpha = 1 - conf_level
        lower_pct = 100 * (alpha / 2)
        upper_pct = 100 * (1 - alpha / 2)

        bootstrap_results[f'{int(conf_level*100)}%'] = {
            'lower': np.percentile(bootstrap_paths, lower_pct, axis=0),
            'upper': np.percentile(bootstrap_paths, upper_pct, axis=0)
        }

    # Bootstrap statistics
    bootstrap_results['statistics'] = {
        'mean': np.mean(bootstrap_paths, axis=0),
        'std': np.std(bootstrap_paths, axis=0),
        'skewness': np.mean(((bootstrap_paths - np.mean(bootstrap_paths, axis=0)[np.newaxis, :, :]) /
                           np.std(bootstrap_paths, axis=0)[np.newaxis, :, :])**3, axis=0),
        'kurtosis': np.mean(((bootstrap_paths - np.mean(bootstrap_paths, axis=0)[np.newaxis, :, :]) /
                           np.std(bootstrap_paths, axis=0)[np.newaxis, :, :])**4, axis=0) - 3
    }

    print(f"Bootstrap forecasting completed with {len(confidence_levels)} confidence levels")
    return bootstrap_results


def forecast_combination_analysis(models_list, weights=None):
    """
    Combine forecasts from multiple DFM models with different specifications

    Parameters:
    - models_list: List of fitted DFM models
    - weights: Optional weights for combination (default: equal weights)
    """
    print("\nForecast Combination Analysis")
    print("="*40)

    n_models = len(models_list)

    if weights is None:
        weights = np.ones(n_models) / n_models
    else:
        weights = np.array(weights)
        weights = weights / np.sum(weights)  # Normalize

    print(f"Combining {n_models} models with weights: {weights}")

    # Combine factor forecasts
    combined_forecasts = {}

    # Get common factors across models
    common_factors = set(models_list[0].factors.columns)
    for model in models_list[1:]:
        common_factors = common_factors.intersection(set(model.factors.columns))

    print(f"Common factors across models: {list(common_factors)}")

    for factor in common_factors:
        factor_forecasts = []

        for model in models_list:
            if hasattr(model, 'factor_forecasts'):
                factor_forecasts.append(model.factor_forecasts['mean'][factor].values)

        if factor_forecasts:
            # Weight and combine
            combined_forecast = np.average(np.array(factor_forecasts), axis=0, weights=weights)
            combined_forecasts[factor] = combined_forecast

    return combined_forecasts


def stress_test_forecasts(model, stress_scenarios):
    """
    Stress test forecasts under adverse scenarios

    Parameters:
    - model: Fitted DFM model
    - stress_scenarios: Dict of stress scenarios with parameter modifications
    """
    print("\nForecast Stress Testing")
    print("="*30)

    stress_results = {}

    for scenario_name, scenario_params in stress_scenarios.items():
        print(f"\nStress scenario: {scenario_name}")

        # Modify model parameters according to stress scenario
        stressed_model = deepcopy(model)  # Would need to import copy

        # Apply stress to residual covariance (increased uncertainty)
        if 'volatility_multiplier' in scenario_params:
            vol_mult = scenario_params['volatility_multiplier']
            stressed_model.factor_var['sigma_u'] *= vol_mult**2
            print(f"  Applied volatility multiplier: {vol_mult}")

        # Apply persistent shock to factors
        if 'persistent_shock' in scenario_params:
            shock_dict = scenario_params['persistent_shock']
            print(f"  Applied persistent shocks: {shock_dict}")

        # Re-run forecasts with stressed parameters
        # This would require re-implementing the forecast method with modified parameters
        stress_results[scenario_name] = {
            'modified_parameters': scenario_params,
            'status': 'computed'  # Placeholder
        }

    return stress_results


def model_selection_criteria(models_dict):
    """
    Compare multiple DFM specifications using various criteria

    Parameters:
    - models_dict: Dictionary of model_name -> fitted_model pairs
    """
    print("\nModel Selection and Comparison")
    print("="*40)

    criteria_results = {}

    for model_name, model in models_dict.items():
        if model.factor_var is None:
            continue

        var_result = model.factor_var
        T_eff = var_result['T_eff']
        K = len(model.factors.columns)
        p = var_result['lags']

        # Log-likelihood
        log_likelihood = var_result['log_likelihood']

        # Number of parameters (VAR coefficients + factor loadings)
        n_var_params = K * (1 + K * p)  # Constants + VAR coefficients
        n_loading_params = model.N_vars * K  # Factor loadings
        total_params = n_var_params + n_loading_params

        # Information criteria
        aic = -2 * log_likelihood + 2 * total_params
        bic = -2 * log_likelihood + total_params * np.log(T_eff)
        hqic = -2 * log_likelihood + 2 * total_params * np.log(np.log(T_eff))

        # Forecast accuracy metrics (if validation sample available)
        # This would require out-of-sample evaluation

        criteria_results[model_name] = {
            'log_likelihood': log_likelihood,
            'AIC': aic,
            'BIC': bic,
            'HQIC': hqic,
            'n_parameters': total_params,
            'n_factors': K,
            'var_lags': p,
            'sample_size': T_eff
        }

        print(f"{model_name}:")
        print(f"  Log-likelihood: {log_likelihood:.2f}")
        print(f"  AIC: {aic:.2f}")
        print(f"  BIC: {bic:.2f}")
        print(f"  Parameters: {total_params}")

    # Select best model by BIC (most conservative)
    best_model_name = min(criteria_results.keys(),
                         key=lambda x: criteria_results[x]['BIC'])
    print(f"\nBest model by BIC: {best_model_name}")

    return criteria_results, best_model_name


def real_time_forecast_evaluation(model, real_time_data, evaluation_periods=8):
    """
    Evaluate forecast accuracy using real-time data

    Parameters:
    - model: Fitted DFM model
    - real_time_data: New data for evaluation
    - evaluation_periods: Number of periods to evaluate
    """
    print(f"\nReal-time Forecast Evaluation")
    print("="*40)

    if not hasattr(model, 'factor_forecasts'):
        print("No forecasts available for evaluation")
        return None

    evaluation_results = {}

    # Compare forecasts with actual realizations
    for var in model.combined_data.columns:
        if var in real_time_data.columns:
            forecast_values = model.variable_forecasts['mean'][var].iloc[:evaluation_periods]
            actual_values = real_time_data[var].iloc[:evaluation_periods]

            # Calculate forecast errors
            errors = actual_values - forecast_values

            # Accuracy metrics
            mae = np.mean(np.abs(errors))
            rmse = np.sqrt(np.mean(errors**2))
            mape = np.mean(np.abs(errors / actual_values)) * 100

            evaluation_results[var] = {
                'MAE': mae,
                'RMSE': rmse,
                'MAPE': mape,
                'bias': np.mean(errors),
                'forecast_values': forecast_values,
                'actual_values': actual_values,
                'errors': errors
            }

            print(f"{var}: RMSE={rmse:.3f}, MAE={mae:.3f}, MAPE={mape:.1f}%")

    return evaluation_results


# Main integration function
def run_complete_advanced_dfm_analysis(core_analysis_results,
                                     include_stress_tests=True,
                                     include_bootstrap_enhanced=True,
                                     stress_scenarios=None):
    """
    Complete advanced analysis pipeline integrating all components

    Parameters:
    - core_analysis_results: Results from core model estimation
    - include_stress_tests: Whether to run stress tests
    - include_bootstrap_enhanced: Whether to run enhanced bootstrap
    - stress_scenarios: Custom stress test scenarios
    """

    if core_analysis_results is None:
        print("No core analysis results provided")
        return None

    model = core_analysis_results['model']

    print("="*80)
    print("COMPLETE ADVANCED DFM ANALYSIS PIPELINE")
    print("="*80)

    # Initialize advanced analyzer
    analyzer = AdvancedDFMAnalyzer(model)

    # Component 1: Enhanced diagnostics
    print("\n1. ENHANCED VAR DIAGNOSTICS")
    analyzer.enhanced_var_diagnostics()
    analyzer.enhanced_forecast_quality()

    # Component 2: Structural analysis
    print("\n2. STRUCTURAL ANALYSIS")
    analyzer.compute_impulse_responses(horizon=20)
    analyzer.historical_decomposition()

    # Component 3: Enhanced bootstrap (optional)
    enhanced_bootstrap_results = None
    if include_bootstrap_enhanced:
        print("\n3. ENHANCED BOOTSTRAP FORECASTING")
        enhanced_bootstrap_results = enhanced_bootstrap_forecasts(model, horizon=8, n_bootstrap=1000)

    # Component 4: Stress testing (optional)
    stress_test_results = None
    if include_stress_tests:
        print("\n4. STRESS TESTING")
        if stress_scenarios is None:
            stress_scenarios = {
                'high_volatility': {'volatility_multiplier': 2.0},
                'recession_scenario': {'volatility_multiplier': 1.5, 'persistent_shock': {'Demand_Factor': -1.0}},
                'inflation_shock': {'persistent_shock': {'Supply_Factor': 1.0}}
            }
        stress_test_results = stress_test_forecasts(model, stress_scenarios)

    # Component 5: Generate comprehensive report
    print("\n5. GENERATING FORECAST REPORT")
    forecast_report = generate_forecast_report(model, analyzer)

    # Component 6: Comprehensive visualizations
    print("\n6. COMPREHENSIVE VISUALIZATIONS")
    analyzer.plot_impulse_responses()
    analyzer.plot_historical_decomposition()
    analyzer.plot_comprehensive_results()

    # Compile final results
    advanced_results = {
        'analyzer': analyzer,
        'forecast_report': forecast_report,
        'enhanced_bootstrap': enhanced_bootstrap_results,
        'stress_tests': stress_test_results,
        'impulse_responses': analyzer.impulse_responses,
        'historical_decomposition': analyzer.historical_decomp
    }

    print("\n" + "="*80)
    print("ADVANCED ANALYSIS PIPELINE COMPLETED")
    print("="*80)
    print(f"Results stored in advanced_results dictionary")
    print(f"Key components:")
    print(f"  • Enhanced VAR diagnostics and forecast quality assessment")
    print(f"  • Impulse response functions and historical decomposition")
    if include_bootstrap_enhanced:
        print(f"  • Enhanced bootstrap forecasting with multiple confidence levels")
    if include_stress_tests:
        print(f"  • Stress test scenarios")
    print(f"  • Comprehensive visualizations and reporting")

    return advanced_results


# Example usage
if __name__ == "__main__":
    print("Advanced DFM Analysis Components")
    print("="*50)
    print("This module provides advanced analysis for econometric DFM models.")
    print("\nMain functions:")
    print("• AdvancedDFMAnalyzer: Core advanced analysis class")
    print("• run_complete_analysis_with_diagnostics: Full diagnostic pipeline")
    print("• enhanced_bootstrap_forecasts: Robust bootstrap forecasting")
    print("• stress_test_forecasts: Scenario analysis")
    print("• model_selection_criteria: Model comparison")
    print("• run_complete_advanced_dfm_analysis: Complete advanced pipeline")
    print("\nTo use with core model:")
    print("advanced_results = run_complete_advanced_dfm_analysis(core_analysis_results)")

# ============================================================================
# EXECUTION
# ============================================================================

# After running your core estimation:
core_results = run_core_economic_analysis()

# Run advanced analysis:
advanced_results = run_complete_advanced_dfm_analysis(core_results)

# Or step by step:
analyzer = AdvancedDFMAnalyzer(core_results['model'])
analyzer.compute_impulse_responses()
analyzer.historical_decomposition()
analyzer.plot_comprehensive_results()

Advanced DFM Analysis Components
This module provides advanced analysis for econometric DFM models.

Main functions:
• AdvancedDFMAnalyzer: Core advanced analysis class
• run_complete_analysis_with_diagnostics: Full diagnostic pipeline
• enhanced_bootstrap_forecasts: Robust bootstrap forecasting
• stress_test_forecasts: Scenario analysis
• model_selection_criteria: Model comparison
• run_complete_advanced_dfm_analysis: Complete advanced pipeline

To use with core model:
advanced_results = run_complete_advanced_dfm_analysis(core_analysis_results)
DSGE-INFORMED VAR-DFM ANALYSIS
No DSGE priors provided - running standard DFM
Fetching economic data from FRED...

Fetching Demand_Block:
Failed to fetch GDPC1: <urlopen error [Errno 8] nodename nor servname provided, or not known>
  GDPC1 ✗
Failed to fetch PAYEMS: <urlopen error [Errno 8] nodename nor servname provided, or not known>
  PAYEMS ✗
Failed to fetch INDPRO: <urlopen error [Errno 8] nodename nor servname provided, or not known>
  IND

ValueError: Only 0 series fetched. Need at least 8.

In [49]:
# DSGE-Informed Enhanced DFM with Theta Optimization
# FIXED: Properly integrates DSGE posterior estimates from Document 1

class DSGEInformedEnhancedDFMTheta:
    """
    Enhanced DFM with theta optimization that preserves DSGE-informed VAR structure
    FIXED: Now properly extracts DSGE posterior estimates from Document 1's estimation results
    """

    def __init__(self, base_dfm_model):
        # Import structural components from DSGE-informed base model
        self.factors = base_dfm_model.factors.copy()
        self.loadings = base_dfm_model.loadings.copy()  # PRESERVE original loadings
        self.data_standardized = base_dfm_model.data_standardized.copy()
        self.scaler = base_dfm_model.scaler
        self.combined_data = base_dfm_model.combined_data.copy()
        
        # CRITICAL: Preserve DSGE-informed VAR structure
        self.base_factor_var = base_dfm_model.factor_var  # Original DSGE-informed VAR
        
        # FIXED: Extract DSGE priors from base model's estimation results
        self.dsge_priors = self._extract_dsge_posterior_estimates(base_dfm_model)
        self.idiosyncratic_var = base_dfm_model.idiosyncratic_var.copy()

        # Model dimensions
        self.T_obs, self.n_factors = self.factors.shape
        self.N_vars = len(self.data_standardized.columns)

        # Initialize theta components
        self.optimal_theta = None
        self.theta_strategy = None
        self.method_results = {}
        
        # Results storage for each method
        self.method_forecasts = {}
        self.method_var_models = {}  # Store theta-adapted VAR models
        self.method_r2 = {}
        self.method_metrics = {}
        
        # Cross-validation setup
        self.tscv = TimeSeriesSplit(n_splits=3, test_size=4)

        print(f"DSGE-Informed Enhanced DFM-Theta initialized:")
        print(f"  Factors: {self.n_factors}, Variables: {self.N_vars}")
        print(f"  DSGE priors: {'Available' if self.dsge_priors else 'None'}")
        print(f"  Base VAR estimation: {'Bayesian' if self.base_factor_var.get('bayesian', False) else 'OLS'}")

    # Add these methods to the DSGEInformedEnhancedDFMTheta class

    def detailed_variable_cv_analysis(self, method_name, theta_params, theta_func):
        """
        Detailed cross-validation analysis at the variable level
        """
        variable_cv_results = {}
        
        for train_idx, test_idx in self.tscv.split(self.factors):
            if len(test_idx) < 2:
                continue
                
            # Split data maintaining temporal structure
            factors_train = self.factors.iloc[train_idx]
            factors_test = self.factors.iloc[test_idx]
            data_train = self.data_standardized.iloc[train_idx]
            data_test = self.data_standardized.iloc[test_idx]
            
            try:
                # Apply theta transformation to training factors
                F_theta_train = theta_func(factors_train, theta_params)
                F_theta_test = theta_func(factors_test, theta_params)
                
                # Estimate theta-aware VAR on training data
                theta_var_cv = self._estimate_theta_aware_var(F_theta_train, theta_params, method_name)
                
                # Generate forecasts
                test_forecasts = self._forecast_theta_var(F_theta_test, theta_var_cv, len(test_idx))
                F_theta_forecast = test_forecasts['mean'].values
                
                # Use original loadings for variable forecasts
                X_pred = F_theta_forecast @ self.loadings.values.T
                
                # Calculate metrics for each variable
                for i, var_name in enumerate(self.data_standardized.columns):
                    if var_name not in variable_cv_results:
                        variable_cv_results[var_name] = {
                            'RMSE': [], 'MAE': [], 'MAPE': [], 'SMAPE': [], 
                            'Theil_U1': [], 'Theil_U2': [], 'MdAPE': [], 'MASE': [], 'R2': []
                        }
                    
                    y_true_var = data_test.values[:, i]
                    y_pred_var = X_pred[:, i]
                    
                    # Calculate all metrics
                    metrics = self._calculate_comprehensive_metrics(y_true_var, y_pred_var, data_train.values[:, i])
                    
                    for metric_name, value in metrics.items():
                        variable_cv_results[var_name][metric_name].append(value)
                    
            except Exception as e:
                print(f"CV fold failed for {method_name}: {e}")
                continue
        
        # Average across folds (rest remains the same)
        averaged_results = {}
        for var_name, metrics in variable_cv_results.items():
            averaged_results[var_name] = {}
            for metric_name, values in metrics.items():
                if values:
                    averaged_results[var_name][metric_name] = np.mean(values)
                else:
                    averaged_results[var_name][metric_name] = np.nan
        
        return averaged_results
    
    def _calculate_comprehensive_metrics(self, y_true, y_pred, y_train):
        """Calculate comprehensive forecast accuracy metrics"""
        epsilon = 1e-8
        
        # Basic metrics
        rmse = np.sqrt(np.mean((y_true - y_pred)**2))
        mae = np.mean(np.abs(y_true - y_pred))
        
        # MAPE
        mape = np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), epsilon))) * 100
        
        # SMAPE
        smape = np.mean(2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred) + epsilon)) * 100
        
        # Theil's U1
        numerator = np.sqrt(np.mean((y_true - y_pred)**2))
        denominator = np.sqrt(np.mean(y_true**2)) + np.sqrt(np.mean(y_pred**2))
        theil_u1 = numerator / (denominator + epsilon)
        
        # Theil's U2
        naive_forecast = np.full_like(y_true, y_true[0] if len(y_true) > 0 else 0)
        theil_u2 = rmse / (np.sqrt(np.mean((y_true - naive_forecast)**2)) + epsilon)
        
        # MdAPE (Median Absolute Percentage Error)
        mdape = np.median(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), epsilon))) * 100
        
        # MASE (Mean Absolute Scaled Error)
        if len(y_train) > 1:
            seasonal_naive_mae = np.mean(np.abs(np.diff(y_train)))
            mase = mae / (seasonal_naive_mae + epsilon)
        else:
            mase = np.nan
        
        # R-squared
        ss_res = np.sum((y_true - y_pred)**2)
        ss_tot = np.sum((y_true - np.mean(y_true))**2)
        r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
        
        return {
            'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'SMAPE': smape,
            'Theil_U1': theil_u1, 'Theil_U2': theil_u2, 'MdAPE': mdape, 
            'MASE': mase, 'R2': r2
        }

    def print_variable_cv_results(self):
        """
        Print detailed variable-level cross-validation results
        """
        if not self.method_results:
            return
        
        print(f"\n{'='*150}")
        print("DETAILED VARIABLE-LEVEL CROSS-VALIDATION RESULTS")
        print("="*150)
        
        for method_name, results in self.method_results.items():
            if 'variable_cv_metrics' not in results:
                continue
                
            print(f"\nMethod: {method_name}")
            print("-" * 140)
            print(f"{'Variable':<15} {'CV RMSE':<8} {'CV MAE':<8} {'CV MAPE':<9} {'CV SMAPE':<10} "
                f"{'CV Theil_U1':<12} {'CV Theil_U2':<12} {'CV MdAPE':<10} {'CV MASE':<9} {'CV R²':<8}")  # ADDED MASE
            print("-" * 140)
            
            var_cv_metrics = results['variable_cv_metrics']
            for var_name, metrics in var_cv_metrics.items():
                print(f"{var_name:<15} {metrics.get('RMSE', np.nan):<8.4f} {metrics.get('MAE', np.nan):<8.4f} "
                    f"{metrics.get('MAPE', np.nan):<9.2f} {metrics.get('SMAPE', np.nan):<10.2f} "
                    f"{metrics.get('Theil_U1', np.nan):<12.4f} {metrics.get('Theil_U2', np.nan):<12.4f} "
                    f"{metrics.get('MdAPE', np.nan):<10.2f} {metrics.get('MASE', np.nan):<9.4f} "  # ADDED MASE
                    f"{metrics.get('R2', np.nan):<8.3f}")

    def get_model_averages(self):
        """
        Get average Theil U1, U2, and MASE for all models
        Returns a DataFrame with model names and their average metrics
        """
        if not hasattr(self, 'method_results') or not self.method_results:
            print("No results available")
            return None
        
        import pandas as pd
        results_data = []
        
        for method_name, results in self.method_results.items():
            if 'variable_cv_metrics' not in results:
                continue
                
            var_cv_metrics = results['variable_cv_metrics']
            
            # Calculate averages
            all_rmse = [metrics.get('RMSE', np.nan) for metrics in var_cv_metrics.values()]
            all_mae = [metrics.get('MAE', np.nan) for metrics in var_cv_metrics.values()]
            all_mape = [metrics.get('MAPE', np.nan) for metrics in var_cv_metrics.values()]
            all_theil_u1 = [metrics.get('Theil_U1', np.nan) for metrics in var_cv_metrics.values()]
            all_theil_u2 = [metrics.get('Theil_U2', np.nan) for metrics in var_cv_metrics.values()]
            all_mase = [metrics.get('MASE', np.nan) for metrics in var_cv_metrics.values()]
            
            results_data.append({
                'Method': method_name,
                'Avg_RMSE': np.nanmean(all_rmse),
                'Avg_MAE': np.nanmean(all_mae),
                'Avg_MAPE': np.nanmean(all_mape),
                'Avg_Theil_U1': np.nanmean(all_theil_u1),
                'Avg_Theil_U2': np.nanmean(all_theil_u2),
                'Avg_MASE': np.nanmean(all_mase),
                'Avg_R2': np.mean(list(results['variable_r2'].values())),
                'CV_RMSE': results['cv_metrics'].get('RMSE', np.nan),
                'DSGE_Informed': results.get('dsge_informed', False)
            })
        
        df = pd.DataFrame(results_data).set_index('Method')
        return df.sort_values('Avg_RMSE')  # Sort by best RMSE

    def print_comprehensive_model_summary(self):
        """
        Print comprehensive summary including all average metrics
        """
        averages_df = self.get_model_averages()
        if averages_df is None:
            return
        
        print(f"\n{'='*120}")
        print("COMPREHENSIVE MODEL COMPARISON - ALL AVERAGE METRICS")
        print("="*120)
        
        # Round for better display
        display_df = averages_df.round(4)
        
        print(display_df.to_string())
        
        print(f"\n{'='*120}")
        print("RANKING BY KEY METRICS:")
        print("="*120)
        
        print("\nBest by RMSE:")
        best_rmse = averages_df.nsmallest(3, 'Avg_RMSE')[['Avg_RMSE', 'Avg_Theil_U1', 'Avg_Theil_U2', 'Avg_MASE']]
        print(best_rmse)
        
        print("\nBest by Theil U1:")
        best_u1 = averages_df.nsmallest(3, 'Avg_Theil_U1')[['Avg_RMSE', 'Avg_Theil_U1', 'Avg_Theil_U2', 'Avg_MASE']]
        print(best_u1)
        
        print("\nBest by Theil U2:")
        best_u2 = averages_df.nsmallest(3, 'Avg_Theil_U2')[['Avg_RMSE', 'Avg_Theil_U1', 'Avg_Theil_U2', 'Avg_MASE']]
        print(best_u2)
        
        print("\nBest by MASE:")
        best_mase = averages_df.nsmallest(3, 'Avg_MASE')[['Avg_RMSE', 'Avg_Theil_U1', 'Avg_Theil_U2', 'Avg_MASE']]
        print(best_mase)

    def select_best_method(self, all_results, selection_criteria=None):
        """
        Select best method based on comprehensive forecasting and fit metrics
        """
        if not all_results:
            print("No results available for selection")
            return None, None
            
        if selection_criteria is None:
            # Simplified weighting scheme - only 4 core metrics
            selection_criteria = {
                'cv_rmse_weight': 0.30,      # Increased weight
                'mase_weight': 0.25,         # Substantial weight for MASE
                'theil_u1_weight': 0.20,     # Increased weight
                'theil_u2_weight': 0.20,     # Increased weight
                'dsge_bonus': 0.05           # Keep DSGE bonus
            }
        
        print(f"\n{'='*80}")
        print("COMPREHENSIVE METHOD SELECTION")
        print("="*80)
        
        method_scores = {}
        detailed_scores = {}
        
        for method_name, results in all_results.items():
            if 'variable_cv_metrics' not in results:
                print(f"Skipping {method_name} - incomplete metrics")
                continue
                
            # Calculate component scores
            cv_metrics = results['cv_metrics']
            var_cv_metrics = results['variable_cv_metrics']
            
            # Aggregate variable-level metrics
            all_rmse = [metrics.get('RMSE', np.nan) for metrics in var_cv_metrics.values()]
            all_mae = [metrics.get('MAE', np.nan) for metrics in var_cv_metrics.values()]
            all_mape = [metrics.get('MAPE', np.nan) for metrics in var_cv_metrics.values()]
            all_theil_u1 = [metrics.get('Theil_U1', np.nan) for metrics in var_cv_metrics.values()]
            all_theil_u2 = [metrics.get('Theil_U2', np.nan) for metrics in var_cv_metrics.values()]
            all_mase = [metrics.get('MASE', np.nan) for metrics in var_cv_metrics.values()]  # ADDED
            
            avg_var_rmse = np.nanmean(all_rmse)
            avg_var_mae = np.nanmean(all_mae)
            avg_var_mape = np.nanmean(all_mape)
            avg_theil_u1 = np.nanmean(all_theil_u1)
            avg_theil_u2 = np.nanmean(all_theil_u2)
            avg_mase = np.nanmean(all_mase)  # ADDED
            
            # Overall fit
            avg_r2 = np.mean(list(results['variable_r2'].values()))
            
            # Normalize metrics (lower is better for most, except R²)
            # Use min-max normalization across methods for fair comparison
            detailed_scores[method_name] = {
                'avg_var_rmse': avg_var_rmse,
                'avg_var_mae': avg_var_mae,
                'avg_var_mape': avg_var_mape,
                'avg_r2': avg_r2,
                'avg_theil_u1': avg_theil_u1,
                'avg_theil_u2': avg_theil_u2,
                'avg_mase': avg_mase,  # ADDED
                'dsge_informed': results.get('dsge_informed', False)
            }
        
        # Normalize scores across methods
        metric_ranges = {}
        for metric in ['avg_var_rmse', 'avg_var_mae', 'avg_var_mape', 'avg_r2', 'avg_theil_u1', 'avg_theil_u2', 'avg_mase']:  # ADDED avg_mase
            values = [scores[metric] for scores in detailed_scores.values() if not np.isnan(scores[metric])]
            if values:
                metric_ranges[metric] = {'min': min(values), 'max': max(values)}
            else:
                metric_ranges[metric] = {'min': 0, 'max': 1}
        
        # Calculate composite scores
        for method_name, scores in detailed_scores.items():
            composite_score = 0.0
            
            # RMSE component (lower is better)
            if not np.isnan(scores['avg_var_rmse']) and metric_ranges['avg_var_rmse']['max'] > metric_ranges['avg_var_rmse']['min']:
                rmse_normalized = 1 - (scores['avg_var_rmse'] - metric_ranges['avg_var_rmse']['min']) / (metric_ranges['avg_var_rmse']['max'] - metric_ranges['avg_var_rmse']['min'])
                composite_score += selection_criteria['cv_rmse_weight'] * rmse_normalized
            
            # MAE component (lower is better)
            if not np.isnan(scores['avg_var_mae']) and metric_ranges['avg_var_mae']['max'] > metric_ranges['avg_var_mae']['min']:
                mae_normalized = 1 - (scores['avg_var_mae'] - metric_ranges['avg_var_mae']['min']) / (metric_ranges['avg_var_mae']['max'] - metric_ranges['avg_var_mae']['min'])
                composite_score += selection_criteria['cv_mae_weight'] * mae_normalized
            
            # MAPE component (lower is better)
            if not np.isnan(scores['avg_var_mape']) and metric_ranges['avg_var_mape']['max'] > metric_ranges['avg_var_mape']['min']:
                mape_normalized = 1 - (scores['avg_var_mape'] - metric_ranges['avg_var_mape']['min']) / (metric_ranges['avg_var_mape']['max'] - metric_ranges['avg_var_mape']['min'])
                composite_score += selection_criteria['cv_mape_weight'] * mape_normalized
            
            # R² component (higher is better)
            if not np.isnan(scores['avg_r2']) and metric_ranges['avg_r2']['max'] > metric_ranges['avg_r2']['min']:
                r2_normalized = (scores['avg_r2'] - metric_ranges['avg_r2']['min']) / (metric_ranges['avg_r2']['max'] - metric_ranges['avg_r2']['min'])
                composite_score += selection_criteria['avg_r2_weight'] * r2_normalized
            
            # Theil U1 component (lower is better)
            if not np.isnan(scores['avg_theil_u1']) and metric_ranges['avg_theil_u1']['max'] > metric_ranges['avg_theil_u1']['min']:
                theil_u1_normalized = 1 - (scores['avg_theil_u1'] - metric_ranges['avg_theil_u1']['min']) / (metric_ranges['avg_theil_u1']['max'] - metric_ranges['avg_theil_u1']['min'])
                composite_score += selection_criteria['theil_u1_weight'] * theil_u1_normalized
            
            # Theil U2 component (lower is better)
            if not np.isnan(scores['avg_theil_u2']) and metric_ranges['avg_theil_u2']['max'] > metric_ranges['avg_theil_u2']['min']:
                theil_u2_normalized = 1 - (scores['avg_theil_u2'] - metric_ranges['avg_theil_u2']['min']) / (metric_ranges['avg_theil_u2']['max'] - metric_ranges['avg_theil_u2']['min'])
                composite_score += selection_criteria['theil_u2_weight'] * theil_u2_normalized
            
            # DSGE bonus
            if scores['dsge_informed']:
                composite_score += selection_criteria['dsge_bonus']
            
            method_scores[method_name] = composite_score
        
        # Print selection results
        print(f"{'Method':<25} {'Composite Score':<15} {'Avg RMSE':<10} {'Avg R²':<8} {'Avg Theil_U1':<12} {'Avg Theil_U2':<12} {'Avg MASE':<10} {'DSGE':<6}")
        print("-" * 105)  # EXTENDED
        
        for method_name in sorted(method_scores.keys(), key=lambda x: method_scores[x], reverse=True):
            score = method_scores[method_name]
            rmse = detailed_scores[method_name]['avg_var_rmse']
            r2 = detailed_scores[method_name]['avg_r2']
            theil_u1 = detailed_scores[method_name]['avg_theil_u1']  # ADDED
            theil_u2 = detailed_scores[method_name]['avg_theil_u2']  # ADDED
            mase = detailed_scores[method_name]['avg_mase']  # ADDED
            dsge = "Yes" if detailed_scores[method_name]['dsge_informed'] else "No"
            
            print(f"{method_name:<25} {score:<15.4f} {rmse:<10.4f} {r2:<8.3f} {theil_u1:<12.4f} {theil_u2:<12.4f} {mase:<10.4f} {dsge:<6}")
        
        # Select best method
        best_method = max(method_scores.keys(), key=lambda x: method_scores[x])
        best_score = method_scores[best_method]
        
        print(f"\nSelected Best Method: {best_method} (Score: {best_score:.4f})")
        
        return best_method, method_scores
            
    def _extract_dsge_posterior_estimates(self, base_dfm_model):
        """
        FIXED: Extract DSGE posterior estimates from the base model's DSGE estimation
        This integrates with Document 1's dsge.posterior_table results
        """
        # Try to get DSGE priors from the base model
        if hasattr(base_dfm_model, 'dsge_priors') and base_dfm_model.dsge_priors is not None:
            return base_dfm_model.dsge_priors
            
        # FIXED: If base model was created with DSGE posterior estimates, extract them
        # This assumes Document 1's DSGE estimation results are available
        if hasattr(base_dfm_model, 'factor_var') and base_dfm_model.factor_var.get('bayesian', False):
            prior_info = base_dfm_model.factor_var.get('prior_info')
            if prior_info and 'dsge_params' in prior_info:
                # Convert DSGE structural parameters to format expected by theta methods
                dsge_params = prior_info['dsge_params']
                return {
                    'sigma': {'mean': dsge_params.get('sigma', 1.25), 'std': 0.15},
                    'theta': {'mean': dsge_params.get('theta', 0.75), 'std': 0.08},
                    'phi_pi': {'mean': dsge_params.get('phi_pi', 1.65), 'std': 0.25},
                    'phi_y': {'mean': dsge_params.get('phi_y', 0.35), 'std': 0.12},
                    'rho_a': {'mean': dsge_params.get('rho_a', 0.82), 'std': 0.10},
                    'rho_v': {'mean': dsge_params.get('rho_v', 0.45), 'std': 0.18},
                }
        
        # FIXED: Try to reconstruct from any DSGE calibration in the base model
        if hasattr(base_dfm_model, 'organized_blocks'):
            # This indicates the model might have been created with DSGE context
            print("  Attempting to extract DSGE parameters from model context...")
            # Look for any stored DSGE results or use reasonable defaults based on literature
            return {
                'sigma': {'mean': 1.30, 'std': 0.20},    # Risk aversion
                'theta': {'mean': 0.75, 'std': 0.10},    # Price stickiness  
                'phi_pi': {'mean': 1.50, 'std': 0.35},   # Taylor rule - inflation
                'phi_y': {'mean': 0.25, 'std': 0.10},    # Taylor rule - output
                'rho_a': {'mean': 0.80, 'std': 0.15},    # Technology persistence
                'rho_v': {'mean': 0.50, 'std': 0.20},    # Monetary persistence
            }
            
        print("  No DSGE priors found - theta optimization will use econometric constraints only")
        return None

    # ============================================================================
    # DSGE-INFORMED THETA TRANSFORMATION METHODS (unchanged from original)
    # ============================================================================

    def _adapt_dsge_priors_to_theta(self, transformed_factors, theta_params, method_name):
        """
        Adapt DSGE priors to transformed factor space while preserving economic structure
        """
        if self.dsge_priors is None:
            return None
            
        # REMOVED: Repetitive print statement that was causing repeated output
        # Only print the method name once at the beginning
        if not hasattr(self, '_adaptation_logged'):
            print(f"  Adapting DSGE priors for {method_name} transformation...")
            self._adaptation_logged = True
        
        # Get original prior structure
        base_priors = self.base_factor_var.get('prior_info')
        if base_priors is None:
            # FIXED: Create prior structure from DSGE estimates if not available
            return self._construct_priors_from_dsge_estimates(transformed_factors, theta_params)
            
        # Calculate transformation scaling factors for each factor
        original_factors = self.factors.values
        theta_factors = transformed_factors.values if hasattr(transformed_factors, 'values') else transformed_factors
        
        scaling_factors = []
        for i in range(self.n_factors):
            orig_std = np.std(original_factors[:, i])
            theta_std = np.std(theta_factors[:, i])
            if orig_std > 0:
                scaling_factors.append(theta_std / orig_std)
            else:
                scaling_factors.append(1.0)
        
        # Adapt prior means and variances
        adapted_prior_mean = base_priors['mean'].copy()
        adapted_prior_var = base_priors['variance'].copy()
        
        # Adjust VAR coefficient priors based on transformation scaling
        K = self.n_factors
        lags = self.base_factor_var['lags']
        
        # Skip constants, adjust VAR coefficients
        coeff_start = K
        for lag in range(1, lags + 1):
            for i in range(K):  # Each factor equation
                for j in range(K):  # Each factor lag
                    coeff_idx = coeff_start + (lag-1)*K*K + i*K + j
                    if coeff_idx < len(adapted_prior_mean):
                        # Scale coefficient priors by transformation ratios
                        scale_ratio = scaling_factors[i] / scaling_factors[j]
                        adapted_prior_mean[coeff_idx] *= scale_ratio
                        # Increase uncertainty for transformed relationships
                        adapted_prior_var[coeff_idx] *= (1.0 + 0.1 * abs(1 - scale_ratio))
        
        # REMOVED: The repetitive print statement that was at the end
        
        return {
            'mean': adapted_prior_mean,
            'variance': adapted_prior_var,
            'dsge_params': base_priors.get('dsge_params', {}),
            'scaling_factors': scaling_factors
        }

    def _construct_priors_from_dsge_estimates(self, transformed_factors, theta_params):
        """
        FIXED: Construct VAR priors directly from DSGE posterior estimates when prior_info unavailable
        """
        if self.dsge_priors is None:
            return None
            
        print(f"  Constructing priors from DSGE posterior estimates...")
        
        K = self.n_factors
        lags = self.base_factor_var['lags']
        n_coeffs = K * K * lags + K  # VAR coeffs + constants
        
        # Extract key DSGE parameters (using posterior means)
        dsge_params = {}
        for param_name in ['sigma', 'theta', 'phi_pi', 'phi_y', 'rho_a', 'rho_v']:
            if param_name in self.dsge_priors:
                # Get posterior mean as point estimate
                dsge_params[param_name] = self.dsge_priors[param_name]['mean']
        
        # Construct theory-consistent priors
        prior_mean = np.zeros(n_coeffs)
        prior_var = np.ones(n_coeffs)
        
        # Constants (intercepts) - weakly informative
        prior_var[:K] = 1.0
        
        # VAR coefficient priors based on DSGE structure
        coeff_start = K
        
        for lag in range(1, lags + 1):
            for i in range(K):  # Each factor equation
                for j in range(K):  # Each factor lag
                    coeff_idx = coeff_start + (lag-1)*K*K + i*K + j
                    
                    if coeff_idx >= n_coeffs:
                        continue
                        
                    if i == j:  # Own lags
                        if lag == 1:
                            # First own lag - use DSGE persistence parameters
                            if i == 0:  # Demand factor - related to output persistence
                                prior_mean[coeff_idx] = dsge_params.get('rho_a', 0.5) * 0.7
                                prior_var[coeff_idx] = 0.1
                            elif i == 1:  # Supply factor - related to price stickiness
                                stickiness = dsge_params.get('theta', 0.7)
                                prior_mean[coeff_idx] = min(0.9, stickiness * 1.2)
                                prior_var[coeff_idx] = 0.1
                            else:  # Monetary factor - use monetary persistence
                                prior_mean[coeff_idx] = dsge_params.get('rho_v', 0.3)
                                prior_var[coeff_idx] = 0.15
                        else:
                            # Higher order own lags - decay
                            first_lag_idx = coeff_start + (i*K + j)
                            if first_lag_idx < len(prior_mean):
                                prior_mean[coeff_idx] = prior_mean[first_lag_idx] * 0.5
                            prior_var[coeff_idx] = 0.2
                    else:  # Cross-lags
                        if lag == 1:
                            # Theory-based cross-effects
                            if (i == 0 and j == 2):  # Monetary -> Demand (IS curve)
                                prior_mean[coeff_idx] = -1.0 / dsge_params.get('sigma', 1.0) * 0.1
                                prior_var[coeff_idx] = 0.05
                            elif (i == 1 and j == 0):  # Demand -> Supply (Phillips curve)  
                                prior_mean[coeff_idx] = dsge_params.get('phi_y', 0.5) * 0.2
                                prior_var[coeff_idx] = 0.05
                            elif (i == 2 and j == 1):  # Supply -> Monetary (Taylor rule)
                                prior_mean[coeff_idx] = dsge_params.get('phi_pi', 1.5) * 0.1
                                prior_var[coeff_idx] = 0.05
                            else:
                                prior_mean[coeff_idx] = 0.0
                                prior_var[coeff_idx] = 0.1
                        else:
                            # Higher order cross-lags - shrink to zero
                            prior_mean[coeff_idx] = 0.0
                            prior_var[coeff_idx] = 0.1 / lag
        
        return {
            'mean': prior_mean,
            'variance': prior_var,
            'dsge_params': dsge_params
        }

    def _estimate_theta_aware_var(self, transformed_factors, theta_params, method_name):
        """
        Estimate VAR on transformed factors using adapted DSGE priors
        """
        F_theta = transformed_factors.values if hasattr(transformed_factors, 'values') else transformed_factors
        
        # Get adapted DSGE priors
        adapted_priors = self._adapt_dsge_priors_to_theta(transformed_factors, theta_params, method_name)
        
        if adapted_priors is not None and self.dsge_priors is not None:
            # Bayesian estimation with adapted priors
            return self._estimate_bayesian_var_theta(F_theta, adapted_priors)
        else:
            # OLS fallback
            return self._estimate_var_ols(F_theta, self.base_factor_var['lags'])

    def _estimate_bayesian_var_theta(self, F_theta, adapted_priors):
        """
        Bayesian VAR estimation for theta-transformed factors with adapted DSGE priors
        """
        T, K = F_theta.shape
        lags = self.base_factor_var['lags']
        
        # Construct regression matrices
        Y = F_theta[lags:, :]
        T_eff = Y.shape[0]
        
        # Design matrix
        X = np.ones((T_eff, 1))  # Constant
        for lag in range(1, lags + 1):
            X = np.hstack([X, F_theta[lags-lag:-lag, :]])
        
        n_regressors = X.shape[1]
        
        # Prior parameters (adapted)
        beta_prior = adapted_priors['mean'][:n_regressors*K].reshape(-1, 1)
        V_prior = np.diag(adapted_priors['variance'][:n_regressors*K])
        
        # Bayesian updating
        V_prior_inv = np.linalg.inv(V_prior)
        XtX = X.T @ X
        
        # Posterior covariance and mean
        V_posterior_inv = V_prior_inv + np.kron(np.eye(K), XtX)
        V_posterior = np.linalg.inv(V_posterior_inv)
        
        XtY = X.T @ Y
        beta_posterior_vec = V_posterior @ (V_prior_inv @ beta_prior + XtY.flatten('F').reshape(-1, 1))
        beta_posterior = beta_posterior_vec.reshape((n_regressors, K), order='F')
        
        # Residuals and covariance
        Y_fitted = X @ beta_posterior
        residuals = Y - Y_fitted
        sigma_u = (residuals.T @ residuals) / T_eff
        
        # Coefficient matrices
        const = beta_posterior[0, :]
        var_coeffs = beta_posterior[1:, :].T
        
        # Model statistics
        log_likelihood = self._compute_log_likelihood(residuals, sigma_u)
        
        return {
            'coefficients': var_coeffs,
            'constant': const,
            'sigma_u': sigma_u,
            'residuals': residuals,
            'log_likelihood': log_likelihood,
            'lags': lags,
            'T_eff': T_eff,
            'bayesian': True,
            'adapted_priors': adapted_priors,
            'theta_informed': True
        }

    def _estimate_var_ols(self, F_theta, lags):
        """OLS VAR estimation fallback"""
        T, K = F_theta.shape
        
        # Construct regression matrices
        Y = F_theta[lags:, :]
        T_eff = Y.shape[0]
        
        X = np.ones((T_eff, 1))
        for lag in range(1, lags + 1):
            X = np.hstack([X, F_theta[lags-lag:-lag, :]])
        
        # OLS estimation
        XtX_inv = np.linalg.inv(X.T @ X)
        beta = XtX_inv @ X.T @ Y
        
        u = Y - X @ beta
        sigma_u = (u.T @ u) / T_eff
        
        const = beta[0, :]
        var_coeffs = beta[1:, :].T
        
        log_likelihood = self._compute_log_likelihood(u, sigma_u)
        
        return {
            'coefficients': var_coeffs,
            'constant': const,
            'sigma_u': sigma_u,
            'residuals': u,
            'log_likelihood': log_likelihood,
            'lags': lags,
            'T_eff': T_eff,
            'bayesian': False,
            'theta_informed': True
        }

    def _compute_log_likelihood(self, residuals, sigma_u):
        """Compute VAR log-likelihood"""
        T, K = residuals.shape
        log_likelihood = -0.5 * T * K * np.log(2 * np.pi)
        log_likelihood -= 0.5 * T * np.log(np.linalg.det(sigma_u))
        log_likelihood -= 0.5 * np.trace(residuals.T @ residuals @ np.linalg.inv(sigma_u))
        return log_likelihood

    # ============================================================================
    # THETA TRANSFORMATION METHODS (unchanged from original)
    # ============================================================================

    def dsge_constrained_hp_method(self):
        """
        HP filter with DSGE-consistent constraints on trend-cycle decomposition
        """
        print("Running DSGE-Constrained HP Trend-Cycle Method...")
        
        def theta_transform(factors, theta_params):
            F_theta_df = factors.copy()
            for i, factor_name in enumerate(factors.columns):
                trend, cycle = self._hp_filter(factors[factor_name])
                theta1, theta2 = theta_params[i], theta_params[i + self.n_factors]
                F_theta_df[factor_name] = theta1 * trend + theta2 * cycle
            return F_theta_df
        
        def objective(theta_vector):
            theta1s = theta_vector[:self.n_factors]  # Trend weights
            theta2s = theta_vector[self.n_factors:]  # Cycle weights
            
            # DSGE-consistent constraints
            if self.dsge_priors is not None:
                # Use DSGE persistence parameters to guide trend-cycle weights
                dsge_params = {k: v['mean'] for k, v in self.dsge_priors.items()}
                
                # High persistence factors should weight trends more heavily
                persistence_penalty = 0.0
                for i in range(self.n_factors):
                    if i == 0:  # Demand factor
                        target_persistence = dsge_params.get('rho_a', 0.5)
                        trend_target = 0.3 + 0.4 * target_persistence
                        cycle_target = 1.2 - 0.3 * target_persistence
                    elif i == 1:  # Supply factor  
                        price_stickiness = dsge_params.get('theta', 0.7)
                        trend_target = 0.2 + 0.5 * price_stickiness
                        cycle_target = 1.3 - 0.4 * price_stickiness
                    else:  # Monetary factor
                        monetary_persistence = dsge_params.get('rho_v', 0.3)
                        trend_target = 0.4 + 0.3 * monetary_persistence
                        cycle_target = 1.1 - 0.2 * monetary_persistence
                    
                    persistence_penalty += 0.3 * ((theta1s[i] - trend_target)**2 + 
                                                 (theta2s[i] - cycle_target)**2)
            else:
                # Standard economic constraints if no DSGE priors
                persistence_penalty = 0.2 * (np.sum((theta1s - 0.4)**2) + 
                                            np.sum((theta2s - 1.1)**2))
            
            # Complementary weighting
            target_sum = 1.5
            complementary_penalty = 0.4 * np.sum((theta1s + theta2s - target_sum)**2)
            
            # Estimate VAR on transformed factors
            F_theta = theta_transform(self.factors, theta_vector)
            theta_var = self._estimate_theta_aware_var(F_theta, theta_vector, 'HP_Trend_Cycle')
            
            # Use original loadings (don't re-estimate)
            Y_reconstructed = F_theta.values @ self.loadings.values.T
            reconstruction_error = np.mean((self.data_standardized.values - Y_reconstructed)**2)
            
            return reconstruction_error + persistence_penalty + complementary_penalty
        
        # DSGE-informed bounds
        if self.dsge_priors is not None:
            dsge_params = {k: v['mean'] for k, v in self.dsge_priors.items()}
            bounds = []
            
            for i in range(self.n_factors):
                if i == 0:  # Demand - based on technology persistence
                    persistence = dsge_params.get('rho_a', 0.5)
                    bounds.append((0.2, 0.3 + 0.5 * persistence))
                elif i == 1:  # Supply - based on price stickiness
                    stickiness = dsge_params.get('theta', 0.7)
                    bounds.append((0.1, 0.4 + 0.4 * stickiness))
                else:  # Monetary
                    bounds.append((0.2, 0.6))
            
            for i in range(self.n_factors):
                bounds.append((0.8, 1.8))  # Cycle weights
                
            initial_guess = []
            for i in range(self.n_factors):
                initial_guess.append((bounds[i][0] + bounds[i][1]) / 2)
            initial_guess.extend([1.2] * self.n_factors)
        else:
            # Standard bounds
            bounds = [(0.2, 0.7) for _ in range(self.n_factors)] + [(0.8, 2.0) for _ in range(self.n_factors)]
            initial_guess = [0.4] * self.n_factors + [1.1] * self.n_factors
        
        result = minimize(objective, initial_guess, bounds=bounds, method='L-BFGS-B')
        return result.x if result.success else initial_guess, theta_transform

    def dsge_regime_switching_method(self):
        """
        Regime-switching transformation informed by DSGE structural breaks
        """
        print("Running DSGE-Informed Regime Switching Method...")
        
        def theta_transform(factors, theta_params):
            F_theta_df = factors.copy()
            for i, factor_name in enumerate(factors.columns):
                factor_vals = factors[factor_name].values
                theta1, theta2, threshold = (theta_params[i], 
                                           theta_params[i + self.n_factors],
                                           theta_params[i + 2*self.n_factors])
                
                # Piecewise linear with DSGE-consistent regime interpretation
                F_theta_df[factor_name] = np.where(factor_vals <= threshold,
                                                 theta1 * factor_vals,
                                                 theta1 * threshold + theta2 * (factor_vals - threshold))
            return F_theta_df
        
        def objective(theta_vector):
            theta1s = theta_vector[:self.n_factors]
            theta2s = theta_vector[self.n_factors:2*self.n_factors]
            thresholds = theta_vector[2*self.n_factors:]
            
            # DSGE-informed regime constraints
            if self.dsge_priors is not None:
                dsge_params = {k: v['mean'] for k, v in self.dsge_priors.items()}
                
                # Different regimes should reflect DSGE parameter uncertainty
                regime_penalty = 0.0
                for i in range(self.n_factors):
                    if i == 0:  # Demand factor - high vs low growth regimes
                        sigma_param = dsge_params.get('sigma', 1.0)
                        regime1_target = 0.8 - 0.2 * (sigma_param - 1.0)  # Risk aversion effect
                        regime2_target = 1.2 + 0.3 * (sigma_param - 1.0)
                    elif i == 1:  # Supply factor - flexible vs sticky regimes
                        theta_param = dsge_params.get('theta', 0.7)
                        regime1_target = 0.6 + 0.4 * theta_param  # Sticky regime
                        regime2_target = 1.4 - 0.3 * theta_param  # Flexible regime
                    else:  # Monetary factor - accommodative vs restrictive
                        phi_pi = dsge_params.get('phi_pi', 1.5)
                        regime1_target = 0.7 + 0.1 * (phi_pi - 1.5)  # Accommodative
                        regime2_target = 1.3 + 0.2 * (phi_pi - 1.5)  # Restrictive
                    
                    regime_penalty += 0.4 * ((theta1s[i] - regime1_target)**2 + 
                                           (theta2s[i] - regime2_target)**2)
            else:
                regime_penalty = 0.3 * (np.sum((theta1s - 0.7)**2) + np.sum((theta2s - 1.3)**2))
            
            # Minimum regime separation
            separation_penalty = 0.6 * np.sum(np.maximum(0, 0.3 - np.abs(theta1s - theta2s))**2)
            
            F_theta = theta_transform(self.factors, theta_vector)
            theta_var = self._estimate_theta_aware_var(F_theta, theta_vector, 'Regime_Switching')
            
            Y_reconstructed = F_theta.values @ self.loadings.values.T
            reconstruction_error = np.mean((self.data_standardized.values - Y_reconstructed)**2)
            
            return reconstruction_error + regime_penalty + separation_penalty
        
        # Set bounds using factor medians and DSGE guidance
        factor_medians = [self.factors.iloc[:, i].median() for i in range(self.n_factors)]
        
        if self.dsge_priors is not None:
            dsge_params = {k: v['mean'] for k, v in self.dsge_priors.items()}
            bounds = []
            
            # Regime 1 bounds (informed by DSGE)
            for i in range(self.n_factors):
                if i == 0:  # Demand
                    bounds.append((0.5, 1.0))
                elif i == 1:  # Supply
                    theta_param = dsge_params.get('theta', 0.7)
                    bounds.append((0.4 + 0.2 * theta_param, 1.0 + 0.2 * theta_param))
                else:  # Monetary
                    bounds.append((0.6, 1.1))
            
            # Regime 2 bounds
            for i in range(self.n_factors):
                bounds.append((1.0, 1.8))
            
            # Threshold bounds
            for med in factor_medians:
                bounds.append((med - 0.6, med + 0.6))
        else:
            bounds = ([(0.4, 1.0) for _ in range(self.n_factors)] +
                     [(1.0, 1.8) for _ in range(self.n_factors)] +
                     [(med - 0.8, med + 0.8) for med in factor_medians])
        
        initial_guess = ([0.7] * self.n_factors + [1.3] * self.n_factors + factor_medians)
        
        result = minimize(objective, initial_guess, bounds=bounds, method='L-BFGS-B')
        return result.x if result.success else initial_guess, theta_transform

    def dsge_time_varying_method(self):
        """
        Time-varying theta with evolution guided by DSGE parameter stability
        """
        print("Running DSGE-Informed Time-Varying Method...")
        
        def theta_transform(factors, theta_params):
            F_theta_df = factors.copy()
            T = len(factors)
            
            for i, factor_name in enumerate(factors.columns):
                theta_init = theta_params[i]
                theta_final = theta_params[i + self.n_factors]
                midpoint = theta_params[i + 2*self.n_factors]
                speed = theta_params[i + 3*self.n_factors]
                
                t_vals = np.arange(T)
                theta_t = theta_init + (theta_final - theta_init) / (1 + np.exp(-speed * (t_vals - midpoint)))
                
                trend = self._extract_trend_linear(factors[factor_name])
                deviation = factors[factor_name] - trend
                F_theta_df[factor_name] = trend + theta_t * deviation
                
            return F_theta_df
        
        def objective(theta_vector):
            theta_inits = theta_vector[:self.n_factors]
            theta_finals = theta_vector[self.n_factors:2*self.n_factors]
            midpoints = theta_vector[2*self.n_factors:3*self.n_factors]
            speeds = theta_vector[3*self.n_factors:]
            
            # DSGE-consistent time variation
            if self.dsge_priors is not None:
                dsge_params = {k: v['mean'] for k, v in self.dsge_priors.items()}
                
                evolution_penalty = 0.0
                for i in range(self.n_factors):
                    if i == 0:  # Demand factor evolution
                        tech_persistence = dsge_params.get('rho_a', 0.5)
                        init_target = 0.7 + 0.2 * tech_persistence
                        final_target = 1.1 + 0.1 * (1 - tech_persistence)
                    elif i == 1:  # Supply factor evolution
                        price_flexibility = 1 - dsge_params.get('theta', 0.7)
                        init_target = 0.8 - 0.2 * price_flexibility
                        final_target = 1.2 + 0.3 * price_flexibility
                    else:  # Monetary factor evolution
                        policy_response = dsge_params.get('phi_pi', 1.5) - 1.0
                        init_target = 0.6 + 0.2 * policy_response
                        final_target = 1.0 + 0.2 * policy_response
                    
                    evolution_penalty += 0.3 * ((theta_inits[i] - init_target)**2 + 
                                               (theta_finals[i] - final_target)**2)
            else:
                evolution_penalty = 0.2 * (np.sum((theta_inits - 0.8)**2) + np.sum((theta_finals - 1.2)**2))
            
            # Minimum meaningful evolution
            min_evolution = 0.25
            evolution_constraint = 0.5 * np.sum(np.maximum(0, min_evolution - np.abs(theta_finals - theta_inits))**2)
            
            # Speed constraints - not too abrupt for economic factors
            speed_penalty = 0.2 * (np.sum(np.maximum(0, 0.03 - speeds)**2) + 
                                  np.sum(np.maximum(0, speeds - 0.10)**2))
            
            F_theta = theta_transform(self.factors, theta_vector)
            theta_var = self._estimate_theta_aware_var(F_theta, theta_vector, 'Time_Varying')
            
            Y_reconstructed = F_theta.values @ self.loadings.values.T
            reconstruction_error = np.mean((self.data_standardized.values - Y_reconstructed)**2)
            
            return reconstruction_error + evolution_penalty + evolution_constraint + speed_penalty
        
        T_mid = self.T_obs // 2
        
        if self.dsge_priors is not None:
            # DSGE-informed bounds and initialization
            dsge_params = {k: v['mean'] for k, v in self.dsge_priors.items()}
            
            init_bounds = []
            final_bounds = []
            
            for i in range(self.n_factors):
                if i == 0:  # Demand
                    persistence = dsge_params.get('rho_a', 0.5)
                    init_bounds.append((0.5 + 0.2 * persistence, 1.0 + 0.1 * persistence))
                    final_bounds.append((0.9, 1.3 + 0.2 * (1 - persistence)))
                elif i == 1:  # Supply
                    flexibility = 1 - dsge_params.get('theta', 0.7)
                    init_bounds.append((0.6, 1.0 - 0.1 * flexibility))
                    final_bounds.append((1.0 + 0.2 * flexibility, 1.5 + 0.1 * flexibility))
                else:  # Monetary
                    init_bounds.append((0.5, 0.9))
                    final_bounds.append((0.9, 1.3))
            
            bounds = (init_bounds + final_bounds + 
                     [(T_mid*0.7, T_mid*1.3) for _ in range(self.n_factors)] +
                     [(0.04, 0.08) for _ in range(self.n_factors)])
            
            initial_guess = []
            for i in range(self.n_factors):
                initial_guess.append((init_bounds[i][0] + init_bounds[i][1]) / 2)
            for i in range(self.n_factors):
                initial_guess.append((final_bounds[i][0] + final_bounds[i][1]) / 2)
            initial_guess.extend([T_mid] * self.n_factors)
            initial_guess.extend([0.06] * self.n_factors)
        else:
            bounds = ([(0.5, 1.1) for _ in range(self.n_factors)] +
                     [(0.9, 1.5) for _ in range(self.n_factors)] +
                     [(T_mid*0.6, T_mid*1.4) for _ in range(self.n_factors)] +
                     [(0.03, 0.12) for _ in range(self.n_factors)])
            
            initial_guess = ([0.8] * self.n_factors + [1.2] * self.n_factors +
                           [T_mid] * self.n_factors + [0.06] * self.n_factors)
        
        result = minimize(objective, initial_guess, bounds=bounds, method='L-BFGS-B')
        return result.x if result.success else initial_guess, theta_transform

    # ============================================================================
    # HELPER METHODS
    # ============================================================================

    def _hp_filter(self, series, lamb=1600):
        """Hodrick-Prescott filter"""
        n = len(series)
        I = np.eye(n)
        D2 = np.diff(np.eye(n), 2, axis=0)
        trend = np.linalg.solve(I + lamb * D2.T @ D2, series.values)
        cycle = series.values - trend
        return pd.Series(trend, index=series.index), pd.Series(cycle, index=series.index)

    def _extract_trend_linear(self, series):
        """Extract linear trend"""
        n = len(series)
        x = np.arange(n)
        X = np.column_stack([np.ones(n), x])
        coeffs = np.linalg.lstsq(X, series.values, rcond=None)[0]
        return pd.Series(coeffs[0] + coeffs[1] * x, index=series.index)

    # ============================================================================
    # FORECASTING AND EVALUATION (unchanged from original)
    # ============================================================================

    def evaluate_theta_method(self, method_name, theta_params, theta_func, horizon=8):
        """
        Evaluate theta method using DSGE-consistent approach:
        MODIFIED: Now includes detailed variable-level CV analysis
        """
        print(f"Evaluating {method_name} with DSGE consistency...")
        
        # Apply theta transformation
        F_theta = theta_func(self.factors, theta_params)
        
        # Estimate theta-aware VAR with adapted DSGE priors
        theta_var_model = self._estimate_theta_aware_var(F_theta, theta_params, method_name)
        
        # Calculate fit using ORIGINAL loadings (preserve economic structure)
        Lambda_original = self.loadings.values  # N x K (PRESERVE)
        Y_fitted = F_theta.values @ Lambda_original.T
        residuals = self.data_standardized.values - Y_fitted
        
        # Variable R-squared with original loadings
        variable_r2 = {}
        for i, var_name in enumerate(self.data_standardized.columns):
            ss_res = np.sum(residuals[:, i]**2)
            ss_tot = np.sum((self.data_standardized.values[:, i] - 
                        np.mean(self.data_standardized.values[:, i]))**2)
            r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
            variable_r2[var_name] = max(0, r2)
        
        # Generate forecasts using theta-aware VAR
        theta_factor_forecasts = self._forecast_theta_var(F_theta, theta_var_model, horizon)
        
        # Variable forecasts using ORIGINAL loadings
        F_theta_forecast = theta_factor_forecasts['mean'].values
        X_forecast_std = F_theta_forecast @ Lambda_original.T
        
        # Transform back to original scale
        X_forecast_original = self.scaler.inverse_transform(X_forecast_std)
        
        # Create forecast DataFrame
        forecast_dates = pd.date_range(
            start=self.factors.index[-1] + pd.DateOffset(months=3),
            periods=horizon, freq='Q'
        )
        
        variable_forecasts = pd.DataFrame(
            X_forecast_original,
            index=forecast_dates,
            columns=self.combined_data.columns
        )
        
        # Cross-validation metrics (DSGE-aware)
        cv_metrics = self._cross_validate_dsge_theta_method(method_name, theta_func, theta_params)
        
        # ADDED: Detailed variable-level CV analysis like Document 2
        variable_cv_metrics = self.detailed_variable_cv_analysis(method_name, theta_params, theta_func)
        
        return {
            'theta_var_model': theta_var_model,
            'variable_r2': variable_r2,
            'forecasts': variable_forecasts,
            'factor_forecasts': theta_factor_forecasts,
            'cv_metrics': cv_metrics,
            'variable_cv_metrics': variable_cv_metrics,  # ADDED THIS LINE
            'theta_params': theta_params,
            'theta_func': theta_func,
            'preserved_loadings': True,
            'dsge_informed': self.dsge_priors is not None
        }

    def _forecast_theta_var(self, F_theta, theta_var_model, horizon):
        """Generate forecasts using theta-aware VAR model"""
        Phi = theta_var_model['coefficients']
        c = theta_var_model['constant']
        p = theta_var_model['lags']
        K = Phi.shape[0]

        forecasts = np.zeros((horizon, K))

        if p == 1:
            F_last = F_theta.values[-1, :]
            for h in range(horizon):
                if h == 0:
                    forecasts[h, :] = c + Phi @ F_last
                else:
                    forecasts[h, :] = c + Phi @ forecasts[h-1, :]
        else:
            F_history = F_theta.values[-p:, :].copy()
            for h in range(horizon):
                F_lagged = F_history.flatten('F')
                F_forecast = c + Phi @ F_lagged
                forecasts[h, :] = F_forecast
                F_history = np.vstack([F_history[1:, :], F_forecast.reshape(1, -1)])

        forecast_dates = pd.date_range(
            start=self.factors.index[-1] + pd.DateOffset(months=3),
            periods=horizon, freq='Q'
        )
        
        return {
            'mean': pd.DataFrame(forecasts, index=forecast_dates, columns=self.factors.columns)
        }

    def _cross_validate_dsge_theta_method(self, method_name, theta_func, theta_params):
        """Cross-validate theta method preserving DSGE structure throughout"""
        cv_metrics = []
        
        for train_idx, test_idx in self.tscv.split(self.factors):
            if len(test_idx) < 2:
                continue
                
            # Split data maintaining temporal structure
            factors_train = self.factors.iloc[train_idx]
            factors_test = self.factors.iloc[test_idx]
            data_train = self.data_standardized.iloc[train_idx]
            data_test = self.data_standardized.iloc[test_idx]
            
            try:
                # Apply theta transformation to training factors
                F_theta_train = theta_func(factors_train, theta_params)
                F_theta_test = theta_func(factors_test, theta_params)
                
                # Estimate theta-aware VAR on training data with DSGE priors
                theta_var_cv = self._estimate_theta_aware_var(F_theta_train, theta_params, method_name)
                
                # Generate forecasts using theta VAR
                test_forecasts = self._forecast_theta_var(F_theta_test, theta_var_cv, len(test_idx))
                F_theta_forecast = test_forecasts['mean'].values
                
                # Use ORIGINAL loadings for variable forecasts (preserve structure)
                X_pred = F_theta_forecast @ self.loadings.values.T
                
                # Calculate metrics
                metrics = self._calculate_forecast_metrics(data_test.values, X_pred)
                cv_metrics.append(metrics)
                
            except Exception as e:
                print(f"CV fold failed for {method_name}: {e}")
                continue
        
        if cv_metrics:
            # Average across folds
            avg_metrics = {}
            for metric in cv_metrics[0].keys():
                values = [fold[metric] for fold in cv_metrics if not np.isnan(fold[metric])]
                avg_metrics[metric] = np.mean(values) if values else np.nan
            return avg_metrics
        
        return {'RMSE': np.nan, 'MAE': np.nan, 'MAPE': np.nan, 'R2': np.nan}

    def _calculate_forecast_metrics(self, y_true, y_pred):
        """Calculate forecast accuracy metrics"""
        y_true_flat = y_true.flatten()
        y_pred_flat = y_pred.flatten()
        
        mask = ~(np.isnan(y_true_flat) | np.isnan(y_pred_flat))
        y_true_clean = y_true_flat[mask]
        y_pred_clean = y_pred_flat[mask]
        
        if len(y_true_clean) == 0:
            return {'RMSE': np.nan, 'MAE': np.nan, 'MAPE': np.nan, 'R2': np.nan}
        
        # Basic metrics
        rmse = np.sqrt(np.mean((y_true_clean - y_pred_clean)**2))
        mae = np.mean(np.abs(y_true_clean - y_pred_clean))
        
        # MAPE
        epsilon = 1e-8
        mape = np.mean(np.abs((y_true_clean - y_pred_clean) / 
                            np.maximum(np.abs(y_true_clean), epsilon))) * 100
        
        # R-squared
        ss_res = np.sum((y_true_clean - y_pred_clean)**2)
        ss_tot = np.sum((y_true_clean - np.mean(y_true_clean))**2)
        r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
        
        return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'R2': r2}

    # ============================================================================
    # MAIN EXECUTION METHODS (unchanged from original)
    # ============================================================================

    def run_dsge_theta_comparison(self, horizon=8):
        """Run all DSGE-informed theta methods and compare results"""
        print("\n" + "="*100)
        print("DSGE-INFORMED THETA METHOD COMPARISON")
        print("="*100)
        print(f"Using {'Bayesian' if self.dsge_priors else 'OLS'} VAR with DSGE structure preservation")

        methods = {
            'DSGE_HP_Trend_Cycle': self.dsge_constrained_hp_method,
            'DSGE_Regime_Switching': self.dsge_regime_switching_method,
            'DSGE_Time_Varying': self.dsge_time_varying_method,
        }

        all_results = {}

        for method_name, method_func in methods.items():
            print(f"\n--- {method_name} Method ---")
            try:
                # Get optimal parameters and transformation function
                theta_params, theta_func = method_func()
                
                # Evaluate method with DSGE consistency
                results = self.evaluate_theta_method(method_name, theta_params, theta_func, horizon)
                all_results[method_name] = results
                
                # Print summary
                avg_r2 = np.mean(list(results['variable_r2'].values()))
                cv_rmse = results['cv_metrics']['RMSE']
                dsge_informed = "Yes" if results['dsge_informed'] else "No"
                print(f"Average R²: {avg_r2:.3f}")
                print(f"CV RMSE: {cv_rmse:.4f}")
                print(f"DSGE-Informed: {dsge_informed}")
                print(f"Original Loadings Preserved: {results['preserved_loadings']}")
                
            except Exception as e:
                print(f"Method {method_name} failed: {str(e)}")
                continue

        print(f"Debug: run_dsge_theta_comparison returning: {list(all_results.keys())}")
        return all_results# REMOVED: No longer select best method here - return all results

    def print_dsge_theta_results(self):
        """
        Print comprehensive results preserving DSGE structure information
        MODIFIED: Now includes call to variable-level CV results
        """
        if not hasattr(self, 'method_results') or not self.method_results:
            print("No results available")
            print(f"Debug: method_results = {getattr(self, 'method_results', 'NOT SET')}")
            return
        
        print("\n" + "="*140)
        print("DSGE-INFORMED THETA METHOD COMPARISON RESULTS")
        print("="*140)
        
        # Summary table
        print(f"\n{'Method':<25} {'Avg R²':<8} {'CV RMSE':<10} {'CV MAE':<10} {'CV MAPE':<10} {'DSGE':<6} {'Loadings':<9}")
        print("-" * 80)
        
        for method_name, results in self.method_results.items():
            avg_r2 = np.mean(list(results['variable_r2'].values()))
            cv_metrics = results['cv_metrics']
            dsge_flag = "Yes" if results.get('dsge_informed', False) else "No"
            loadings_flag = "Original" if results.get('preserved_loadings', False) else "Re-est"
            
            print(f"{method_name:<25} {avg_r2:<8.3f} {cv_metrics['RMSE']:<10.4f} "
                f"{cv_metrics['MAE']:<10.4f} {cv_metrics['MAPE']:<10.2f} "
                f"{dsge_flag:<6} {loadings_flag:<9}")
        
        # DSGE-specific information
        if self.dsge_priors is not None:
            print(f"\n\n{'='*80}")
            print("DSGE PRIOR INTEGRATION SUMMARY")
            print("="*80)
            
            dsge_params = {k: v['mean'] for k, v in self.dsge_priors.items()}
            if dsge_params:
                print("DSGE Parameters Used:")
                for param, value in dsge_params.items():
                    print(f"  {param}: {value:.3f}")
            
            print(f"\nStructural Consistency:")
            print(f"  • Original factor loadings preserved: YES")
            print(f"  • VAR priors adapted to theta transformations: YES") 
            print(f"  • Economic interpretation maintained: YES")
            print(f"  • DSGE constraints applied to theta parameters: YES")
        
        # Best method detailed results
        if self.theta_strategy:
            best_results = self.method_results[self.theta_strategy]
            print(f"\n\nDETAILED RESULTS FOR BEST METHOD: {self.theta_strategy}")
            print("="*80)
            
            print("\nVariable Fit (using original DSGE-informed loadings):")
            print("-"*60)
            print(f"{'Variable':<15} {'R²':<8} {'Factor_1':<12} {'Factor_2':<12} {'Factor_3':<12}")
            print("-" * 60)
            
            original_loadings = self.loadings  # These are the DSGE-informed loadings
            for var_name in original_loadings.index:
                r2 = best_results['variable_r2'].get(var_name, 0.0)
                print(f"{var_name:<15} {r2:<8.3f}", end="")
                for j, factor in enumerate(original_loadings.columns):
                    if j < 3:
                        loading_val = original_loadings.loc[var_name, factor]
                        print(f" {loading_val:<12.3f}", end="")
                print()
            
            avg_r2_best = np.mean(list(best_results['variable_r2'].values()))
            print(f"\nAverage R² (Best Method): {avg_r2_best:.3f}")
            
            # VAR model comparison
            theta_var = best_results['theta_var_model']
            base_var = self.base_factor_var
            
            print(f"\nVAR Model Comparison:")
            print(f"  Base VAR Log-Likelihood: {base_var['log_likelihood']:.2f}")
            print(f"  Theta VAR Log-Likelihood: {theta_var['log_likelihood']:.2f}")
            print(f"  Improvement: {theta_var['log_likelihood'] - base_var['log_likelihood']:.2f}")
        
        # ADDED: Call to print detailed variable-level CV results
        self.print_variable_cv_results()

    def run_complete_dsge_theta_analysis(self, strategy='auto', forecast_horizon=8):
        """Run complete DSGE-informed theta analysis"""
        print("DSGE-INFORMED ENHANCED DFM-THETA ANALYSIS")
        print("=" * 80)
        
        if self.dsge_priors is not None:
            print("✓ Using DSGE structural priors for VAR coefficient adaptation")
            print("✓ Preserving original factor loadings throughout")
            print("✓ Maintaining economic interpretation of factors")
        else:
            print("! No DSGE priors available - using econometric constraints only")

        # Step 1: Run method comparison
        if strategy == 'auto':
            all_results = self.run_dsge_theta_comparison(forecast_horizon)
            if not all_results:
                print("No successful methods found, analysis cannot continue")
                return None
                
            # Store results first
            self.method_results = all_results
            print(f"Debug: After assignment, method_results has: {list(self.method_results.keys())}")

            # NEW: Select best method after all analysis is complete
            best_method, method_scores = self.select_best_method(all_results)
            print(f"Debug: After select_best_method, method_results has: {list(self.method_results.keys())}")

            # Set the selected method attributes
            self.theta_strategy = best_method
            if best_method in all_results:
                self.optimal_theta = all_results[best_method]['theta_params']
                
        else:
            # Run specific method (unchanged)
            strategy_map = {
                'dsge_hp_trend_cycle': self.dsge_constrained_hp_method,
                'dsge_regime_switching': self.dsge_regime_switching_method,
                'dsge_time_varying': self.dsge_time_varying_method,
            }
            
            if strategy in strategy_map:
                method_func = strategy_map[strategy]
                theta_params, theta_func = method_func()
                results = self.evaluate_theta_method(strategy.title(), theta_params, theta_func, forecast_horizon)
                self.method_results = {strategy: results}
                self.theta_strategy = strategy
                self.optimal_theta = theta_params
            else:
                print(f"Unknown strategy: {strategy}")
                return None

        # Step 2: Print comprehensive results
        self.print_dsge_theta_results()

        # Step 3: Generate enhanced forecasts
        if self.theta_strategy and self.theta_strategy in self.method_results:
            best_results = self.method_results[self.theta_strategy]
            
            print(f"\n\nDSGE-INFORMED FORECAST SUMMARY ({forecast_horizon} periods ahead):")
            print("="*90)
            
            forecasts_df = best_results['forecasts']
            
            print(f"\n{'Variable':<20} {'Latest Forecast':<15} {'CV RMSE':<10} {'Original R²':<12}")
            print("-" * 60)
            
            for var in forecasts_df.columns:
                latest_forecast = forecasts_df[var].iloc[-1]
                cv_rmse = best_results['cv_metrics'].get('RMSE', np.nan)
                fit_r2 = best_results['variable_r2'].get(var, 0.0)
                
                print(f"{var:<20} {latest_forecast:>14.2f} {cv_rmse:>9.4f} {fit_r2:>11.3f}")

        print("\nDSGE-INFORMED ENHANCED DFM-THETA ANALYSIS COMPLETE")
        print("Key advantages:")
        print("• Theta transformations respect DSGE economic structure")
        print("• VAR priors properly adapted to transformed factor space") 
        print("• Original loadings preserved (no loss of economic interpretation)")
        print("• Cross-validation accounts for DSGE temporal structure")

        return {
            'strategy_used': self.theta_strategy,
            'optimal_theta': self.optimal_theta,
            'method_results': self.method_results,
            'best_method_results': self.method_results.get(self.theta_strategy, {}),
            'forecasts': self.method_results.get(self.theta_strategy, {}).get('forecasts', None),
            'preserved_structure': True,
            'dsge_informed': self.dsge_priors is not None
        }


# ============================================================================
# CONVENIENCE FUNCTIONS FOR DSGE-INFORMED THETA ANALYSIS
# ============================================================================

def enhance_dsge_dfm_with_theta(existing_dsge_dfm_model, strategy='auto', forecast_horizon=8):
    """
    FIXED: Enhance existing DSGE-informed DFM with theta optimization while preserving structure
    
    Parameters:
    -----------
    existing_dsge_dfm_model : EconometricDFM (from document 1)
        Fitted DSGE-informed DFM model with Bayesian VAR and DSGE posterior estimates
    strategy : str
        'auto' - compare all DSGE-informed methods and pick best
        Or specific: 'dsge_hp_trend_cycle', 'dsge_regime_switching', 'dsge_time_varying'
    forecast_horizon : int
        Number of quarters to forecast
    
    Returns:
    --------
    enhanced_dfm : DSGEInformedEnhancedDFMTheta
        DSGE-consistent theta-enhanced model with proper DSGE integration
    results : dict
        Dictionary with all results preserving DSGE structure
    """
    
    print("ENHANCING DSGE-INFORMED DFM WITH THETA OPTIMIZATION")
    print("=" * 60)
    
    # Verify input model has DSGE components
    if not hasattr(existing_dsge_dfm_model, 'dsge_priors'):
        print("Warning: Input model doesn't appear to be DSGE-informed")
        print("Will attempt to extract DSGE information from VAR estimation results")
    
    # Create DSGE-informed enhanced model (will extract DSGE priors automatically)
    enhanced_dfm = DSGEInformedEnhancedDFMTheta(existing_dsge_dfm_model)
    
    if enhanced_dfm.dsge_priors is not None:
        print("✓ DSGE priors successfully extracted - will be preserved throughout theta optimization")
        print(f"  Available DSGE parameters: {list(enhanced_dfm.dsge_priors.keys())}")
    else:
        print("! No DSGE priors found - using econometric constraints only")
    
    # Run complete analysis with DSGE consistency
    results = enhanced_dfm.run_complete_dsge_theta_analysis(strategy, forecast_horizon)
    
    return enhanced_dfm, results


def compare_dsge_theta_methods(existing_dsge_dfm_model):
    """
    Run comprehensive comparison of DSGE-informed theta methods
    
    Parameters:
    -----------
    existing_dsge_dfm_model : EconometricDFM
        Fitted DSGE-informed DFM model
    
    Returns:
    --------
    enhanced_dfm : DSGEInformedEnhancedDFMTheta
        Enhanced model object
    comparison_results : dict
        Comprehensive results with DSGE consistency metrics
    """
    
    enhanced_dfm = DSGEInformedEnhancedDFMTheta(existing_dsge_dfm_model)
    
    # Run method comparison
    all_results = enhanced_dfm.run_dsge_theta_comparison()
    
    enhanced_dfm.print_dsge_theta_results()
    
    return enhanced_dfm, all_results


# ============================================================================
# INTEGRATION WITH DOCUMENT 1 - MAIN EXECUTION FUNCTION
# ============================================================================

def integrate_dsge_theta_with_document1_results(dsge_posterior_estimates=None, start_date="2000-01-01", forecast_horizon=8):
    """
    FIXED: Complete integration function that uses Document 1's DSGE estimation results
    
    Parameters:
    -----------
    dsge_posterior_estimates : dict
        DSGE posterior parameter estimates from Document 1's dsge.posterior_table
        Should contain keys like 'sigma', 'theta', 'phi_pi', 'phi_y', etc. with 'mean' values
    start_date : str
        Start date for data fetching
    forecast_horizon : int
        Number of quarters to forecast
        
    Returns:
    --------
    dict with keys:
        'base_dsge_dfm': Base DSGE-informed DFM model
        'enhanced_theta_dfm': Theta-enhanced version
        'theta_results': Theta optimization results
        'comparison': Method comparison results
    """
    
    print("COMPLETE DSGE-THETA INTEGRATION PIPELINE")
    print("=" * 60)
    
    # Import necessary functions from Document 1
    try:
        from run_core_economic_analysis import run_core_economic_analysis
    except ImportError:
        print("Warning: Cannot import Document 1 functions. Please ensure Document 1 is available.")
        return None
    
    # STEP 1: Run base DSGE-informed DFM from Document 1
    print("\n1. Running base DSGE-informed DFM estimation...")
    base_analysis = run_core_economic_analysis(
        dsge_posterior_estimates=dsge_posterior_estimates,
        start_date=start_date,
        forecast_horizon=forecast_horizon
    )
    
    if base_analysis is None:
        print("Failed to run base DSGE-informed DFM")
        return None
    
    base_dsge_dfm = base_analysis['model']
    base_results = base_analysis['results']
    
    print("✓ Base DSGE-informed DFM completed successfully")
    
    # STEP 2: Enhance with theta optimization
    print("\n2. Enhancing with DSGE-informed theta optimization...")
    enhanced_dfm, theta_results = enhance_dsge_dfm_with_theta(
        base_dsge_dfm, 
        strategy='auto', 
        forecast_horizon=forecast_horizon
    )
    
    print("✓ Theta enhancement completed successfully")
    
    # STEP 3: Compare methods
    print("\n3. Running comprehensive method comparison...")
    comparison_dfm, comparison_results = compare_dsge_theta_methods(base_dsge_dfm)
    
    # STEP 4: Final summary
    print("\n" + "="*80)
    print("DSGE-THETA INTEGRATION COMPLETE")
    print("="*80)
    
    print(f"Pipeline Summary:")
    print(f"  • DSGE structural estimation: {'✓' if dsge_posterior_estimates else '○'}")
    print(f"  • Base DFM with DSGE priors: ✓")
    print(f"  • Theta optimization: ✓")
    print(f"  • Method comparison: ✓")
    
    if enhanced_dfm.dsge_priors is not None:
        print(f"  • DSGE parameters used: {len(enhanced_dfm.dsge_priors)}")
        print(f"  • Structure preservation: ✓")
    
    return {
        'base_dsge_dfm': base_dsge_dfm,
        'base_results': base_results,
        'enhanced_theta_dfm': enhanced_dfm,
        'theta_results': theta_results,
        'comparison_dfm': comparison_dfm,
        'comparison_results': comparison_results,
        'dsge_parameters': dsge_posterior_estimates,
        'integration_successful': True
    }


# ============================================================================
# EXAMPLE USAGE WITH DSGE INTEGRATION
# ============================================================================

def example_dsge_theta_usage():
    """
    FIXED: Complete example showing DSGE-informed theta enhancement
    """
    print("DSGE-INFORMED THETA ENHANCEMENT EXAMPLE")
    print("=" * 50)
    
    print("\nFIXED INTEGRATION APPROACH:")
    print("="*50)
    
    print("\n1. Complete Pipeline (uses Document 1's DSGE results):")
    print("   # This automatically runs DSGE estimation then theta enhancement")
    print("   results = integrate_dsge_theta_with_document1_results(")
    print("       dsge_posterior_estimates=dsge_estimates)")
    print("   ")
    
    print("2. Manual Two-Step Process:")
    print("   # Step 1: Run Document 1's DSGE-informed DFM")
    print("   base_analysis = run_core_economic_analysis(dsge_posterior_estimates)")
    print("   base_dfm = base_analysis['model']")
    print("   ")
    print("   # Step 2: Enhance with theta (automatically extracts DSGE priors)")
    print("   enhanced_dfm, results = enhance_dsge_dfm_with_theta(base_dfm)")
    print("   ")
    
    print("3. Method Comparison with DSGE Consistency:")
    print("   enhanced_dfm, results = compare_dsge_theta_methods(base_dfm)")
    print("   ")
    
    print("KEY FIXES IN THIS VERSION:")
    print("• _extract_dsge_posterior_estimates() properly extracts DSGE priors")
    print("• _construct_priors_from_dsge_estimates() handles missing prior_info")
    print("• Integration function imports and uses Document 1 functions")
    print("• DSGE parameter extraction from posterior_table format")
    print("• Automatic fallback to econometric constraints if no DSGE priors")
    print("   ")
    
    print("DSGE-Informed Methods Available:")
    print("• DSGE_HP_Trend_Cycle: DSGE persistence guides trend-cycle weights")
    print("• DSGE_Regime_Switching: Structural break regimes from DSGE parameters")  
    print("• DSGE_Time_Varying: Parameter evolution consistent with DSGE stability")


if __name__ == "__main__":
    example_dsge_theta_usage()

DSGE-INFORMED THETA ENHANCEMENT EXAMPLE

FIXED INTEGRATION APPROACH:

1. Complete Pipeline (uses Document 1's DSGE results):
   # This automatically runs DSGE estimation then theta enhancement
   results = integrate_dsge_theta_with_document1_results(
       dsge_posterior_estimates=dsge_estimates)
   
2. Manual Two-Step Process:
   # Step 1: Run Document 1's DSGE-informed DFM
   base_analysis = run_core_economic_analysis(dsge_posterior_estimates)
   base_dfm = base_analysis['model']
   
   # Step 2: Enhance with theta (automatically extracts DSGE priors)
   enhanced_dfm, results = enhance_dsge_dfm_with_theta(base_dfm)
   
3. Method Comparison with DSGE Consistency:
   enhanced_dfm, results = compare_dsge_theta_methods(base_dfm)
   
KEY FIXES IN THIS VERSION:
• _extract_dsge_posterior_estimates() properly extracts DSGE priors
• _construct_priors_from_dsge_estimates() handles missing prior_info
• Integration function imports and uses Document 1 functions
• DSGE parameter extraction from 

In [50]:
base_analysis = run_core_economic_analysis(dsge_posterior_estimates)
base_dfm = base_analysis['model']
enhanced_dfm, initial_results = enhance_dsge_dfm_with_theta(base_dfm)

DSGE-INFORMED VAR-DFM ANALYSIS
Using DSGE-estimated priors for VAR coefficient restrictions
PREPARING ECONOMETRIC DATA FOR DSGE-INFORMED DFM
Fetching economic data from FRED...

Fetching Demand_Block:
  GDPC1 ✓ (102 obs)
  PAYEMS ✓ (308 obs)
  INDPRO ✓ (307 obs)
  ICSA ✓ (1341 obs)

Fetching Supply_Block:
  CPIAUCSL ✓ (308 obs)
  PCEPI ✓ (307 obs)
  AHETPI ✓ (308 obs)
  DCOILWTICO ✓ (6441 obs)

Fetching Monetary_Block:
  FEDFUNDS ✓ (308 obs)
  DGS10 ✓ (6428 obs)
  M2SL ✓ (307 obs)
  AAA ✓ (308 obs)

Successfully fetched 12 series
Common date range: 2000-01 to 2025-04
Quarterly observations: 101
Final dataset shape: (97, 12)
Date range: 2001-Qq to 2025-Qq
Demand_Block: ['GDPC1', 'PAYEMS', 'INDPRO', 'ICSA']
Supply_Block: ['CPIAUCSL', 'PCEPI', 'AHETPI', 'DCOILWTICO']
Monetary_Block: ['FEDFUNDS', 'DGS10', 'M2SL', 'AAA']

INITIALIZING DSGE-INFORMED DFM
DSGE priors available: ['sigma', 'theta', 'phi_pi', 'phi_y', 'rho_a', 'sigma_a', 'rho_v', 'sigma_v', 'rho_d', 'sigma_d', 'rho_s', 'sigma_s']

In [51]:
# Now run the complete analysis which includes comparison + selection + printing
final_results = enhanced_dfm.run_complete_dsge_theta_analysis()

DSGE-INFORMED ENHANCED DFM-THETA ANALYSIS
✓ Using DSGE structural priors for VAR coefficient adaptation
✓ Preserving original factor loadings throughout
✓ Maintaining economic interpretation of factors

DSGE-INFORMED THETA METHOD COMPARISON
Using Bayesian VAR with DSGE structure preservation

--- DSGE_HP_Trend_Cycle Method ---
Running DSGE-Constrained HP Trend-Cycle Method...
Evaluating DSGE_HP_Trend_Cycle with DSGE consistency...
Average R²: 0.632
CV RMSE: 0.9066
DSGE-Informed: Yes
Original Loadings Preserved: True

--- DSGE_Regime_Switching Method ---
Running DSGE-Informed Regime Switching Method...
Evaluating DSGE_Regime_Switching with DSGE consistency...
Average R²: 0.665
CV RMSE: 0.5702
DSGE-Informed: Yes
Original Loadings Preserved: True

--- DSGE_Time_Varying Method ---
Running DSGE-Informed Time-Varying Method...
Evaluating DSGE_Time_Varying with DSGE consistency...
Average R²: 0.699
CV RMSE: 0.6356
DSGE-Informed: Yes
Original Loadings Preserved: True
Debug: run_dsge_theta_comp

In [19]:
# DSGE-Informed Enhanced DFM with Five Core Theta Strategies
# UPDATED: Now implements the five specific theta strategies: FREQUENCY_DUAL, LAGGED, SEQUENTIAL, CONSTRAINED, GLOBAL

class DSGEInformedEnhancedDFMTheta:
    """
    Enhanced DFM with theta optimization using five core strategies
    Implements: FREQUENCY_DUAL, LAGGED, SEQUENTIAL, CONSTRAINED, GLOBAL
    """

    def __init__(self, base_dfm_model):
        # Import structural components from DSGE-informed base model
        self.factors = base_dfm_model.factors.copy()
        self.loadings = base_dfm_model.loadings.copy()
        self.data_standardized = base_dfm_model.data_standardized.copy()
        self.scaler = base_dfm_model.scaler
        self.combined_data = base_dfm_model.combined_data.copy()
        
        # CRITICAL: Preserve DSGE-informed VAR structure
        self.base_factor_var = base_dfm_model.factor_var
        
        # Extract DSGE priors from base model's estimation results
        self.dsge_priors = self._extract_dsge_posterior_estimates(base_dfm_model)
        self.idiosyncratic_var = base_dfm_model.idiosyncratic_var.copy()

        # Model dimensions
        self.T_obs, self.n_factors = self.factors.shape
        self.N_vars = len(self.data_standardized.columns)

        # Initialize theta components
        self.optimal_theta = None
        self.theta_strategy = None
        self.method_results = {}
        
        # Results storage for each method
        self.method_forecasts = {}
        self.method_var_models = {}
        self.method_r2 = {}
        self.method_metrics = {}
        
        # Cross-validation setup
        self.tscv = TimeSeriesSplit(n_splits=3, test_size=4)

        print(f"DSGE-Informed Enhanced DFM-Theta initialized:")
        print(f"  Factors: {self.n_factors}, Variables: {self.N_vars}")
        print(f"  DSGE priors: {'Available' if self.dsge_priors else 'None'}")
        print(f"  Base VAR estimation: {'Bayesian' if self.base_factor_var.get('bayesian', False) else 'OLS'}")

    def _extract_dsge_posterior_estimates(self, base_dfm_model):
        """Extract DSGE posterior estimates from the base model's DSGE estimation"""
        # Try to get DSGE priors from the base model
        if hasattr(base_dfm_model, 'dsge_priors') and base_dfm_model.dsge_priors is not None:
            return base_dfm_model.dsge_priors
            
        # If base model was created with DSGE posterior estimates, extract them
        if hasattr(base_dfm_model, 'factor_var') and base_dfm_model.factor_var.get('bayesian', False):
            prior_info = base_dfm_model.factor_var.get('prior_info')
            if prior_info and 'dsge_params' in prior_info:
                dsge_params = prior_info['dsge_params']
                return {
                    'sigma': {'mean': dsge_params.get('sigma', 1.25), 'std': 0.15},
                    'theta': {'mean': dsge_params.get('theta', 0.75), 'std': 0.08},
                    'phi_pi': {'mean': dsge_params.get('phi_pi', 1.65), 'std': 0.25},
                    'phi_y': {'mean': dsge_params.get('phi_y', 0.35), 'std': 0.12},
                    'rho_a': {'mean': dsge_params.get('rho_a', 0.82), 'std': 0.10},
                    'rho_v': {'mean': dsge_params.get('rho_v', 0.45), 'std': 0.18},
                }
        
        # Try to reconstruct from any DSGE calibration in the base model
        if hasattr(base_dfm_model, 'organized_blocks'):
            return {
                'sigma': {'mean': 1.30, 'std': 0.20},
                'theta': {'mean': 0.75, 'std': 0.10},
                'phi_pi': {'mean': 1.50, 'std': 0.35},
                'phi_y': {'mean': 0.25, 'std': 0.10},
                'rho_a': {'mean': 0.80, 'std': 0.15},
                'rho_v': {'mean': 0.50, 'std': 0.20},
            }
            
        print("  No DSGE priors found - theta optimization will use econometric constraints only")
        return None

    # ============================================================================
    # FIVE CORE THETA STRATEGIES
    # ============================================================================

    def frequency_dual_theta_strategy(self):
        """
        FREQUENCY_DUAL Strategy: Dual frequency decomposition with theta optimization
        Separates factors into trend, low-frequency, and high-frequency components
        FIXED VERSION - Prevents NaN errors during cross-validation
        """
        print("FREQUENCY_DUAL Strategy: Optimizing dual frequency decomposition with robustness...")

        def extract_frequency_components(series, window_short=4, window_long=12):
            """Extract trend, low-freq, and high-freq components with NaN prevention"""
            series_length = len(series)
            
            # Minimum length check and adjust windows dynamically
            if series_length < 8:  # Minimum viable series length
                # Return simple decomposition for very short series
                trend = pd.Series([series.mean()] * series_length, index=series.index)
                cyclical = pd.Series([0.0] * series_length, index=series.index) 
                high_freq = series - trend
                return trend, cyclical, high_freq
            
            # More conservative window sizing
            window_short = max(3, min(window_short, series_length // 6))  # Changed from //4
            window_long = max(window_short * 2, min(window_long, series_length // 3))  # Changed from //2
            
            # Robust trend extraction with fallback
            try:
                trend = series.rolling(window=window_long, center=True, min_periods=1).mean()
            except:
                # Fallback to simple linear trend
                x = np.arange(len(series))
                coeffs = np.polyfit(x, series.values, 1)
                trend = pd.Series(coeffs[0] * x + coeffs[1], index=series.index)
            
            # Handle edge cases in trend
            if trend.isna().any():
                # Forward/backward fill with boundary handling
                trend = trend.fillna(method='bfill').fillna(method='ffill')
                if trend.isna().any():  # Still NaN after filling
                    trend = trend.fillna(series.mean())
            
            # Robust cyclical component
            try:
                cyclical = series.rolling(window=window_short, center=True, min_periods=1).mean()
                if cyclical.isna().any():
                    cyclical = cyclical.fillna(method='bfill').fillna(method='ffill')
                    if cyclical.isna().any():
                        cyclical = cyclical.fillna(series.mean())
            except:
                cyclical = pd.Series([0.0] * len(series), index=series.index)
            
            cyclical = cyclical - trend
            
            # High-frequency with validation
            high_freq = series - trend - cyclical
            
            # Final NaN check and cleanup
            components = [(trend, 'trend'), (cyclical, 'cyclical'), (high_freq, 'high_freq')]
            cleaned_components = []
            
            for component, name in components:
                if component.isna().any() or np.isinf(component.values).any():
                    print(f"Warning: Invalid values in {name} component, using fallback")
                    if name == 'trend':
                        component = pd.Series([series.mean()] * len(series), index=series.index)
                    else:
                        component = pd.Series([0.0] * len(series), index=series.index)
                cleaned_components.append(component)
            
            return cleaned_components[0], cleaned_components[1], cleaned_components[2]

        def frequency_dual_objective(theta_vector):
            """Objective function with comprehensive NaN prevention"""
            # Three parameters per factor: trend weight, cyclical weight, high-freq weight
            n_params_per_factor = 3
            if len(theta_vector) != n_params_per_factor * self.n_factors:
                return 1e10
                
            F_theta = np.zeros((self.T_obs, self.n_factors))
            
            for i, factor_name in enumerate(self.factors.columns):
                factor_series = self.factors[factor_name]
                
                # Add input validation
                if factor_series.isna().any() or np.isinf(factor_series.values).any():
                    print(f"Warning: Invalid input data for factor {factor_name}")
                    return 1e10
                    
                trend, cyclical, high_freq = extract_frequency_components(factor_series)
                
                # Get theta parameters for this factor
                idx_start = i * n_params_per_factor
                theta_trend = theta_vector[idx_start]
                theta_cyclical = theta_vector[idx_start + 1] 
                theta_high = theta_vector[idx_start + 2]
                
                # Validate theta parameters
                if any(np.isnan([theta_trend, theta_cyclical, theta_high])) or \
                any(np.isinf([theta_trend, theta_cyclical, theta_high])):
                    return 1e10
                
                # Combine components with validation
                combined = (theta_trend * trend + 
                        theta_cyclical * cyclical + 
                        theta_high * high_freq)
                
                # Validate combined result
                if combined.isna().any() or np.isinf(combined.values).any():
                    print(f"Warning: Invalid combined values for factor {factor_name}")
                    return 1e10
                    
                F_theta[:, i] = combined.values

            # Final validation before VAR estimation
            if np.isnan(F_theta).any() or np.isinf(F_theta).any():
                print("Warning: NaN/Inf values in F_theta matrix")
                return 1e10

            try:
                # Estimate theta-aware VAR
                theta_var = self._estimate_theta_aware_var(
                    pd.DataFrame(F_theta, columns=self.factors.columns), 
                    theta_vector, 'FREQUENCY_DUAL')
                
                # Validate VAR results
                if theta_var is None or 'coefficients' not in theta_var:
                    return 1e10
                    
                # Calculate reconstruction error using original loadings
                Y_reconstructed = F_theta @ self.loadings.values.T
                
                # Validate reconstruction
                if np.isnan(Y_reconstructed).any() or np.isinf(Y_reconstructed).any():
                    return 1e10
                    
                reconstruction_error = np.sum((self.data_standardized.values - Y_reconstructed)**2)
                
                # Validate error calculation
                if np.isnan(reconstruction_error) or np.isinf(reconstruction_error):
                    return 1e10
                
                # Add VAR stability penalty
                stability_penalty = self._calculate_var_stability_penalty(theta_var)
                if np.isnan(stability_penalty) or np.isinf(stability_penalty):
                    stability_penalty = 1000.0
                
                total_objective = reconstruction_error + stability_penalty
                
                # Final objective validation
                if np.isnan(total_objective) or np.isinf(total_objective):
                    return 1e10
                    
                return total_objective
                
            except Exception as e:
                print(f"Exception in frequency_dual_objective: {e}")
                return 1e10

        # More conservative bounds to prevent extreme values
        bounds = []
        for _ in range(self.n_factors):
            bounds.extend([
                (0.8, 1.2),   # Trend weight - narrower range
                (-0.3, 0.3),  # Cyclical weight - narrower range  
                (0.7, 1.3)    # High-freq weight - narrower range
            ])
        
        # More conservative initial guess
        initial_guess = []
        for _ in range(self.n_factors):
            initial_guess.extend([1.0, 0.05, 1.0])  # Smaller cyclical component

        # Add multiple attempts with fallbacks
        for attempt in range(3):
            try:
                result = minimize(frequency_dual_objective, initial_guess, bounds=bounds,
                                method='L-BFGS-B', options={'maxiter': 200, 'ftol': 1e-6})
                
                if result.success and not np.isnan(result.fun):
                    return result.x, lambda factors, theta_params: self._apply_frequency_dual_transform(factors, theta_params)
                    
            except Exception as e:
                print(f"Optimization attempt {attempt + 1} failed: {e}")
                # Adjust initial guess for next attempt
                initial_guess = [x * (0.9 + 0.2 * np.random.random()) for x in initial_guess]
        
        # Final fallback
        print("All optimization attempts failed, using conservative defaults")
        default_params = [1.0, 0.0, 1.0] * self.n_factors
        return default_params, lambda factors, theta_params: self._apply_frequency_dual_transform(factors, theta_params)

    def _apply_frequency_dual_transform(self, factors, theta_params):
        """Apply frequency dual transformation with NaN prevention"""
        F_theta_df = factors.copy()
        n_params_per_factor = 3
        
        def extract_frequency_components(series, window_short=4, window_long=12):
            """Same robust extraction function as in objective"""
            series_length = len(series)
            
            if series_length < 8:
                trend = pd.Series([series.mean()] * series_length, index=series.index)
                cyclical = pd.Series([0.0] * series_length, index=series.index) 
                high_freq = series - trend
                return trend, cyclical, high_freq
            
            window_short = max(3, min(window_short, series_length // 6))
            window_long = max(window_short * 2, min(window_long, series_length // 3))
            
            try:
                trend = series.rolling(window=window_long, center=True, min_periods=1).mean()
            except:
                x = np.arange(len(series))
                coeffs = np.polyfit(x, series.values, 1)
                trend = pd.Series(coeffs[0] * x + coeffs[1], index=series.index)
            
            if trend.isna().any():
                trend = trend.fillna(method='bfill').fillna(method='ffill')
                if trend.isna().any():
                    trend = trend.fillna(series.mean())
            
            try:
                cyclical = series.rolling(window=window_short, center=True, min_periods=1).mean()
                if cyclical.isna().any():
                    cyclical = cyclical.fillna(method='bfill').fillna(method='ffill')
                    if cyclical.isna().any():
                        cyclical = cyclical.fillna(series.mean())
            except:
                cyclical = pd.Series([0.0] * len(series), index=series.index)
            
            cyclical = cyclical - trend
            high_freq = series - trend - cyclical
            
            components = [(trend, 'trend'), (cyclical, 'cyclical'), (high_freq, 'high_freq')]
            cleaned_components = []
            
            for component, name in components:
                if component.isna().any() or np.isinf(component.values).any():
                    if name == 'trend':
                        component = pd.Series([series.mean()] * len(series), index=series.index)
                    else:
                        component = pd.Series([0.0] * len(series), index=series.index)
                cleaned_components.append(component)
            
            return cleaned_components[0], cleaned_components[1], cleaned_components[2]
        
        for i, factor_name in enumerate(factors.columns):
            # Input validation
            if factors[factor_name].isna().any():
                print(f"Warning: NaN input for factor {factor_name}, using interpolation")
                factors[factor_name] = factors[factor_name].interpolate().fillna(factors[factor_name].mean())
            
            trend, cyclical, high_freq = extract_frequency_components(factors[factor_name])
            
            idx_start = i * n_params_per_factor
            theta_trend = theta_params[idx_start]
            theta_cyclical = theta_params[idx_start + 1]
            theta_high = theta_params[idx_start + 2]
            
            # Parameter validation
            if any(np.isnan([theta_trend, theta_cyclical, theta_high])):
                print(f"Warning: NaN theta parameters for factor {factor_name}, using defaults")
                theta_trend, theta_cyclical, theta_high = 1.0, 0.05, 1.0
            
            combined = theta_trend * trend + theta_cyclical * cyclical + theta_high * high_freq
            
            # Output validation  
            if combined.isna().any():
                print(f"Warning: NaN in transformed factor {factor_name}, using fallback")
                combined = factors[factor_name]  # Fallback to original
            
            F_theta_df[factor_name] = combined
        
        return F_theta_df

    def lagged_theta_strategy(self):
        """
        LAGGED Strategy: Incorporates lagged factor values with theta weights
        F_theta[t] = theta1 * F[t] + theta2 * F[t-1] + theta3 * F[t-2]
        """
        print("LAGGED Strategy: Optimizing with multi-lag factor transformation...")

        def lagged_objective(theta_vector):
            """Objective with multi-lag structure"""
            # Three parameters per factor: current, lag-1, lag-2 weights
            n_params_per_factor = 3
            if len(theta_vector) != n_params_per_factor * self.n_factors:
                return 1e10
                
            F_theta = np.zeros((self.T_obs, self.n_factors))
            
            for i, factor_name in enumerate(self.factors.columns):
                factor_values = self.factors[factor_name].values
                
                idx_start = i * n_params_per_factor
                theta_current = theta_vector[idx_start]
                theta_lag1 = theta_vector[idx_start + 1]
                theta_lag2 = theta_vector[idx_start + 2]
                
                # Apply lagged transformation
                F_theta[0, i] = theta_current * factor_values[0]  # First observation
                F_theta[1, i] = theta_current * factor_values[1] + theta_lag1 * factor_values[0]  # Second observation
                
                for t in range(2, self.T_obs):
                    F_theta[t, i] = (theta_current * factor_values[t] + 
                                   theta_lag1 * factor_values[t-1] + 
                                   theta_lag2 * factor_values[t-2])

            try:
                theta_var = self._estimate_theta_aware_var(
                    pd.DataFrame(F_theta, columns=self.factors.columns),
                    theta_vector, 'LAGGED')
                
                Y_reconstructed = F_theta @ self.loadings.values.T
                reconstruction_error = np.sum((self.data_standardized.values - Y_reconstructed)**2)
                
                stability_penalty = self._calculate_var_stability_penalty(theta_var)
                return reconstruction_error + stability_penalty
            except:
                return 1e10

        # Bounds: current weight [0.5, 1.5], lag weights [-0.8, 0.8]
        bounds = []
        for _ in range(self.n_factors):
            bounds.extend([(0.5, 1.5), (-0.8, 0.8), (-0.8, 0.8)])
        
        initial_guess = []
        for _ in range(self.n_factors):
            initial_guess.extend([1.0, 0.3, 0.1])

        result = minimize(lagged_objective, initial_guess, bounds=bounds,
                         method='L-BFGS-B', options={'maxiter': 250})

        if result.success:
            return result.x, lambda factors, theta_params: self._apply_lagged_transform(factors, theta_params)
        else:
            return initial_guess, lambda factors, theta_params: self._apply_lagged_transform(factors, theta_params)

    def _apply_lagged_transform(self, factors, theta_params):
        """Apply multi-lag transformation"""
        F_theta_df = factors.copy()
        n_params_per_factor = 3
        
        for i, factor_name in enumerate(factors.columns):
            factor_values = factors[factor_name].values
            transformed_values = np.zeros_like(factor_values)
            
            idx_start = i * n_params_per_factor
            theta_current = theta_params[idx_start]
            theta_lag1 = theta_params[idx_start + 1]
            theta_lag2 = theta_params[idx_start + 2]
            
            # Apply transformation
            transformed_values[0] = theta_current * factor_values[0]
            transformed_values[1] = theta_current * factor_values[1] + theta_lag1 * factor_values[0]
            
            for t in range(2, len(factor_values)):
                transformed_values[t] = (theta_current * factor_values[t] + 
                                       theta_lag1 * factor_values[t-1] + 
                                       theta_lag2 * factor_values[t-2])
            
            F_theta_df[factor_name] = transformed_values
        
        return F_theta_df

    def sequential_theta_strategy(self):
        """
        SEQUENTIAL Strategy: Sequential factor-by-factor theta optimization
        Optimizes each factor's theta parameters sequentially while holding others fixed
        """
        print("SEQUENTIAL Strategy: Factor-by-factor sequential optimization...")

        # Initialize with basic parameters
        theta_params = np.ones(2 * self.n_factors)  # Two parameters per factor
        theta_params[self.n_factors:] = 0.1  # Initialize second parameter set
        
        max_iterations = 5
        tolerance = 1e-4
        
        for iteration in range(max_iterations):
            old_params = theta_params.copy()
            
            for factor_idx in range(self.n_factors):
                # Define single-factor objective
                def single_factor_objective(factor_theta_params):
                    """Objective for optimizing single factor"""
                    full_params = theta_params.copy()
                    full_params[factor_idx] = factor_theta_params[0]
                    full_params[factor_idx + self.n_factors] = factor_theta_params[1]
                    
                    try:
                        F_theta = self._apply_sequential_transform(self.factors, full_params)
                        theta_var = self._estimate_theta_aware_var(F_theta, full_params, 'SEQUENTIAL')
                        
                        Y_reconstructed = F_theta.values @ self.loadings.values.T
                        reconstruction_error = np.sum((self.data_standardized.values - Y_reconstructed)**2)
                        
                        stability_penalty = self._calculate_var_stability_penalty(theta_var)
                        return reconstruction_error + stability_penalty
                    except:
                        return 1e10
                
                # Optimize this factor
                factor_bounds = [(0.1, 2.0), (-1.0, 1.0)]
                factor_initial = [theta_params[factor_idx], theta_params[factor_idx + self.n_factors]]
                
                factor_result = minimize(single_factor_objective, factor_initial, 
                                       bounds=factor_bounds, method='L-BFGS-B')
                
                if factor_result.success:
                    theta_params[factor_idx] = factor_result.x[0]
                    theta_params[factor_idx + self.n_factors] = factor_result.x[1]
            
            # Check convergence
            if np.abs(theta_params - old_params).max() < tolerance:
                print(f"Sequential optimization converged after {iteration + 1} iterations")
                break

        return theta_params, lambda factors, theta_params: self._apply_sequential_transform(factors, theta_params)

    def _apply_sequential_transform(self, factors, theta_params):
        """Apply sequential transformation (nonlinear with trend-deviation structure)"""
        F_theta_df = factors.copy()
        
        for i, factor_name in enumerate(factors.columns):
            theta1 = theta_params[i]
            theta2 = theta_params[i + self.n_factors]
            
            # Extract trend and deviation
            trend = self._extract_trend_linear(factors[factor_name])
            deviation = factors[factor_name] - trend
            
            # Apply nonlinear transformation
            F_theta_df[factor_name] = trend + theta1 * deviation + theta2 * (deviation ** 2)
        
        return F_theta_df

    def constrained_theta_strategy(self):
        """
        CONSTRAINED Strategy: Theta optimization with DSGE-based constraints
        Incorporates structural economic constraints from DSGE parameters
        """
        print("CONSTRAINED Strategy: DSGE-constrained theta optimization...")

        def constrained_objective(theta_vector):
            """Objective with DSGE constraints"""
            F_theta = self._apply_constrained_transform(self.factors, theta_vector)
            
            try:
                theta_var = self._estimate_theta_aware_var(F_theta, theta_vector, 'CONSTRAINED')
                
                Y_reconstructed = F_theta.values @ self.loadings.values.T
                reconstruction_error = np.sum((self.data_standardized.values - Y_reconstructed)**2)
                
                # Add DSGE consistency penalty
                dsge_penalty = self._calculate_dsge_constraint_penalty(theta_vector)
                stability_penalty = self._calculate_var_stability_penalty(theta_var)
                
                return reconstruction_error + dsge_penalty + stability_penalty
            except:
                return 1e10

        # DSGE-informed bounds
        if self.dsge_priors is not None:
            bounds = self._get_dsge_informed_bounds()
            initial_guess = self._get_dsge_informed_initial_guess()
        else:
            # Fallback bounds
            bounds = [(0.5, 1.5) for _ in range(self.n_factors)] + [(-0.5, 0.5) for _ in range(self.n_factors)]
            initial_guess = [1.0] * self.n_factors + [0.0] * self.n_factors

        result = minimize(constrained_objective, initial_guess, bounds=bounds,
                         method='L-BFGS-B', options={'maxiter': 300})

        if result.success:
            return result.x, lambda factors, theta_params: self._apply_constrained_transform(factors, theta_params)
        else:
            return initial_guess, lambda factors, theta_params: self._apply_constrained_transform(factors, theta_params)

    def _apply_constrained_transform(self, factors, theta_params):
        """Apply DSGE-constrained transformation"""
        F_theta_df = factors.copy()
        
        for i, factor_name in enumerate(factors.columns):
            theta1 = theta_params[i]
            theta2 = theta_params[i + self.n_factors] if i + self.n_factors < len(theta_params) else 0.0
            
            # Apply constrained transformation
            if i == 0:  # Demand factor - output gap transformation
                hp_trend, hp_cycle = self._hp_filter(factors[factor_name])
                F_theta_df[factor_name] = theta1 * hp_trend + theta2 * hp_cycle
            elif i == 1:  # Supply factor - inflation persistence
                F_theta_df[factor_name] = theta1 * factors[factor_name] + theta2 * factors[factor_name].shift(1).fillna(0)
            else:  # Monetary/other factors - standard transformation
                trend = self._extract_trend_linear(factors[factor_name])
                deviation = factors[factor_name] - trend
                F_theta_df[factor_name] = trend + theta1 * deviation
        
        return F_theta_df

    def global_theta_strategy(self):
        """
        GLOBAL Strategy: Global optimization across all factors simultaneously
        Uses multiple algorithms and selects best result
        """
        print("GLOBAL Strategy: Multi-algorithm global optimization...")

        def global_objective(theta_vector):
            """Global objective with ensemble error metrics"""
            F_theta = self._apply_global_transform(self.factors, theta_vector)
            
            try:
                theta_var = self._estimate_theta_aware_var(F_theta, theta_vector, 'GLOBAL')
                
                Y_reconstructed = F_theta.values @ self.loadings.values.T
                errors = self.data_standardized.values - Y_reconstructed
                
                # Ensemble error metrics
                mse_error = np.mean(errors**2)
                mae_error = np.mean(np.abs(errors))
                huber_error = np.mean(np.where(np.abs(errors) < 1, 0.5 * errors**2, np.abs(errors) - 0.5))
                
                # Cross-validation penalty
                cv_penalty = self._calculate_cv_penalty(F_theta)
                
                # Weighted combination
                ensemble_error = 0.5 * mse_error + 0.3 * mae_error + 0.1 * huber_error + 0.1 * cv_penalty
                return ensemble_error
            except:
                return 1e10

        bounds = [(0.1, 2.5) for _ in range(self.n_factors)] + [(-1.2, 1.2) for _ in range(self.n_factors)]

        # Try multiple global optimization algorithms
        best_result = None
        best_objective = float('inf')
        
        try:
            from scipy.optimize import differential_evolution, basinhopping, dual_annealing
        except ImportError:
            from scipy.optimize import differential_evolution
            basinhopping = None
            dual_annealing = None

        algorithms = [
            ('differential_evolution', lambda: differential_evolution(
                global_objective, bounds, seed=42, maxiter=100, popsize=15))
        ]
        
        if basinhopping is not None:
            algorithms.append(('basinhopping', lambda: basinhopping(
                global_objective, [1.0] * len(bounds), minimizer_kwargs={'bounds': bounds})))
        
        if dual_annealing is not None:
            algorithms.append(('dual_annealing', lambda: dual_annealing(
                global_objective, bounds, seed=42, maxiter=200)))
        
        for alg_name, alg_func in algorithms:
            try:
                print(f"  Trying {alg_name}...")
                result = alg_func()
                if result.success and result.fun < best_objective:
                    best_result = result
                    best_objective = result.fun
                    print(f"  {alg_name} succeeded with objective: {result.fun:.4f}")
            except Exception as e:
                print(f"  {alg_name} failed: {e}")
                continue

        if best_result:
            return best_result.x, lambda factors, theta_params: self._apply_global_transform(factors, theta_params)
        else:
            # Fallback
            default_params = [1.0] * self.n_factors + [0.1] * self.n_factors
            return default_params, lambda factors, theta_params: self._apply_global_transform(factors, theta_params)

    def _apply_global_transform(self, factors, theta_params):
        """Apply global transformation (adaptive nonlinear)"""
        F_theta_df = factors.copy()
        
        for i, factor_name in enumerate(factors.columns):
            theta1 = theta_params[i]
            theta2 = theta_params[i + self.n_factors] if i + self.n_factors < len(theta_params) else 0.1
            
            # Adaptive transformation based on factor characteristics
            factor_std = factors[factor_name].std()
            
            if factor_std > 1.0:  # High volatility factor
                trend = self._extract_trend_linear(factors[factor_name])
                deviation = factors[factor_name] - trend
                F_theta_df[factor_name] = trend + theta1 * deviation + theta2 * np.sign(deviation) * (deviation ** 2)
            else:  # Low volatility factor
                F_theta_df[factor_name] = theta1 * factors[factor_name] + theta2 * factors[factor_name].shift(1).fillna(factors[factor_name].iloc[0])
        
        return F_theta_df

    # ============================================================================
    # HELPER METHODS FOR THETA STRATEGIES
    # ============================================================================

    def _calculate_var_stability_penalty(self, theta_var):
        """Calculate VAR stability penalty"""
        try:
            coeffs = theta_var['coefficients']
            eigenvals = np.linalg.eigvals(coeffs)
            max_eigenval = np.max(np.abs(eigenvals))
            
            if max_eigenval >= 1.0:
                return 1000 * (max_eigenval - 0.99)**2  # Heavy penalty for instability
            else:
                return 0.0
        except:
            return 1000.0

    def _calculate_dsge_constraint_penalty(self, theta_params):
        """Calculate penalty for violating DSGE constraints"""
        if self.dsge_priors is None:
            return 0.0
        
        penalty = 0.0
        
        # Check if theta parameters are consistent with DSGE persistence
        for i in range(min(len(theta_params), self.n_factors)):
            theta_val = theta_params[i]
            
            # Demand factor should have high persistence (close to 1)
            if i == 0 and self.dsge_priors.get('rho_a'):
                target_persistence = self.dsge_priors['rho_a']['mean']
                penalty += 10 * (theta_val - target_persistence)**2
            
            # Supply factor should reflect price stickiness
            elif i == 1 and self.dsge_priors.get('theta'):
                target_stickiness = self.dsge_priors['theta']['mean']
                penalty += 10 * (theta_val - target_stickiness)**2
        
        return penalty

    def _calculate_cv_penalty(self, F_theta):
        """Calculate cross-validation penalty"""
        try:
            n_splits = min(3, self.T_obs // 10)
            cv_errors = []
            
            for i in range(n_splits):
                split_point = int((i + 1) * self.T_obs / (n_splits + 1))
                train_data = F_theta.iloc[:split_point]
                test_data = F_theta.iloc[split_point:split_point + 4]
                
                if len(test_data) < 2:
                    continue
                
                # Simple VAR forecast
                try:
                    from sklearn.linear_model import LinearRegression
                    if len(train_data) > 2:
                        X_train = train_data.values[:-1]
                        y_train = train_data.values[1:]
                        
                        reg = LinearRegression().fit(X_train, y_train)
                        
                        X_test = test_data.values[:-1]
                        y_test = test_data.values[1:]
                        y_pred = reg.predict(X_test)
                        
                        cv_error = np.mean((y_test - y_pred)**2)
                        cv_errors.append(cv_error)
                except:
                    continue
            
            return np.mean(cv_errors) if cv_errors else 0.0
        except:
            return 0.0

    def _get_dsge_informed_bounds(self):
        """Get DSGE-informed parameter bounds"""
        bounds = []
        
        if self.dsge_priors:
            for i in range(self.n_factors):
                if i == 0:  # Demand factor
                    rho_a = self.dsge_priors.get('rho_a', {'mean': 0.8, 'std': 0.1})
                    bounds.append((max(0.1, rho_a['mean'] - 2*rho_a['std']), 
                                 min(1.5, rho_a['mean'] + 2*rho_a['std'])))
                elif i == 1:  # Supply factor
                    theta_dsge = self.dsge_priors.get('theta', {'mean': 0.75, 'std': 0.1})
                    bounds.append((max(0.1, theta_dsge['mean'] - 2*theta_dsge['std']), 
                                 min(1.5, theta_dsge['mean'] + 2*theta_dsge['std'])))
                else:  # Other factors
                    bounds.append((0.5, 1.5))
            
            # Second parameter bounds (interaction terms)
            for i in range(self.n_factors):
                bounds.append((-0.5, 0.5))
        else:
            bounds = [(0.5, 1.5) for _ in range(self.n_factors)] + [(-0.5, 0.5) for _ in range(self.n_factors)]
        
        return bounds

    def _get_dsge_informed_initial_guess(self):
        """Get DSGE-informed initial parameter guess"""
        initial_guess = []
        
        if self.dsge_priors:
            for i in range(self.n_factors):
                if i == 0:  # Demand factor
                    rho_a = self.dsge_priors.get('rho_a', {'mean': 0.8})
                    initial_guess.append(rho_a['mean'])
                elif i == 1:  # Supply factor
                    theta_dsge = self.dsge_priors.get('theta', {'mean': 0.75})
                    initial_guess.append(theta_dsge['mean'])
                else:  # Other factors
                    initial_guess.append(1.0)
            
            # Second parameter initial values
            for i in range(self.n_factors):
                initial_guess.append(0.1)
        else:
            initial_guess = [1.0] * self.n_factors + [0.1] * self.n_factors
        
        return initial_guess

    def _hp_filter(self, series, lamb=1600):
        """Hodrick-Prescott filter"""
        n = len(series)
        I = np.eye(n)
        D2 = np.diff(np.eye(n), 2, axis=0)
        trend = np.linalg.solve(I + lamb * D2.T @ D2, series.values)
        cycle = series.values - trend
        return pd.Series(trend, index=series.index), pd.Series(cycle, index=series.index)

    def _extract_trend_linear(self, series):
        """Extract linear trend"""
        n = len(series)
        x = np.arange(n)
        X = np.column_stack([np.ones(n), x])
        coeffs = np.linalg.lstsq(X, series.values, rcond=None)[0]
        return pd.Series(coeffs[0] + coeffs[1] * x, index=series.index)

    # ============================================================================
    # VAR ESTIMATION METHODS (DSGE-INFORMED)
    # ============================================================================

    def _adapt_dsge_priors_to_theta(self, transformed_factors, theta_params, method_name):
        """Adapt DSGE priors to transformed factor space while preserving economic structure"""
        if self.dsge_priors is None:
            return None
            
        # Get original prior structure
        base_priors = self.base_factor_var.get('prior_info')
        if base_priors is None:
            return self._construct_priors_from_dsge_estimates(transformed_factors, theta_params)
            
        # Calculate transformation scaling factors for each factor
        original_factors = self.factors.values
        theta_factors = transformed_factors.values if hasattr(transformed_factors, 'values') else transformed_factors
        
        scaling_factors = []
        for i in range(self.n_factors):
            orig_std = np.std(original_factors[:, i])
            theta_std = np.std(theta_factors[:, i])
            if orig_std > 0:
                scaling_factors.append(theta_std / orig_std)
            else:
                scaling_factors.append(1.0)
        
        # Adapt prior means and variances
        adapted_prior_mean = base_priors['mean'].copy()
        adapted_prior_var = base_priors['variance'].copy()
        
        # Adjust VAR coefficient priors based on transformation scaling
        K = self.n_factors
        lags = self.base_factor_var['lags']
        
        # Skip constants, adjust VAR coefficients
        coeff_start = K
        for lag in range(1, lags + 1):
            for i in range(K):  # Each factor equation
                for j in range(K):  # Each factor lag
                    coeff_idx = coeff_start + (lag-1)*K*K + i*K + j
                    if coeff_idx < len(adapted_prior_mean):
                        # Scale coefficient priors by transformation ratios
                        scale_ratio = scaling_factors[i] / scaling_factors[j]
                        adapted_prior_mean[coeff_idx] *= scale_ratio
                        # Increase uncertainty for transformed relationships
                        adapted_prior_var[coeff_idx] *= (1.0 + 0.1 * abs(1 - scale_ratio))
        
        return {
            'mean': adapted_prior_mean,
            'variance': adapted_prior_var,
            'dsge_params': base_priors.get('dsge_params', {}),
            'scaling_factors': scaling_factors
        }

    def _construct_priors_from_dsge_estimates(self, transformed_factors, theta_params):
        """Construct VAR priors directly from DSGE posterior estimates when prior_info unavailable"""
        if self.dsge_priors is None:
            return None
            
        K = self.n_factors
        lags = self.base_factor_var['lags']
        n_coeffs = K * K * lags + K  # VAR coeffs + constants
        
        # Extract key DSGE parameters (using posterior means)
        dsge_params = {}
        for param_name in ['sigma', 'theta', 'phi_pi', 'phi_y', 'rho_a', 'rho_v']:
            if param_name in self.dsge_priors:
                dsge_params[param_name] = self.dsge_priors[param_name]['mean']
        
        # Construct theory-consistent priors
        prior_mean = np.zeros(n_coeffs)
        prior_var = np.ones(n_coeffs)
        
        # Constants (intercepts) - weakly informative
        prior_var[:K] = 1.0
        
        # VAR coefficient priors based on DSGE structure
        coeff_start = K
        
        for lag in range(1, lags + 1):
            for i in range(K):  # Each factor equation
                for j in range(K):  # Each factor lag
                    coeff_idx = coeff_start + (lag-1)*K*K + i*K + j
                    
                    if coeff_idx >= n_coeffs:
                        continue
                        
                    if i == j:  # Own lags
                        if lag == 1:
                            # First own lag - use DSGE persistence parameters
                            if i == 0:  # Demand factor - related to output persistence
                                prior_mean[coeff_idx] = dsge_params.get('rho_a', 0.5) * 0.7
                                prior_var[coeff_idx] = 0.1
                            elif i == 1:  # Supply factor - related to price stickiness
                                stickiness = dsge_params.get('theta', 0.7)
                                prior_mean[coeff_idx] = min(0.9, stickiness * 1.2)
                                prior_var[coeff_idx] = 0.1
                            else:  # Monetary factor - use monetary persistence
                                prior_mean[coeff_idx] = dsge_params.get('rho_v', 0.3)
                                prior_var[coeff_idx] = 0.15
                        else:
                            # Higher order own lags - decay
                            first_lag_idx = coeff_start + (i*K + j)
                            if first_lag_idx < len(prior_mean):
                                prior_mean[coeff_idx] = prior_mean[first_lag_idx] * 0.5
                            prior_var[coeff_idx] = 0.2
                    else:  # Cross-lags
                        if lag == 1:
                            # Theory-based cross-effects
                            if (i == 0 and j == 2):  # Monetary -> Demand (IS curve)
                                prior_mean[coeff_idx] = -1.0 / dsge_params.get('sigma', 1.0) * 0.1
                                prior_var[coeff_idx] = 0.05
                            elif (i == 1 and j == 0):  # Demand -> Supply (Phillips curve)  
                                prior_mean[coeff_idx] = dsge_params.get('phi_y', 0.5) * 0.2
                                prior_var[coeff_idx] = 0.05
                            elif (i == 2 and j == 1):  # Supply -> Monetary (Taylor rule)
                                prior_mean[coeff_idx] = dsge_params.get('phi_pi', 1.5) * 0.1
                                prior_var[coeff_idx] = 0.05
                            else:
                                prior_mean[coeff_idx] = 0.0
                                prior_var[coeff_idx] = 0.1
                        else:
                            # Higher order cross-lags - shrink to zero
                            prior_mean[coeff_idx] = 0.0
                            prior_var[coeff_idx] = 0.1 / lag
        
        return {
            'mean': prior_mean,
            'variance': prior_var,
            'dsge_params': dsge_params
        }

    def _estimate_theta_aware_var(self, transformed_factors, theta_params, method_name):
        """Estimate VAR on transformed factors using adapted DSGE priors"""
        F_theta = transformed_factors.values if hasattr(transformed_factors, 'values') else transformed_factors
        
        # Get adapted DSGE priors
        adapted_priors = self._adapt_dsge_priors_to_theta(transformed_factors, theta_params, method_name)
        
        if adapted_priors is not None and self.dsge_priors is not None:
            # Bayesian estimation with adapted priors
            return self._estimate_bayesian_var_theta(F_theta, adapted_priors)
        else:
            # OLS fallback
            return self._estimate_var_ols(F_theta, self.base_factor_var['lags'])

    def _estimate_bayesian_var_theta(self, F_theta, adapted_priors):
        """Bayesian VAR estimation for theta-transformed factors with adapted DSGE priors"""
        T, K = F_theta.shape
        lags = self.base_factor_var['lags']
        
        # Construct regression matrices
        Y = F_theta[lags:, :]
        T_eff = Y.shape[0]
        
        # Design matrix
        X = np.ones((T_eff, 1))  # Constant
        for lag in range(1, lags + 1):
            X = np.hstack([X, F_theta[lags-lag:-lag, :]])
        
        n_regressors = X.shape[1]
        
        # Prior parameters (adapted)
        beta_prior = adapted_priors['mean'][:n_regressors*K].reshape(-1, 1)
        V_prior = np.diag(adapted_priors['variance'][:n_regressors*K])
        
        # Bayesian updating
        V_prior_inv = np.linalg.inv(V_prior)
        XtX = X.T @ X
        
        # Posterior covariance and mean
        V_posterior_inv = V_prior_inv + np.kron(np.eye(K), XtX)
        V_posterior = np.linalg.inv(V_posterior_inv)
        
        XtY = X.T @ Y
        beta_posterior_vec = V_posterior @ (V_prior_inv @ beta_prior + XtY.flatten('F').reshape(-1, 1))
        beta_posterior = beta_posterior_vec.reshape((n_regressors, K), order='F')
        
        # Residuals and covariance
        Y_fitted = X @ beta_posterior
        residuals = Y - Y_fitted
        sigma_u = (residuals.T @ residuals) / T_eff
        
        # Coefficient matrices
        const = beta_posterior[0, :]
        var_coeffs = beta_posterior[1:, :].T
        
        # Model statistics
        log_likelihood = self._compute_log_likelihood(residuals, sigma_u)
        
        return {
            'coefficients': var_coeffs,
            'constant': const,
            'sigma_u': sigma_u,
            'residuals': residuals,
            'log_likelihood': log_likelihood,
            'lags': lags,
            'T_eff': T_eff,
            'bayesian': True,
            'adapted_priors': adapted_priors,
            'theta_informed': True
        }

    def _estimate_var_ols(self, F_theta, lags):
        """OLS VAR estimation fallback"""
        T, K = F_theta.shape
        
        # Construct regression matrices
        Y = F_theta[lags:, :]
        T_eff = Y.shape[0]
        
        X = np.ones((T_eff, 1))
        for lag in range(1, lags + 1):
            X = np.hstack([X, F_theta[lags-lag:-lag, :]])
        
        # OLS estimation
        XtX_inv = np.linalg.inv(X.T @ X)
        beta = XtX_inv @ X.T @ Y
        
        u = Y - X @ beta
        sigma_u = (u.T @ u) / T_eff
        
        const = beta[0, :]
        var_coeffs = beta[1:, :].T
        
        log_likelihood = self._compute_log_likelihood(u, sigma_u)
        
        return {
            'coefficients': var_coeffs,
            'constant': const,
            'sigma_u': sigma_u,
            'residuals': u,
            'log_likelihood': log_likelihood,
            'lags': lags,
            'T_eff': T_eff,
            'bayesian': False,
            'theta_informed': True
        }

    def _compute_log_likelihood(self, residuals, sigma_u):
        """Compute VAR log-likelihood"""
        T, K = residuals.shape
        log_likelihood = -0.5 * T * K * np.log(2 * np.pi)
        log_likelihood -= 0.5 * T * np.log(np.linalg.det(sigma_u))
        log_likelihood -= 0.5 * np.trace(residuals.T @ residuals @ np.linalg.inv(sigma_u))
        return log_likelihood

    # ============================================================================
    # EVALUATION METHODS
    # ============================================================================

    def detailed_variable_cv_analysis(self, method_name, theta_params, theta_func):
        """Detailed cross-validation analysis at the variable level"""
        variable_cv_results = {}
        
        for train_idx, test_idx in self.tscv.split(self.factors):
            if len(test_idx) < 2:
                continue
                
            # Split data maintaining temporal structure
            factors_train = self.factors.iloc[train_idx]
            factors_test = self.factors.iloc[test_idx]
            data_train = self.data_standardized.iloc[train_idx]
            data_test = self.data_standardized.iloc[test_idx]
            
            try:
                # Apply theta transformation to training factors
                F_theta_train = theta_func(factors_train, theta_params)
                F_theta_test = theta_func(factors_test, theta_params)
                
                # Estimate theta-aware VAR on training data
                theta_var_cv = self._estimate_theta_aware_var(F_theta_train, theta_params, method_name)
                
                # Generate forecasts
                test_forecasts = self._forecast_theta_var(F_theta_test, theta_var_cv, len(test_idx))
                F_theta_forecast = test_forecasts['mean'].values
                
                # Use original loadings for variable forecasts
                X_pred = F_theta_forecast @ self.loadings.values.T
                
                # Calculate metrics for each variable
                for i, var_name in enumerate(self.data_standardized.columns):
                    if var_name not in variable_cv_results:
                        variable_cv_results[var_name] = {
                            'RMSE': [], 'MAE': [], 'MAPE': [], 'SMAPE': [], 
                            'Theil_U1': [], 'Theil_U2': [], 'MdAPE': [], 'MASE': [], 'R2': []
                        }
                    
                    y_true_var = data_test.values[:, i]
                    y_pred_var = X_pred[:, i]
                    
                    # Skip if either contains NaN/Inf
                    if np.isnan(y_true_var).any() or np.isnan(y_pred_var).any():
                        continue
                    if np.isinf(y_true_var).any() or np.isinf(y_pred_var).any():
                        continue
                    
                    # Calculate all metrics
                    metrics = self._calculate_comprehensive_metrics(y_true_var, y_pred_var, data_train.values[:, i])
                    
                    for metric_name, value in metrics.items():
                        if not (np.isnan(value) or np.isinf(value)):
                            variable_cv_results[var_name][metric_name].append(value)
                    
            except Exception as e:
                print(f"CV fold failed for {method_name}: {e}")
                continue
        
        # Average across folds
        averaged_results = {}
        for var_name, metrics in variable_cv_results.items():
            averaged_results[var_name] = {}
            for metric_name, values in metrics.items():
                if values:
                    averaged_results[var_name][metric_name] = np.mean(values)
                else:
                    averaged_results[var_name][metric_name] = np.nan
        
        return averaged_results
    
    def _calculate_comprehensive_metrics(self, y_true, y_pred, y_train):
        """Calculate comprehensive forecast accuracy metrics"""
        epsilon = 1e-8
        
        # Basic metrics
        rmse = np.sqrt(np.mean((y_true - y_pred)**2))
        mae = np.mean(np.abs(y_true - y_pred))
        
        # MAPE
        mape = np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), epsilon))) * 100
        
        # SMAPE
        smape = np.mean(2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred) + epsilon)) * 100
        
        # Theil's U1
        numerator = np.sqrt(np.mean((y_true - y_pred)**2))
        denominator = np.sqrt(np.mean(y_true**2)) + np.sqrt(np.mean(y_pred**2))
        theil_u1 = numerator / (denominator + epsilon)
        
        # Theil's U2
        naive_forecast = np.full_like(y_true, y_true[0] if len(y_true) > 0 else 0)
        theil_u2 = rmse / (np.sqrt(np.mean((y_true - naive_forecast)**2)) + epsilon)
        
        # MdAPE (Median Absolute Percentage Error)
        mdape = np.median(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), epsilon))) * 100
        
        # MASE (Mean Absolute Scaled Error)
        if len(y_train) > 1:
            seasonal_naive_mae = np.mean(np.abs(np.diff(y_train)))
            mase = mae / (seasonal_naive_mae + epsilon)
        else:
            mase = np.nan
        
        # R-squared
        ss_res = np.sum((y_true - y_pred)**2)
        ss_tot = np.sum((y_true - np.mean(y_true))**2)
        r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
        
        return {
            'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'SMAPE': smape,
            'Theil_U1': theil_u1, 'Theil_U2': theil_u2, 'MdAPE': mdape, 
            'MASE': mase, 'R2': r2
        }

    def evaluate_theta_method(self, method_name, theta_params, theta_func, horizon=8):
        """Evaluate theta method using DSGE-consistent approach"""
        print(f"Evaluating {method_name} with DSGE consistency...")
        
        # Apply theta transformation
        F_theta = theta_func(self.factors, theta_params)
        
        # Estimate theta-aware VAR with adapted DSGE priors
        theta_var_model = self._estimate_theta_aware_var(F_theta, theta_params, method_name)
        
        # Calculate fit using ORIGINAL loadings (preserve economic structure)
        Lambda_original = self.loadings.values  # N x K (PRESERVE)
        Y_fitted = F_theta.values @ Lambda_original.T
        residuals = self.data_standardized.values - Y_fitted
        
        # Variable R-squared with original loadings
        variable_r2 = {}
        for i, var_name in enumerate(self.data_standardized.columns):
            ss_res = np.sum(residuals[:, i]**2)
            ss_tot = np.sum((self.data_standardized.values[:, i] - 
                        np.mean(self.data_standardized.values[:, i]))**2)
            r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
            variable_r2[var_name] = max(0, r2)
        
        # Generate forecasts using theta-aware VAR
        theta_factor_forecasts = self._forecast_theta_var(F_theta, theta_var_model, horizon)
        
        # Variable forecasts using ORIGINAL loadings
        F_theta_forecast = theta_factor_forecasts['mean'].values
        X_forecast_std = F_theta_forecast @ Lambda_original.T
        
        # Transform back to original scale
        X_forecast_original = self.scaler.inverse_transform(X_forecast_std)
        
        # Create forecast DataFrame
        forecast_dates = pd.date_range(
            start=self.factors.index[-1] + pd.DateOffset(months=3),
            periods=horizon, freq='Q'
        )
        
        variable_forecasts = pd.DataFrame(
            X_forecast_original,
            index=forecast_dates,
            columns=self.combined_data.columns
        )
        
        # Cross-validation metrics (DSGE-aware)
        cv_metrics = self._cross_validate_dsge_theta_method(method_name, theta_func, theta_params)
        
        # Detailed variable-level CV analysis
        variable_cv_metrics = self.detailed_variable_cv_analysis(method_name, theta_params, theta_func)
        
        return {
            'theta_var_model': theta_var_model,
            'variable_r2': variable_r2,
            'forecasts': variable_forecasts,
            'factor_forecasts': theta_factor_forecasts,
            'cv_metrics': cv_metrics,
            'variable_cv_metrics': variable_cv_metrics,
            'theta_params': theta_params,
            'theta_func': theta_func,
            'preserved_loadings': True,
            'dsge_informed': self.dsge_priors is not None
        }

    def _forecast_theta_var(self, F_theta, theta_var_model, horizon):
        """Generate forecasts using theta-aware VAR model"""
        Phi = theta_var_model['coefficients']
        c = theta_var_model['constant']
        p = theta_var_model['lags']
        K = Phi.shape[0]

        forecasts = np.zeros((horizon, K))

        if p == 1:
            F_last = F_theta.values[-1, :]
            for h in range(horizon):
                if h == 0:
                    forecasts[h, :] = c + Phi @ F_last
                else:
                    forecasts[h, :] = c + Phi @ forecasts[h-1, :]
        else:
            F_history = F_theta.values[-p:, :].copy()
            for h in range(horizon):
                F_lagged = F_history.flatten('F')
                F_forecast = c + Phi @ F_lagged
                forecasts[h, :] = F_forecast
                F_history = np.vstack([F_history[1:, :], F_forecast.reshape(1, -1)])

        forecast_dates = pd.date_range(
            start=self.factors.index[-1] + pd.DateOffset(months=3),
            periods=horizon, freq='Q'
        )
        
        return {
            'mean': pd.DataFrame(forecasts, index=forecast_dates, columns=self.factors.columns)
        }

    def _cross_validate_dsge_theta_method(self, method_name, theta_func, theta_params):
        """Cross-validate theta method preserving DSGE structure throughout"""
        cv_metrics = []
        
        for train_idx, test_idx in self.tscv.split(self.factors):
            if len(test_idx) < 2:
                continue
                
            # Split data maintaining temporal structure
            factors_train = self.factors.iloc[train_idx]
            factors_test = self.factors.iloc[test_idx]
            data_train = self.data_standardized.iloc[train_idx]
            data_test = self.data_standardized.iloc[test_idx]
            
            try:
                # Apply theta transformation to training factors
                F_theta_train = theta_func(factors_train, theta_params)
                F_theta_test = theta_func(factors_test, theta_params)
                
                # Check for valid data
                if F_theta_train.isna().all().any() or F_theta_test.isna().all().any():
                    print(f"Warning: NaN values detected in CV fold for {method_name}")
                    continue
                
                # Estimate theta-aware VAR on training data with DSGE priors
                theta_var_cv = self._estimate_theta_aware_var(F_theta_train, theta_params, method_name)
                
                # Generate forecasts using theta VAR
                test_forecasts = self._forecast_theta_var(F_theta_test, theta_var_cv, len(test_idx))
                F_theta_forecast = test_forecasts['mean'].values
                
                # Check for valid forecasts
                if np.isnan(F_theta_forecast).any():
                    print(f"Warning: NaN forecasts detected in CV fold for {method_name}")
                    continue
                
                # Use ORIGINAL loadings for variable forecasts (preserve structure)
                X_pred = F_theta_forecast @ self.loadings.values.T
                
                # Calculate metrics
                metrics = self._calculate_forecast_metrics(data_test.values, X_pred)
                cv_metrics.append(metrics)
                
            except Exception as e:
                print(f"CV fold failed for {method_name}: {e}")
                continue
        
        if cv_metrics:
            # Average across folds
            avg_metrics = {}
            for metric in cv_metrics[0].keys():
                values = [fold[metric] for fold in cv_metrics if not np.isnan(fold[metric])]
                avg_metrics[metric] = np.mean(values) if values else np.nan
            return avg_metrics
        
        return {'RMSE': np.nan, 'MAE': np.nan, 'MAPE': np.nan, 'R2': np.nan}

    def _calculate_forecast_metrics(self, y_true, y_pred):
        """Calculate forecast accuracy metrics"""
        y_true_flat = y_true.flatten()
        y_pred_flat = y_pred.flatten()
        
        mask = ~(np.isnan(y_true_flat) | np.isnan(y_pred_flat))
        y_true_clean = y_true_flat[mask]
        y_pred_clean = y_pred_flat[mask]
        
        if len(y_true_clean) == 0:
            return {'RMSE': np.nan, 'MAE': np.nan, 'MAPE': np.nan, 'R2': np.nan}
        
        # Basic metrics
        rmse = np.sqrt(np.mean((y_true_clean - y_pred_clean)**2))
        mae = np.mean(np.abs(y_true_clean - y_pred_clean))
        
        # MAPE
        epsilon = 1e-8
        mape = np.mean(np.abs((y_true_clean - y_pred_clean) / 
                            np.maximum(np.abs(y_true_clean), epsilon))) * 100
        
        # R-squared
        ss_res = np.sum((y_true_clean - y_pred_clean)**2)
        ss_tot = np.sum((y_true_clean - np.mean(y_true_clean))**2)
        r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
        
        return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'R2': r2}

    # ============================================================================
    # MAIN EXECUTION METHODS - UPDATED TO USE FIVE STRATEGIES
    # ============================================================================

    def run_five_theta_strategies_comparison(self, horizon=8):
        """Run the five core theta strategies and compare results"""
        print("\n" + "="*100)
        print("FIVE CORE THETA STRATEGIES COMPARISON")
        print("="*100)
        print(f"Using {'Bayesian' if self.dsge_priors else 'OLS'} VAR with DSGE structure preservation")

        # THE FIVE CORE STRATEGIES
        strategies = {
            'FREQUENCY_DUAL': self.frequency_dual_theta_strategy,
            'LAGGED': self.lagged_theta_strategy, 
            'SEQUENTIAL': self.sequential_theta_strategy,
            'CONSTRAINED': self.constrained_theta_strategy,
            'GLOBAL': self.global_theta_strategy,
        }

        all_results = {}

        for strategy_name, strategy_func in strategies.items():
            print(f"\n--- {strategy_name} Strategy ---")
            try:
                # Get optimal parameters and transformation function
                theta_params, theta_func = strategy_func()
                
                # Evaluate method with DSGE consistency
                results = self.evaluate_theta_method(strategy_name, theta_params, theta_func, horizon)
                all_results[strategy_name] = results
                
                # Print summary
                avg_r2 = np.mean(list(results['variable_r2'].values()))
                cv_rmse = results['cv_metrics']['RMSE']
                dsge_informed = "Yes" if results['dsge_informed'] else "No"
                print(f"Average R²: {avg_r2:.3f}")
                print(f"CV RMSE: {cv_rmse:.4f}")
                print(f"DSGE-Informed: {dsge_informed}")
                print(f"Original Loadings Preserved: {results['preserved_loadings']}")
                
            except Exception as e:
                print(f"Strategy {strategy_name} failed: {str(e)}")
                continue

        return all_results

    def select_best_strategy(self, all_results, selection_criteria=None):
        """Select best strategy based on comprehensive forecasting and fit metrics"""
        if not all_results:
            print("No results available for selection")
            return None, None
            
        if selection_criteria is None:
            # Simplified weighting scheme - only 4 core metrics
            selection_criteria = {
                'cv_rmse_weight': 0.30,      # Increased weight
                'mase_weight': 0.25,         # Substantial weight for MASE
                'theil_u1_weight': 0.20,     # Increased weight
                'theil_u2_weight': 0.20,     # Increased weight
                'dsge_bonus': 0.05           # Keep DSGE bonus
            }
        
        print(f"\n{'='*80}")
        print("COMPREHENSIVE STRATEGY SELECTION")
        print("="*80)
        
        strategy_scores = {}
        detailed_scores = {}
        
        for strategy_name, results in all_results.items():
            if 'variable_cv_metrics' not in results:
                print(f"Skipping {strategy_name} - incomplete metrics")
                continue
                
            # Calculate component scores
            cv_metrics = results['cv_metrics']
            var_cv_metrics = results['variable_cv_metrics']
            
            # Aggregate variable-level metrics
            all_rmse = [metrics.get('RMSE', np.nan) for metrics in var_cv_metrics.values()]
            all_mase = [metrics.get('MASE', np.nan) for metrics in var_cv_metrics.values()]
            all_theil_u1 = [metrics.get('Theil_U1', np.nan) for metrics in var_cv_metrics.values()]
            all_theil_u2 = [metrics.get('Theil_U2', np.nan) for metrics in var_cv_metrics.values()]
            
            avg_var_rmse = np.nanmean(all_rmse)
            avg_mase = np.nanmean(all_mase)
            avg_theil_u1 = np.nanmean(all_theil_u1)
            avg_theil_u2 = np.nanmean(all_theil_u2)
            
            # Overall fit
            avg_r2 = np.mean(list(results['variable_r2'].values()))
            
            detailed_scores[strategy_name] = {
            'avg_var_rmse': avg_var_rmse,
            'avg_mase': avg_mase,
            'avg_theil_u1': avg_theil_u1,
            'avg_theil_u2': avg_theil_u2,
            'dsge_informed': results.get('dsge_informed', False)
        }
        
        # Normalize scores across strategies
        metric_ranges = {}
        for metric in ['avg_var_rmse', 'avg_mase', 'avg_theil_u1', 'avg_theil_u2']:
            values = [scores[metric] for scores in detailed_scores.values() if not np.isnan(scores[metric])]
            if values:
                metric_ranges[metric] = {'min': min(values), 'max': max(values)}
            else:
                metric_ranges[metric] = {'min': 0, 'max': 1}
        
        # Calculate composite scores
        for strategy_name, scores in detailed_scores.items():
            composite_score = 0.0
            
            # RMSE component (lower is better)
            if not np.isnan(scores['avg_var_rmse']) and metric_ranges['avg_var_rmse']['max'] > metric_ranges['avg_var_rmse']['min']:
                rmse_normalized = 1 - (scores['avg_var_rmse'] - metric_ranges['avg_var_rmse']['min']) / (metric_ranges['avg_var_rmse']['max'] - metric_ranges['avg_var_rmse']['min'])
                composite_score += selection_criteria['cv_rmse_weight'] * rmse_normalized
            
            # MASE component (lower is better)
            if not np.isnan(scores['avg_mase']) and metric_ranges['avg_mase']['max'] > metric_ranges['avg_mase']['min']:
                mase_normalized = 1 - (scores['avg_mase'] - metric_ranges['avg_mase']['min']) / (metric_ranges['avg_mase']['max'] - metric_ranges['avg_mase']['min'])
                composite_score += selection_criteria['mase_weight'] * mase_normalized
            
            # Theil U1 component (lower is better)
            if not np.isnan(scores['avg_theil_u1']) and metric_ranges['avg_theil_u1']['max'] > metric_ranges['avg_theil_u1']['min']:
                theil_u1_normalized = 1 - (scores['avg_theil_u1'] - metric_ranges['avg_theil_u1']['min']) / (metric_ranges['avg_theil_u1']['max'] - metric_ranges['avg_theil_u1']['min'])
                composite_score += selection_criteria['theil_u1_weight'] * theil_u1_normalized
            
            # Theil U2 component (lower is better)
            if not np.isnan(scores['avg_theil_u2']) and metric_ranges['avg_theil_u2']['max'] > metric_ranges['avg_theil_u2']['min']:
                theil_u2_normalized = 1 - (scores['avg_theil_u2'] - metric_ranges['avg_theil_u2']['min']) / (metric_ranges['avg_theil_u2']['max'] - metric_ranges['avg_theil_u2']['min'])
                composite_score += selection_criteria['theil_u2_weight'] * theil_u2_normalized
            
            # DSGE bonus
            if scores['dsge_informed']:
                composite_score += selection_criteria['dsge_bonus']
            
            strategy_scores[strategy_name] = composite_score
        
        # Print selection results
        print(f"{'Strategy':<15} {'Composite Score':<15} {'Avg RMSE':<10} {'Avg MASE':<10} {'Avg Theil_U1':<12} {'Avg Theil_U2':<12} {'DSGE':<6}")
        print("-" * 85)
        
        for strategy_name in sorted(strategy_scores.keys(), key=lambda x: strategy_scores[x], reverse=True):
            score = strategy_scores[strategy_name]
            rmse = detailed_scores[strategy_name]['avg_var_rmse']
            mase = detailed_scores[strategy_name]['avg_mase']
            theil_u1 = detailed_scores[strategy_name]['avg_theil_u1']
            theil_u2 = detailed_scores[strategy_name]['avg_theil_u2']
            dsge = "Yes" if detailed_scores[strategy_name]['dsge_informed'] else "No"
            
            print(f"{strategy_name:<15} {score:<15.4f} {rmse:<10.4f} {mase:<10.4f} {theil_u1:<12.4f} {theil_u2:<12.4f} {dsge:<6}")
        
        # Select best strategy
        best_strategy = max(strategy_scores.keys(), key=lambda x: strategy_scores[x])
        best_score = strategy_scores[best_strategy]
        
        print(f"\nSelected Best Strategy: {best_strategy} (Score: {best_score:.4f})")
        
        return best_strategy, strategy_scores

    def print_five_strategies_results(self):
        """Print comprehensive results for the five theta strategies"""
        if not hasattr(self, 'method_results') or not self.method_results:
            print("No results available")
            return
        
        print("\n" + "="*100)
        print("FIVE THETA STRATEGIES COMPARISON RESULTS")
        print("="*100)
        
        # Summary table
        print(f"\n{'Strategy':<15} {'Avg R²':<8} {'CV RMSE':<10} {'CV MAE':<10} {'CV MAPE':<10} {'DSGE':<6} {'Loadings':<9}")
        print("-" * 75)
        
        for strategy_name, results in self.method_results.items():
            avg_r2 = np.mean(list(results['variable_r2'].values()))
            cv_metrics = results['cv_metrics']
            dsge_flag = "Yes" if results.get('dsge_informed', False) else "No"
            loadings_flag = "Original" if results.get('preserved_loadings', False) else "Re-est"
            
            print(f"{strategy_name:<15} {avg_r2:<8.3f} {cv_metrics['RMSE']:<10.4f} "
                f"{cv_metrics['MAE']:<10.4f} {cv_metrics['MAPE']:<10.2f} "
                f"{dsge_flag:<6} {loadings_flag:<9}")
        
        # DSGE-specific information
        if self.dsge_priors is not None:
            print(f"\n\n{'='*80}")
            print("DSGE PRIOR INTEGRATION SUMMARY")
            print("="*80)
            
            dsge_params = {k: v['mean'] for k, v in self.dsge_priors.items()}
            if dsge_params:
                print("DSGE Parameters Used:")
                for param, value in dsge_params.items():
                    print(f"  {param}: {value:.3f}")
            
            print(f"\nStructural Consistency:")
            print(f"  • Original factor loadings preserved: YES")
            print(f"  • VAR priors adapted to theta transformations: YES") 
            print(f"  • Economic interpretation maintained: YES")
            print(f"  • DSGE constraints applied to theta parameters: YES")
        
        # Best strategy detailed results
        if self.theta_strategy:
            best_results = self.method_results[self.theta_strategy]
            print(f"\n\nDETAILED RESULTS FOR BEST STRATEGY: {self.theta_strategy}")
            print("="*80)
            
            print("\nVariable Fit (using original DSGE-informed loadings):")
            print("-"*60)
            print(f"{'Variable':<15} {'R²':<8} {'Factor_1':<12} {'Factor_2':<12} {'Factor_3':<12}")
            print("-" * 60)
            
            original_loadings = self.loadings
            for var_name in original_loadings.index:
                r2 = best_results['variable_r2'].get(var_name, 0.0)
                print(f"{var_name:<15} {r2:<8.3f}", end="")
                for j, factor in enumerate(original_loadings.columns):
                    if j < 3:
                        loading_val = original_loadings.loc[var_name, factor]
                        print(f" {loading_val:<12.3f}", end="")
                print()
            
            avg_r2_best = np.mean(list(best_results['variable_r2'].values()))
            print(f"\nAverage R² (Best Strategy): {avg_r2_best:.3f}")
            
            # VAR model comparison
            theta_var = best_results['theta_var_model']
            base_var = self.base_factor_var
            
            print(f"\nVAR Model Comparison:")
            print(f"  Base VAR Log-Likelihood: {base_var['log_likelihood']:.2f}")
            print(f"  Theta VAR Log-Likelihood: {theta_var['log_likelihood']:.2f}")
            print(f"  Improvement: {theta_var['log_likelihood'] - base_var['log_likelihood']:.2f}")

        # Print variable-level CV results
        self.print_variable_cv_results()

    def print_variable_cv_results(self):
        """Print detailed variable-level cross-validation results"""
        if not self.method_results:
            return
        
        print(f"\n{'='*120}")
        print("DETAILED VARIABLE-LEVEL CROSS-VALIDATION RESULTS")
        print("="*120)
        
        for strategy_name, results in self.method_results.items():
            if 'variable_cv_metrics' not in results:
                continue
                
            print(f"\nStrategy: {strategy_name}")
            print("-" * 115)
            print(f"{'Variable':<15} {'CV RMSE':<8} {'CV MAE':<8} {'CV MAPE':<9} {'CV SMAPE':<10} "
                f"{'CV Theil_U1':<12} {'CV Theil_U2':<12} {'CV MdAPE':<10} {'CV MASE':<9} {'CV R²':<8}")
            print("-" * 115)
            
            var_cv_metrics = results['variable_cv_metrics']
            for var_name, metrics in var_cv_metrics.items():
                print(f"{var_name:<15} {metrics.get('RMSE', np.nan):<8.4f} {metrics.get('MAE', np.nan):<8.4f} "
                    f"{metrics.get('MAPE', np.nan):<9.2f} {metrics.get('SMAPE', np.nan):<10.2f} "
                    f"{metrics.get('Theil_U1', np.nan):<12.4f} {metrics.get('Theil_U2', np.nan):<12.4f} "
                    f"{metrics.get('MdAPE', np.nan):<10.2f} {metrics.get('MASE', np.nan):<9.4f} "
                    f"{metrics.get('R2', np.nan):<8.3f}")

    def run_complete_five_strategies_analysis(self, strategy='auto', forecast_horizon=8):
        """Run complete analysis using the five core theta strategies"""
        print("DSGE-INFORMED FIVE THETA STRATEGIES ANALYSIS")
        print("=" * 80)
        
        if self.dsge_priors is not None:
            print("✓ Using DSGE structural priors for VAR coefficient adaptation")
            print("✓ Preserving original factor loadings throughout")
            print("✓ Maintaining economic interpretation of factors")
        else:
            print("! No DSGE priors available - using econometric constraints only")

        # Step 1: Run five strategies comparison
        if strategy == 'auto':
            all_results = self.run_five_theta_strategies_comparison(forecast_horizon)
            if not all_results:
                print("No successful strategies found, analysis cannot continue")
                return None
                
            # Store results
            self.method_results = all_results

            # Select best strategy
            best_strategy, strategy_scores = self.select_best_strategy(all_results)

            # Set the selected strategy attributes
            self.theta_strategy = best_strategy
            if best_strategy in all_results:
                self.optimal_theta = all_results[best_strategy]['theta_params']
                
        else:
            # Run specific strategy
            strategy_map = {
                'frequency_dual': self.frequency_dual_theta_strategy,
                'lagged': self.lagged_theta_strategy,
                'sequential': self.sequential_theta_strategy,
                'constrained': self.constrained_theta_strategy,
                'global': self.global_theta_strategy,
            }
            
            if strategy in strategy_map:
                strategy_func = strategy_map[strategy]
                theta_params, theta_func = strategy_func()
                results = self.evaluate_theta_method(strategy.upper(), theta_params, theta_func, forecast_horizon)
                self.method_results = {strategy.upper(): results}
                self.theta_strategy = strategy.upper()
                self.optimal_theta = theta_params
            else:
                print(f"Unknown strategy: {strategy}")
                return None

        # Step 2: Print comprehensive results
        self.print_five_strategies_results()

        # Step 3: Generate enhanced forecasts
        if self.theta_strategy and self.theta_strategy in self.method_results:
            best_results = self.method_results[self.theta_strategy]
            
            print(f"\n\nDSGE-INFORMED FORECAST SUMMARY ({forecast_horizon} periods ahead):")
            print("="*90)
            
            forecasts_df = best_results['forecasts']
            
            print(f"\n{'Variable':<20} {'Latest Forecast':<15} {'CV RMSE':<10} {'Original R²':<12}")
            print("-" * 60)
            
            for var in forecasts_df.columns:
                latest_forecast = forecasts_df[var].iloc[-1]
                cv_rmse = best_results['cv_metrics'].get('RMSE', np.nan)
                fit_r2 = best_results['variable_r2'].get(var, 0.0)
                
                print(f"{var:<20} {latest_forecast:>14.2f} {cv_rmse:>9.4f} {fit_r2:>11.3f}")

        print("\nFIVE THETA STRATEGIES ANALYSIS COMPLETE")
        print("Strategies implemented:")
        print("• FREQUENCY_DUAL: Multi-frequency decomposition with trend/cyclical/high-freq weights")
        print("• LAGGED: Multi-lag factor transformation with current/lag-1/lag-2 weights")
        print("• SEQUENTIAL: Factor-by-factor sequential optimization with convergence")
        print("• CONSTRAINED: DSGE-constrained optimization with economic theory penalties")
        print("• GLOBAL: Multi-algorithm global optimization with ensemble metrics")

        return {
            'strategy_used': self.theta_strategy,
            'optimal_theta': self.optimal_theta,
            'method_results': self.method_results,
            'best_strategy_results': self.method_results.get(self.theta_strategy, {}),
            'forecasts': self.method_results.get(self.theta_strategy, {}).get('forecasts', None),
            'preserved_structure': True,
            'dsge_informed': self.dsge_priors is not None
        }


# ============================================================================
# CONVENIENCE FUNCTIONS FOR FIVE THETA STRATEGIES
# ============================================================================

def enhance_dsge_dfm_with_five_theta_strategies(existing_dsge_dfm_model, strategy='auto', forecast_horizon=8):
    """
    Enhance existing DSGE-informed DFM with the five core theta strategies
    
    Parameters:
    -----------
    existing_dsge_dfm_model : EconometricDFM
        Fitted DSGE-informed DFM model with Bayesian VAR and DSGE posterior estimates
    strategy : str
        'auto' - compare all five strategies and pick best
        Or specific: 'frequency_dual', 'lagged', 'sequential', 'constrained', 'global'
    forecast_horizon : int
        Number of quarters to forecast
    
    Returns:
    --------
    enhanced_dfm : DSGEInformedEnhancedDFMTheta
        DSGE-consistent theta-enhanced model
    results : dict
        Dictionary with all results preserving DSGE structure
    """
    
    print("ENHANCING DSGE-INFORMED DFM WITH FIVE THETA STRATEGIES")
    print("=" * 60)
    
    # Create DSGE-informed enhanced model
    enhanced_dfm = DSGEInformedEnhancedDFMTheta(existing_dsge_dfm_model)
    
    if enhanced_dfm.dsge_priors is not None:
        print("✓ DSGE priors successfully extracted - will be preserved throughout theta optimization")
        print(f"  Available DSGE parameters: {list(enhanced_dfm.dsge_priors.keys())}")
    else:
        print("! No DSGE priors found - using econometric constraints only")
    
    # Run complete analysis with the five strategies
    results = enhanced_dfm.run_complete_five_strategies_analysis(strategy, forecast_horizon)
    
    return enhanced_dfm, results


def compare_five_theta_strategies(existing_dsge_dfm_model):
    """
    Run comprehensive comparison of the five theta strategies
    
    Parameters:
    -----------
    existing_dsge_dfm_model : EconometricDFM
        Fitted DSGE-informed DFM model
    
    Returns:
    --------
    enhanced_dfm : DSGEInformedEnhancedDFMTheta
        Enhanced model object
    comparison_results : dict
        Comprehensive results with DSGE consistency metrics
    """
    
    enhanced_dfm = DSGEInformedEnhancedDFMTheta(existing_dsge_dfm_model)
    
    # Run strategies comparison
    all_results = enhanced_dfm.run_five_theta_strategies_comparison()
    enhanced_dfm.method_results = all_results
    
    enhanced_dfm.print_five_strategies_results()
    
    return enhanced_dfm, all_results


# ============================================================================
# EXAMPLE USAGE WITH FIVE THETA STRATEGIES
# ============================================================================

def example_five_theta_strategies_usage():
    """Complete example showing the five theta strategies"""
    print("FIVE THETA STRATEGIES EXAMPLE")
    print("=" * 50)
    
    print("\nThe Five Core Theta Strategies:")
    print("="*50)
    
    print("\n1. FREQUENCY_DUAL:")
    print("   • Decomposes factors into trend, cyclical, and high-frequency components")
    print("   • Optimizes weights: F_theta = θ1*trend + θ2*cyclical + θ3*high_freq")
    print("   • Best for: Capturing multiple time scales in economic data")
    
    print("\n2. LAGGED:")
    print("   • Uses multi-lag structure with current and lagged factor values")
    print("   • Optimizes weights: F_theta[t] = θ1*F[t] + θ2*F[t-1] + θ3*F[t-2]")
    print("   • Best for: Modeling temporal dependencies and persistence")
    
    print("\n3. SEQUENTIAL:")
    print("   • Factor-by-factor optimization with convergence checking")
    print("   • Iteratively optimizes each factor while holding others fixed")
    print("   • Best for: Handling factor interdependencies systematically")
    
    print("\n4. CONSTRAINED:")
    print("   • Incorporates DSGE structural constraints and economic theory")
    print("   • Penalizes violations of economic relationships")
    print("   • Best for: Maintaining theoretical consistency with DSGE models")
    
    print("\n5. GLOBAL:")
    print("   • Multi-algorithm global optimization with ensemble metrics")
    print("   • Uses differential evolution, basin hopping, dual annealing")
    print("   • Best for: Finding global optimum across complex parameter space")
    
    print("\nUsage Examples:")
    print("="*50)
    
    print("\n# Compare all five strategies and select best:")
    print("enhanced_dfm, results = enhance_dsge_dfm_with_five_theta_strategies(")
    print("    existing_dsge_dfm_model, strategy='auto')")
    
    print("\n# Use specific strategy:")
    print("enhanced_dfm, results = enhance_dsge_dfm_with_five_theta_strategies(")
    print("    existing_dsge_dfm_model, strategy='constrained')")
    
    print("\n# Compare all strategies without selection:")
    print("enhanced_dfm, results = compare_five_theta_strategies(existing_dsge_dfm_model)")


if __name__ == "__main__":
    example_five_theta_strategies_usage()

FIVE THETA STRATEGIES EXAMPLE

The Five Core Theta Strategies:

1. FREQUENCY_DUAL:
   • Decomposes factors into trend, cyclical, and high-frequency components
   • Optimizes weights: F_theta = θ1*trend + θ2*cyclical + θ3*high_freq
   • Best for: Capturing multiple time scales in economic data

2. LAGGED:
   • Uses multi-lag structure with current and lagged factor values
   • Optimizes weights: F_theta[t] = θ1*F[t] + θ2*F[t-1] + θ3*F[t-2]
   • Best for: Modeling temporal dependencies and persistence

3. SEQUENTIAL:
   • Factor-by-factor optimization with convergence checking
   • Iteratively optimizes each factor while holding others fixed
   • Best for: Handling factor interdependencies systematically

4. CONSTRAINED:
   • Incorporates DSGE structural constraints and economic theory
   • Penalizes violations of economic relationships
   • Best for: Maintaining theoretical consistency with DSGE models

5. GLOBAL:
   • Multi-algorithm global optimization with ensemble metrics
   • Uses 

In [20]:
enhanced_dfm, results = enhance_dsge_dfm_with_five_theta_strategies(model, strategy='auto')

ENHANCING DSGE-INFORMED DFM WITH FIVE THETA STRATEGIES
DSGE-Informed Enhanced DFM-Theta initialized:
  Factors: 3, Variables: 12
  DSGE priors: Available
  Base VAR estimation: Bayesian
✓ DSGE priors successfully extracted - will be preserved throughout theta optimization
  Available DSGE parameters: ['sigma', 'theta', 'phi_pi', 'phi_y', 'rho_a', 'sigma_a', 'rho_v', 'sigma_v', 'rho_d', 'sigma_d', 'rho_s', 'sigma_s']
DSGE-INFORMED FIVE THETA STRATEGIES ANALYSIS
✓ Using DSGE structural priors for VAR coefficient adaptation
✓ Preserving original factor loadings throughout
✓ Maintaining economic interpretation of factors

FIVE CORE THETA STRATEGIES COMPARISON
Using Bayesian VAR with DSGE structure preservation

--- FREQUENCY_DUAL Strategy ---
FREQUENCY_DUAL Strategy: Optimizing dual frequency decomposition with robustness...
Evaluating FREQUENCY_DUAL with DSGE consistency...
Average R²: 0.664
CV RMSE: 0.6443
DSGE-Informed: Yes
Original Loadings Preserved: True

--- LAGGED Strategy ---
LAGG